In [ ]:
# Creates all plots in the paper.

In [2]:
import matplotlib.pyplot as plt
import numpy
os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2015/bin/x86_64-darwin'
print(os.getenv("PATH"))
plt.rc('text', usetex=True)
plt.rcParams["text.usetex"] = True

/Applications/SageMath-9.1.app/Contents/Resources/sage/local/libexec/ccache:/Applications/SageMath-9.1.app/Contents/Resources/sage/build/bin:/Applications/SageMath-9.1.app/Contents/Resources/sage/src/bin:/Applications/SageMath-9.1.app/Contents/Resources/sage/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/texlive/2015/bin/x86_64-darwin


In [3]:


# Find maximum fidelity (with input F = 0.7) and corresponding symbolic expression for the Fidelity
from itertools import product as iterprod
import numpy as np

def find_F_max(polies):
    Fmax = 0
    for i in polies:
        p, fid = symb_poly(i)
    #     G = var('G')
    #     k = fid.substitute(F=(1-G))
    #     series = k.taylor(G, 0, 4)
    #     coef = series.coefficients(sparse=False)
        Fnum = fid(F = 0.7)

        if Fnum > Fmax:
            Fmax = Fnum
            fidmax = fid
    print('---')
    print('Max fidelity = ', Fmax)
    print('Max fidelity (symbolic) =', fidmax)
    
def find_specific_F(polies, Ftarget):
    for i in polies:
        p, fid = symb_poly(i)

        Fnum = round(fid(F=0.7), 4)
        if Fnum == Ftarget:
            return i
    print(af)
    
def create_M_from_triple(E, a, b, N):
    M = matrix.identity(2*N)
    
    M[1:int(N), 0] = a.T
    M[N, (N+1):(2*N)] = a
    M[1:int(N), N] = b.T
    M[0, (N+1):(2*N)] = b
    M[1:N, (N+1):(2*N)] = E + b.T*a
    return M 
    
def create_all_basis(N):
    meas_qubits_I_Z = iterprod([[0], [1]], repeat=int(N-1))
    Id1 = (N+1)*[0]
    X1 = (N+1)*[0]
    Y1 = (N+1)*[0]
    Z1 = (N+1)*[0]
    
    basis1 = []
    basis2 = []
    basis3 = []
    basis4 = []
    
    X1[0] = 1
    Y1[0] = 1
    Y1[N] = 1
    Z1[N] = 1
    
    for i in meas_qubits_I_Z:
        vec = []
        for j in i:
            vec += j
        basis1.append(Id1 + vec)
        basis2.append(X1 + vec)
        basis3.append(Y1 + vec)
        basis4.append(Z1 + vec)

    return (basis1, basis2, basis3, basis4)

def get_all_statistics(M, N, all_basis):
    coeff = []
    Minv = get_M_inv(M, N)
    for i in range(4):
        basis = all_basis[i]
        polycoeff = int(N+1)*[0]
        for vec in basis:
            k = get_k(np.matmul(M, vec), N)
            polycoeff[k] += 1
        coeff.append(polycoeff)
    myprob = [a + b + c + d for a, b, c, d in zip(coeff[0], coeff[1], coeff[2], coeff[3])]
    return [symb_poly_coeff(c)/symb_poly_coeff(myprob) for c in coeff], symb_poly_coeff(myprob)

def get_M_inv(M, N):
    M_inv = 0*matrix.identity(2*N)
    Amatrix = M[0:N, 0:N]
    Bmatrix = M[0:N, (N):(2*N)]
    Cmatrix = M[(N):(2*N), 0:N]
    Dmatrix = M[(N):(2*N), (N):(2*N)]
    
    M_inv[0:N, 0:N] = Dmatrix.transpose()
    M_inv[(N):(2*N), (N):(2*N)] = Amatrix.transpose()
    M_inv[0:N, (N):(2*N)] = Bmatrix.transpose()
    M_inv[(N):(2*N), 0:N] = Cmatrix.transpose()
    return M_inv

def get_k(vec, N):
#     print(vec)
    k = 0
    for i in range(N):
        if vec[i]%2 == 0 and vec[N+i]%2 == 0:
            k += 1
    return k

def symb_poly_coeff(poly):
    n = len(poly) - 1
    F = var('F')
    
    num = 0
    
    for k in range(n+1):
        num += poly[k]*(F**k)*((1-F)/3)**(n-k)
    return num


In [4]:
def symb_poly(poly):
    p, q = poly
    n = len(p) - 1
    F = var('F')
    
    prob = 0
    num = 0
    
    for k in range(n+1):
        prob += q[k]*(F**k)*((1-F)/3)**(n-k)
        num += p[k]*(F**k)*((1-F)/3)**(n-k)
    fidelity = num/prob
    
    return prob, fidelity


In [19]:
def load_data():
    polies2 = load('p+F+a+b+B+n2.sobj')
    polies3 = load('p+F+a+b+B+n3.sobj')
    polies4 = load('p+F+a+b+B+n4.sobj')
    polies5 = load('p+F+a+b+B+n5.sobj')
    polies6 = load('p+F+a+b+B+n6.sobj')
    polies7 = load('p+F+a+b+B+n7.sobj')
    polies8 = load('p+F+a+b+B+n8.sobj')

    list_of_polies = [polies2, polies3, polies4, polies5, polies6, polies7, polies8]
    list_of_n = [2, 3, 4, 5, 6, 7, 8]
    
#     list_of_polies = [polies2, polies3, polies4]
#     list_of_n = [2, 3, 4]
    assert len(list_of_polies) == len(list_of_n)
    
    rgbcolors=[(0+x/(len(list_of_n)-1), 1-x/(len(list_of_n)-1), 1) for x in range(len(list_of_n))]
    return list_of_polies, list_of_n, rgbcolors

def load_data2():
    polies4 = load('p+F+a+b+B+n4.sobj')
    polies5 = load('p+F+a+b+B+n5.sobj')
    polies6 = load('p+F+a+b+B+n6.sobj')
    polies7 = load('p+F+a+b+B+n7.sobj')
    polies8 = load('p+F+a+b+B+n8.sobj')

    list_of_polies = [polies4, polies5, polies6, polies7, polies8]
    list_of_n = [4, 5, 6, 7, 8]
    
#     list_of_polies = [polies2, polies3, polies4]
#     list_of_n = [2, 3, 4]
    assert len(list_of_polies) == len(list_of_n)
    
    rgbcolors=[(0+x/(len(list_of_n)-1), 1-x/(len(list_of_n)-1), 1) for x in range(len(list_of_n))]
    return list_of_polies, list_of_n, rgbcolors

# all optimal dejmps protocols
def load_dejmps_data():
    polydejmps2 = ((1, 0, 1), (5, 2, 1))
    polydejmps3 = ((2, 1, 0, 1), (7, 7, 1, 1))
    polydejmps4 = ((5, 0, 2, 0, 1), (13, 8, 10, 0, 1))
    polydejmps5 = ((6, 5, 2, 2, 0, 1), (18, 23, 14, 8, 0, 1))
    polydejmps6 = ((5, 16, 7, 0, 3, 0, 1), (31, 40, 27, 24, 5, 0, 1))
    polydejmps7 = ((0, 47, 0, 11, 0, 5, 0, 1), (60, 47, 96, 11, 36, 5, 0, 1))
    polydejmps8 = ((25, 0, 84, 0, 14, 0, 4, 0, 1), (89, 80, 212, 32, 78, 16, 4, 0, 1))
    return [polydejmps2, polydejmps3, polydejmps4, polydejmps5, polydejmps6, polydejmps7, polydejmps8]

def load_dejmps_data2():
    polydejmps4 = ((5, 0, 2, 0, 1), (13, 8, 10, 0, 1))
    polydejmps5 = ((6, 5, 2, 2, 0, 1), (18, 23, 14, 8, 0, 1))
    polydejmps6 = ((5, 16, 7, 0, 3, 0, 1), (31, 40, 27, 24, 5, 0, 1))
    polydejmps7 = ((0, 47, 0, 11, 0, 5, 0, 1), (60, 47, 96, 11, 36, 5, 0, 1))
    polydejmps8 = ((25, 0, 84, 0, 14, 0, 4, 0, 1), (89, 80, 212, 32, 78, 16, 4, 0, 1))
    return [polydejmps4, polydejmps5, polydejmps6, polydejmps7, polydejmps8]

In [28]:
def calc_rate_actual(E, a, b, N, all_basis):
    M = create_M_from_triple(E, a, b, N)
    coeffs, p = get_all_statistics(M, N, all_basis)
#     return (1-calc_entropy_coeffs(coeffs))
#     return p*(1-calc_entropy_coeffs(coeffs))/n
    return max_symbolic(p*(1-calc_entropy_coeffs(coeffs))/N, 0)

def calc_entropy_coeffs(coeffs):
    entropy = 0
    for coeff in coeffs:
        if coeff == 1 or coeff == 0:
            return 0
        entropy += -coeff*log(coeff)/log(2)
    return entropy

def bin_entropy(p):
    if p == 0:
        return 0
    return -p*log(p)/log(2)-(1-p)*log(1-p)/log(2)


def plot_all_rates_zoom2():
    list_of_polies, list_of_n, rgbcolors = load_data()
    
    
    rates = plot_semilogy([])
#     list_of_n = list_of_n[0:3]
    list_of_rates = []
    for i in range(len(list_of_n)):
        polies = list_of_polies[i]
        n = list_of_n[i]
        print(n)
#         if n > 5:
#             break
        all_basis = create_all_basis(n)
        for triple in polies.values():
            E, a, b = triple
            list_of_rates.append(calc_rate_actual(matrix(GF(2), E), matrix(GF(2), a), matrix(GF(2), b), n, all_basis))
    max_rate = max_symbolic(list_of_rates)
    rates += plot_semilogy(max_rate, (1/2, 1), rgbcolor=(0.96, 1/9, 1/9), fontsize=16, thickness=2, legend_label='Optimised', figsize=(6.4, 4.8))
    rates.set_axes_range(xmin=1/2, xmax=1, ymin=1e-4, ymax=1/2)
#     save(rates,'max_all_rates_zoom2.pdf',axes=True, axes_labels=['$F_{in}$','Rate'],axes_labels_size=1.6, legend_font_size=16)

    polydejmps2 = find_specific_F(list_of_polies[0], 0.7353)
    polydejmps3 = find_specific_F(list_of_polies[1], 0.7857)
    polydejmps4 = find_specific_F(list_of_polies[2], 0.8459)
    polydejmps5 = find_specific_F(list_of_polies[3], 0.8636)
    polydejmps6 = find_specific_F(list_of_polies[4], 0.8982)
    polydejmps7 = find_specific_F(list_of_polies[5], 0.9091)
    polydejmps8 = find_specific_F(list_of_polies[6], 0.9344)

    list_of_polies, list_of_n, rgbcolors = load_data()

    dejmps_triples = [polydejmps2, polydejmps3, polydejmps4, polydejmps5, polydejmps6, polydejmps7, polydejmps8]
    list_of_rates = []
    for i in range(len(dejmps_triples)):
    #         polies = list_of_polies[i]
        n = list_of_n[i]
        all_basis = create_all_basis(n)

        E, a, b = list_of_polies[i][dejmps_triples[i]]
        dejmps_rate = calc_rate_actual(matrix(GF(2), E), matrix(GF(2), a), matrix(GF(2), b), n, all_basis)
        list_of_rates.append(dejmps_rate)

    max_rate = max_symbolic(list_of_rates)
    rates += plot_semilogy(max_rate, (1/2, 1), linestyle=':', fontsize=16, thickness=2.5, rgbcolor=(4/5, 1/7, 1/7), figsize=(6.4, 4.8), legend_label='Conc. DEJMPS')
    rates.set_axes_range(xmin=1/2, xmax=1, ymin=1e-4, ymax=1/2)
    save(rates,'max_all_rates_zoom3.pdf',axes=True, axes_labels=[r'$F_\textrm{in}$','Rate'],axes_labels_size=1.6, legend_font_size=16)

def plot_all_fidelities():
    list_of_polies, list_of_n, rgbcolors = load_data()
    
    fids = plot([])
    for i in range(len(list_of_n)):
        polies = list_of_polies[i]
        n = list_of_n[i]
        list_of_fids = [symb_poly(poly)[1] for poly in polies]
        max_fidelity = max_symbolic(list_of_fids)
        print(n)
        print(max_fidelity(0.7))
        fids += plot(max_fidelity-F, (1/2, 1), rgbcolor=rgbcolors[i], legend_label='n = ' + str(n))
    
    data = load_dejmps_data()
    for i, d in enumerate(data):
        max_fid = symb_poly(d)[1]
        fids += plot(max_fid-F, (1/2, 1), linestyle=':', rgbcolor=rgbcolors[i])
    fids.set_axes_range(xmin=1/2, xmax=1, ymin=0, ymax=0.285)
    save(fids,'max_all_fidelities.pdf',axes=True, axes_labels=[r'$F_\textrm{in}$',r'$F_\textrm{out}$'])
    
def plot_all_fidelities_zoom():
    list_of_polies, list_of_n, rgbcolors = load_data2()
    Fmin = 0.9
    fids = plot_semilogy([])
    for i in range(len(list_of_n)):
        polies = list_of_polies[i]
        n = list_of_n[i]
        list_of_fids = [1-symb_poly(poly)[1] for poly in polies]
        max_fidelity = min_symbolic(list_of_fids)
        fids += plot_semilogy(max_fidelity, (Fmin, 1), rgbcolor=rgbcolors[i], legend_label='n = ' + str(n))
    
    data = load_dejmps_data2()
    for i, d in enumerate(data):
        max_fid = 1-symb_poly(d)[1]
        fids += plot_semilogy(max_fid, (Fmin, 1), linestyle=':', thickness=1.6, rgbcolor=rgbcolors[i])
#     fids.set_axes_range(xmin=Fmin, xmax=1)
    fids.set_axes_range(xmin=Fmin, xmax=1, ymin=9e-6, ymax=0.02)
    save(fids,'max_all_fidelities_zoom.pdf',axes=True, axes_labels=[r'$F_\textrm{in}$',r'$1-F_\textrm{out}$'])
    

In [29]:
#Create figs 5, 6 and 7 (takes a long time to run).

plot_all_rates_zoom2()
plot_all_fidelities()
plot_all_fidelities_zoom()

2
0.735294117647059
3
0.785714285714286
4
0.863636363636364
5

/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python2.7/site-packages/sage/repl/ipython_kernel/__main__.py:4: DeprecationWarning: Substitution using function-call syntax and unnamed arguments is deprecated and will be removed from a future release of Sage; you can use named arguments instead, like EXPR(x=..., y=...)
See http://trac.sagemath.org/5930 for details.



0.919130434782609
6
0.926326129666012
7
0.945287423402393
8
0.956448345712356


In [30]:
# Creates scatter plot (fig 4)

sp2 = [0.75, 0.65, 0.71, 0.625, 0.8, 0.675, 0.68, 0.85, 0.745]
fid2 = [0.7, 0.7615384615, 0.7112676056, 0.788, 0.7, 0.737037037, 0.7352941176, 0.7, 0.6879194631]

sort2 = list(np.argsort(sp2))
sp2= [sp2[i] for i in sort2] 
fid2= [fid2[i] for i in sort2] 

sp3 = [0.47225, 0.5325, 0.4385, 0.4625, 0.68, 0.57375, 0.50625, 0.444, 0.54, 0.42375, 0.4625, 0.4875, 0.42125, 0.5, 0.4325, 0.568, 0.51, 0.5175, 0.46175, 0.6, 0.42625, 0.5225, 0.43625, 0.52, 0.4625, 0.46875, 0.43875, 0.4635, 0.44875, 0.4405, 0.63325, 0.55, 0.45975, 0.596, 0.4435, 0.4695, 0.46325, 0.5525, 0.46875, 0.4275, 0.49, 0.7225, 0.4875, 0.464, 0.6375, 0.47925, 0.53125, 0.4455, 0.578, 0.44, 0.515, 0.5825, 0.64, 0.4555, 0.46125, 0.4595, 0.445, 0.4275, 0.451, 0.5625, 0.41875, 0.544, 0.55875, 0.4525, 0.4475, 0.5475, 0.4175, 0.6035, 0.4775, 0.525, 0.4375, 0.6175, 0.52]
fid3 = [0.7541026998, 0.7112676056, 0.7930444698, 0.7605405405, 0.7, 0.737037037, 0.737037037, 0.7882882883, 0.737037037, 0.8209439528, 0.7778378378, 0.7287179487, 0.8252225519, 0.788, 0.8121387283, 0.7112676056, 0.7352941176, 0.7, 0.7598808879, 0.7, 0.8190615836, 0.7, 0.811747851, 0.7615384615, 0.7616216216, 0.788, 0.8076923077, 0.7578209277, 0.8008356546, 0.7917139614, 0.6879194631, 0.6745454545, 0.7610114192, 0.6879194631, 0.7874859076, 0.7534611289, 0.7568807339, 0.7615384615, 0.7581333333, 0.8169590643, 0.7285714286, 0.7, 0.7615384615, 0.775862069, 0.7, 0.7524778299, 0.788, 0.7861952862, 0.7352941176, 0.7909090909, 0.7, 0.6519313305, 0.7, 0.7799121844, 0.7579945799, 0.7600652884, 0.7898876404, 0.8204678363, 0.7815964523, 0.7, 0.8271641791, 0.7352941176, 0.6879194631, 0.7839779006, 0.8027932961, 0.6744292237, 0.8293413174, 0.7112676056, 0.7544502618, 0.7, 0.796, 0.6319838057, 0.7]

sort3 = list(np.argsort(sp3))
sp3= [sp3[i] for i in sort3] 
fid3= [fid3[i] for i in sort3] 

sp4 = [0.281575, 0.284, 0.36175, 0.293875, 0.313275, 0.378675, 0.478125, 0.289625, 0.306675, 0.3395, 0.288675, 0.3694, 0.3685, 0.3040625, 0.2949375, 0.2898125, 0.352125, 0.31505, 0.287375, 0.3186, 0.282625, 0.307, 0.3774, 0.282375, 0.3226, 0.399075, 0.3459375, 0.297, 0.2838125, 0.322875, 0.3123125, 0.405, 0.278, 0.390575, 0.439875, 0.3524, 0.436875, 0.357675, 0.2818, 0.2872875, 0.306675, 0.3369375, 0.3405, 0.33455, 0.3892625, 0.410625, 0.282975, 0.290475, 0.2834375, 0.308125, 0.312, 0.3152, 0.3016, 0.4303125, 0.326425, 0.298375, 0.3815625, 0.305925, 0.3221875, 0.3018, 0.318475, 0.438, 0.3944, 0.304925, 0.3552, 0.36225, 0.3463125, 0.4190625, 0.298125, 0.2835875, 0.33375, 0.51, 0.308375, 0.3064875, 0.3937625, 0.39, 0.316875, 0.310175, 0.37, 0.279375, 0.367425, 0.332625, 0.29595, 0.4633, 0.3016875, 0.3090625, 0.308525, 0.320625, 0.344, 0.304, 0.356625, 0.334875, 0.2799375, 0.341625, 0.2898125, 0.291625, 0.414, 0.290825, 0.36565, 0.314125, 0.3274375, 0.316, 0.324625, 0.3358125, 0.2895625, 0.2961875, 0.372125, 0.3756, 0.444125, 0.3548, 0.2814375, 0.280675, 0.376975, 0.469625, 0.3644, 0.317, 0.2894875, 0.3018125, 0.2989375, 0.357875, 0.3187, 0.392, 0.418, 0.3245, 0.3385, 0.299925, 0.337875, 0.393975, 0.320125, 0.311875, 0.300125, 0.2843125, 0.2831625, 0.392, 0.2892625, 0.312, 0.3343125, 0.325125, 0.3202375, 0.2830625, 0.323025, 0.399375, 0.4335, 0.3115625, 0.307625, 0.3170375, 0.3, 0.299875, 0.4749375, 0.3087, 0.292625, 0.355, 0.391875, 0.325375, 0.33, 0.2779625, 0.2974375, 0.2783125, 0.412, 0.322125, 0.31475, 0.3515625, 0.31535, 0.3907875, 0.383675, 0.278925, 0.2945625, 0.2834875, 0.327, 0.305875, 0.282825, 0.288875, 0.351, 0.301875, 0.309, 0.302975, 0.4828, 0.524875, 0.3273125, 0.329625, 0.318, 0.3920625, 0.303925, 0.3328125, 0.3250375, 0.308275, 0.337125, 0.359, 0.2831875, 0.324625, 0.326725, 0.314625, 0.292375, 0.332925, 0.3712, 0.2808125, 0.3167625, 0.328, 0.295875, 0.357, 0.2826875, 0.317125, 0.3623125, 0.34125, 0.387175, 0.3327, 0.328125, 0.3429375, 0.336375, 0.3834, 0.3389, 0.3580625, 0.365625, 0.3011, 0.332375, 0.3365, 0.304275, 0.291125, 0.2798125, 0.349, 0.3051875, 0.3228125, 0.3676, 0.301375, 0.45, 0.339, 0.2891875, 0.283975, 0.2864375, 0.3515625, 0.5066, 0.2946875, 0.3774375, 0.360925, 0.3055, 0.2868125, 0.3261, 0.35515, 0.283, 0.2995, 0.324425, 0.3358125, 0.283225, 0.2818375, 0.305025, 0.3290625, 0.318775, 0.30955, 0.3049375, 0.316525, 0.3234375, 0.307875, 0.281625, 0.2969125, 0.320175, 0.3265625, 0.37375, 0.32275, 0.2798625, 0.342, 0.319475, 0.305625, 0.37425, 0.306625, 0.292875, 0.345775, 0.5382625, 0.313325, 0.318125, 0.333125, 0.291, 0.307375, 0.284625, 0.365875, 0.3375, 0.312375, 0.3924875, 0.3178125, 0.291875, 0.2939375, 0.3984375, 0.305575, 0.299675, 0.3675, 0.3346125, 0.2917125, 0.2816, 0.3048, 0.358375, 0.366125, 0.442, 0.2825375, 0.31075, 0.3546875, 0.303425, 0.30725, 0.284375, 0.3337375, 0.3150625, 0.280725, 0.318875, 0.339625, 0.3063375, 0.3365625, 0.324875, 0.319875, 0.3548625, 0.37825, 0.358125, 0.347625, 0.2838, 0.2837625, 0.301525, 0.335625, 0.3271875, 0.323175, 0.301125, 0.330375, 0.307525, 0.2973125, 0.465375, 0.3608, 0.3200625, 0.3778, 0.3118, 0.318375, 0.300625, 0.3140625, 0.391125, 0.382125, 0.346875, 0.3133125, 0.39, 0.3265, 0.3984375, 0.282175, 0.334, 0.302, 0.3389375, 0.2969375, 0.4095, 0.347625, 0.3625, 0.294875, 0.319125, 0.354375, 0.3241625, 0.283425, 0.466, 0.327125, 0.2877125, 0.372, 0.3559375, 0.4364, 0.2805875, 0.2990875, 0.365625, 0.345625, 0.3366875, 0.541875, 0.459, 0.3601875, 0.4515625, 0.3149125, 0.336, 0.4352, 0.432, 0.393125, 0.3247, 0.2902, 0.3008375, 0.3706, 0.512975, 0.3814375, 0.31125, 0.447, 0.31005, 0.354875, 0.38625, 0.4675, 0.2842125, 0.3553, 0.359125, 0.2804375, 0.339375, 0.38335, 0.324625, 0.341, 0.3194375, 0.3729375, 0.320875, 0.33595, 0.2933125, 0.3708125, 0.286075, 0.2854875, 0.2996875, 0.2968125, 0.2909375, 0.3015625, 0.292225, 0.3029375, 0.2803375, 0.3519, 0.3384375, 0.3587, 0.339375, 0.3541875, 0.2925, 0.44625, 0.340125, 0.3526875, 0.313425, 0.346875, 0.39375, 0.3493125, 0.3167875, 0.299375, 0.327625, 0.2958125, 0.3054375, 0.388125, 0.294375, 0.361975, 0.434625, 0.295375, 0.276825, 0.303375, 0.279625, 0.2802375, 0.356, 0.2817375, 0.318625, 0.286375, 0.2865375, 0.2842625, 0.31, 0.318, 0.3159375, 0.314725, 0.381875, 0.34, 0.2965, 0.2930625, 0.319625, 0.291125, 0.3258125, 0.283625, 0.2874375, 0.42, 0.3448125, 0.312, 0.344125, 0.342, 0.310625, 0.294075, 0.363375, 0.320225, 0.301375, 0.290125, 0.393125, 0.3564, 0.307225, 0.2965625, 0.2780625, 0.358, 0.3339125, 0.335, 0.31725, 0.374425, 0.3176875, 0.2870625, 0.2952375, 0.3066, 0.295175, 0.44, 0.363375, 0.3414375, 0.33425, 0.3708, 0.578, 0.2913125, 0.293375, 0.4125, 0.3230625, 0.374, 0.3229375, 0.324, 0.288375, 0.313, 0.310875, 0.3636, 0.408, 0.380375, 0.325125, 0.414375, 0.322875, 0.324375, 0.358125, 0.384625, 0.3046875, 0.37, 0.369, 0.3274125, 0.39, 0.452625, 0.321525, 0.2812125, 0.3319375, 0.3058, 0.32, 0.288375, 0.30635, 0.544, 0.334125, 0.4876875, 0.37275, 0.33475, 0.3048625, 0.2926125, 0.3825125, 0.298275, 0.4624, 0.3196875, 0.294, 0.3124375, 0.43775, 0.328375, 0.2961, 0.315775, 0.288, 0.346, 0.367625, 0.341525, 0.3508, 0.344625, 0.376875, 0.299375, 0.3232625, 0.36, 0.332025, 0.3798125, 0.375, 0.300025, 0.305375, 0.425, 0.341625, 0.3855375, 0.307125, 0.3594375, 0.3365625, 0.36165, 0.301625, 0.416, 0.333, 0.405875, 0.295, 0.3081625, 0.384375, 0.29245, 0.375, 0.2849375, 0.4, 0.278875, 0.321175, 0.36, 0.3016, 0.303125, 0.303625, 0.324525, 0.348, 0.3345, 0.319875, 0.3426125, 0.372725, 0.37, 0.3023625, 0.326625, 0.320375, 0.33825, 0.292225, 0.3419, 0.3251, 0.285625, 0.293625, 0.2938125, 0.2930625, 0.2846125, 0.380375, 0.315, 0.362, 0.426, 0.290125, 0.3140375, 0.3103125, 0.303625, 0.328375, 0.48, 0.355875, 0.3187375, 0.322075, 0.326, 0.327075, 0.3678, 0.309725, 0.314825, 0.3474375, 0.298475, 0.2801875, 0.614125, 0.313225, 0.2965625, 0.324375, 0.313125, 0.298375, 0.299875, 0.3225125, 0.5225125, 0.3422, 0.2819125, 0.321875, 0.3758, 0.359875, 0.4014125, 0.3244375, 0.292875, 0.2825125, 0.3675, 0.370975, 0.381725, 0.3351875, 0.4913, 0.382, 0.3298625, 0.342525, 0.3436875, 0.358625, 0.3155625, 0.495125, 0.3003125, 0.283975, 0.352, 0.3825, 0.3756875, 0.31325, 0.3009375, 0.2845625, 0.332775, 0.337, 0.2826, 0.414375, 0.319, 0.2876875, 0.328875, 0.346875, 0.31375, 0.33, 0.4073625, 0.2803625, 0.301875, 0.2958125, 0.4768, 0.286875, 0.2853625, 0.4614375, 0.303125, 0.3088, 0.3198875, 0.320325, 0.334625, 0.284025, 0.2985, 0.298125, 0.3135, 0.4544, 0.3078125, 0.314725, 0.286, 0.344475, 0.3362, 0.4078125, 0.3502, 0.3796875, 0.3503125, 0.3433, 0.4165, 0.2835125, 0.3828, 0.393125, 0.301175, 0.378225, 0.442, 0.4112, 0.2968625, 0.33525, 0.319675, 0.324025, 0.29965, 0.421875, 0.3842625, 0.35, 0.328125, 0.416, 0.29, 0.3448, 0.320625, 0.371875, 0.303375, 0.3669375, 0.305, 0.31425, 0.491975, 0.356, 0.463125, 0.386, 0.356125, 0.297625, 0.340625, 0.494, 0.512, 0.350375, 0.2874375, 0.3609375, 0.367025, 0.3203125]
fid4 = [0.8636686496, 0.8591549296, 0.7, 0.8347511697, 0.7974223925, 0.7861952862, 0.7, 0.846741476, 0.8012146409, 0.740353461, 0.8546375682, 0.7598808879, 0.7, 0.8211510791, 0.8487179487, 0.846430882, 0.7534611289, 0.7947151246, 0.849630274, 0.7736974262, 0.862406015, 0.8061889251, 0.7882882883, 0.8625498008, 0.7687538748, 0.7534611289, 0.7579945799, 0.8286195286, 0.8593922044, 0.8009678668, 0.8125075045, 0.737037037, 0.8726618705, 0.7600652884, 0.7, 0.7917139614, 0.6519313305, 0.719123506, 0.8619588361, 0.8550450333, 0.8361865167, 0.7396586904, 0.7408223201, 0.777238081, 0.6879194631, 0.6744292237, 0.8589539712, 0.8496858594, 0.8572216097, 0.821703854, 0.7923076923, 0.7766497462, 0.8116710875, 0.737037037, 0.7669066401, 0.8256807708, 0.6793939394, 0.8021165318, 0.7656838021, 0.8114645461, 0.7715283774, 0.6744292237, 0.775862069, 0.8070427154, 0.7882882883, 0.7, 0.7598808879, 0.6879194631, 0.8435639413, 0.8596773483, 0.7898876404, 0.7, 0.7982569923, 0.8328031323, 0.7568807339, 0.7, 0.7915187377, 0.8042637221, 0.7778378378, 0.8693064877, 0.7112676056, 0.7874859076, 0.8460043926, 0.6165551479, 0.8367723224, 0.8195146613, 0.7995705372, 0.8169590643, 0.7465116279, 0.8052631579, 0.7143007361, 0.7447928331, 0.8674034383, 0.7799121844, 0.8450506793, 0.8538791256, 0.7, 0.8521017794, 0.7112676056, 0.7967767608, 0.7664630655, 0.8012658228, 0.7834809395, 0.7525218686, 0.8427584718, 0.8403249631, 0.7, 0.7534611289, 0.7, 0.7874859076, 0.864023984, 0.8651019863, 0.7874859076, 0.7615384615, 0.7799121844, 0.7826498423, 0.8523468198, 0.8247049078, 0.8249843195, 0.7, 0.7949482272, 0.7, 0.7, 0.7862865948, 0.7398818316, 0.8388347087, 0.7402515723, 0.7578209277, 0.7849668098, 0.7923446894, 0.8306122449, 0.8587601671, 0.857568534, 0.7285714286, 0.8496823819, 0.8012820513, 0.7476724621, 0.7830449827, 0.7712049651, 0.8576948554, 0.7861233651, 0.7112676056, 0.7352941176, 0.8134202608, 0.7991466883, 0.7752040374, 0.8243333333, 0.8264693622, 0.6879194631, 0.7993197279, 0.8390858607, 0.7163380282, 0.7, 0.7617748751, 0.7690909091, 0.872532266, 0.8279680605, 0.8722434314, 0.7, 0.7708575863, 0.7908657665, 0.7581333333, 0.7941176471, 0.7610114192, 0.6879194631, 0.8697230438, 0.8369403777, 0.8598042242, 0.7593272171, 0.827176134, 0.8620613454, 0.8460839463, 0.8076923077, 0.8245548654, 0.7964401294, 0.8088538658, 0.7112676056, 0.6319838057, 0.7590796257, 0.7584755404, 0.7880503145, 0.7579945799, 0.8067368594, 0.7572018779, 0.7671999385, 0.7996512854, 0.7504263997, 0.8008356546, 0.8601853895, 0.7847131305, 0.7670441503, 0.7769169646, 0.8433946131, 0.7571900578, 0.775862069, 0.8660805698, 0.7751272641, 0.7780487805, 0.8324883819, 0.7352941176, 0.860822463, 0.7800945999, 0.8190615836, 0.7615384615, 0.7799121844, 0.7777276826, 0.788, 0.7485875706, 0.7615384615, 0.7524778299, 0.7502213042, 0.8252225519, 0.7615384615, 0.8121886416, 0.7646859722, 0.7448736999, 0.8062197026, 0.8486474882, 0.8688184052, 0.811747851, 0.8198033995, 0.7651694095, 0.7600652884, 0.8104935711, 0.7, 0.8209439528, 0.8457964124, 0.8576899375, 0.8520837879, 0.788, 0.6879194631, 0.8366277837, 0.6866534194, 0.7150723835, 0.8024549918, 0.8514491175, 0.7637227844, 0.7183584401, 0.8604240283, 0.8138564274, 0.7639284889, 0.7453750233, 0.858813664, 0.8620880827, 0.8069420539, 0.8076923077, 0.7960552114, 0.799628493, 0.8199016192, 0.7919990522, 0.788, 0.8221274868, 0.863692854, 0.8435355534, 0.7700866714, 0.788, 0.7, 0.7749806352, 0.8671445799, 0.8204678363, 0.7703654433, 0.817791411, 0.7, 0.8001222992, 0.8386256936, 0.769467139, 0.6879194631, 0.7793425357, 0.7716699411, 0.7570356473, 0.8436426117, 0.8031313542, 0.8542380325, 0.7, 0.7453333333, 0.7918767507, 0.7598808879, 0.8209439528, 0.8451820128, 0.8345949394, 0.7581333333, 0.8351059478, 0.8135897222, 0.7285714286, 0.7617953603, 0.8496593392, 0.8622159091, 0.8070866142, 0.708650157, 0.7, 0.7615384615, 0.860837057, 0.7984714401, 0.7503612335, 0.8085605998, 0.8061025224, 0.8547252747, 0.7614330125, 0.7766117834, 0.8645026271, 0.7959623677, 0.7615384615, 0.83312931, 0.8008356546, 0.7843401308, 0.7750293083, 0.7500968685, 0.7898876404, 0.7544502618, 0.737037037, 0.8594080338, 0.8564600678, 0.8384462317, 0.8027932961, 0.811747851, 0.7858358475, 0.8295143213, 0.7917139614, 0.8005446712, 0.8457641371, 0.6744292237, 0.7815964523, 0.7722905682, 0.7541026998, 0.8014753047, 0.7800942285, 0.8297297297, 0.8271641791, 0.6879194631, 0.7, 0.7616216216, 0.7792738879, 0.7615384615, 0.7633996937, 0.788, 0.8628067688, 0.8293413174, 0.8364238411, 0.7535865757, 0.8285203115, 0.6553113553, 0.7578209277, 0.7063448276, 0.833615939, 0.7946337642, 0.737037037, 0.7713145413, 0.8585604657, 0.6519313305, 0.7666411922, 0.8573011253, 0.7, 0.8271641791, 0.6347387718, 0.8663741257, 0.8410373219, 0.7287179487, 0.7697287523, 0.7898830518, 0.7, 0.737037037, 0.8209439528, 0.788, 0.7778549597, 0.7476190476, 0.7352941176, 0.737037037, 0.7778378378, 0.7845703727, 0.8511371468, 0.8442057589, 0.7568807339, 0.7112676056, 0.8008356546, 0.798313253, 0.6879194631, 0.7994678278, 0.8293413174, 0.7, 0.6745454545, 0.8588863966, 0.7352941176, 0.7094674556, 0.8667483842, 0.7533333333, 0.7815964523, 0.8005005776, 0.8190615836, 0.7693797691, 0.8076923077, 0.7733151539, 0.7604554249, 0.8532069039, 0.811747851, 0.8550642314, 0.8600858181, 0.8216684046, 0.8396083386, 0.8562620838, 0.8118756477, 0.852895885, 0.833897256, 0.8665224952, 0.7352941176, 0.7374145891, 0.7620574296, 0.7839779006, 0.7541026998, 0.8514529915, 0.7, 0.7458654906, 0.737037037, 0.7948871341, 0.7605405405, 0.7, 0.737037037, 0.7752633863, 0.8195824635, 0.7784433422, 0.8299387281, 0.8354205034, 0.7, 0.8328237792, 0.719835624, 0.6344262295, 0.8330512061, 0.8756886119, 0.8245158632, 0.8691551185, 0.8666532852, 0.7898876404, 0.8632814233, 0.7750098078, 0.8575731122, 0.8571958295, 0.8558330768, 0.795483871, 0.7716981132, 0.8252225519, 0.7768289777, 0.7, 0.7376470588, 0.8332209106, 0.8359351674, 0.7854125929, 0.8464147703, 0.7615000959, 0.8568973116, 0.8546858013, 0.7, 0.7610114192, 0.8128205128, 0.7558663276, 0.8169590643, 0.8039839034, 0.8439598742, 0.8169590643, 0.7711374815, 0.8426793861, 0.8585523481, 0.7605405405, 0.7861952862, 0.8034421027, 0.8522023182, 0.8727579231, 0.8027932961, 0.7611462584, 0.8271641791, 0.785106383, 0.7917139614, 0.7719653748, 0.8523622904, 0.8480248952, 0.8065883888, 0.8442025917, 0.6745454545, 0.8204678363, 0.7508511807, 0.7777860883, 0.7578209277, 0.7, 0.8432739755, 0.8353216873, 0.6745454545, 0.7651189785, 0.7909090909, 0.8008322044, 0.7682098765, 0.8470307759, 0.7926517572, 0.8154804986, 0.7178217822, 0.7352941176, 0.682122905, 0.7670511342, 0.7615384615, 0.7799070848, 0.8121387283, 0.7, 0.7839779006, 0.8373948718, 0.7616216216, 0.7579945799, 0.766330699, 0.7287179487, 0.7112676056, 0.7687971386, 0.8655598524, 0.7577292412, 0.8384565075, 0.7725, 0.8594278283, 0.8066753713, 0.7, 0.7861952862, 0.737037037, 0.7112676056, 0.7615384615, 0.8370371889, 0.8472168824, 0.6921505833, 0.8471628531, 0.7352941176, 0.8190615836, 0.8391156463, 0.8122224445, 0.7, 0.7594594595, 0.8417764269, 0.8014013142, 0.8472222222, 0.8121387283, 0.8121387283, 0.7507137106, 0.7930444698, 0.7600652884, 0.6856716418, 0.8261795407, 0.8129036, 0.7, 0.7648143965, 0.6811420109, 0.788, 0.8424714607, 0.8277118297, 0.788, 0.750640322, 0.6879194631, 0.7996336996, 0.7524778299, 0.7901392758, 0.7102861883, 0.8234148363, 0.7615384615, 0.7882882883, 0.7544502618, 0.8372881356, 0.8285604186, 0.6789268293, 0.8556163447, 0.7581333333, 0.8581487168, 0.788, 0.8703272075, 0.7689343816, 0.7066666667, 0.8103448276, 0.8336082474, 0.8239193084, 0.7638471612, 0.775862069, 0.7499252616, 0.7690894881, 0.7556824401, 0.7930444698, 0.7605405405, 0.8401546157, 0.7619211634, 0.784588373, 0.7815964523, 0.8488750107, 0.7456858731, 0.7851430329, 0.8529102845, 0.8419327373, 0.833546054, 0.8506078055, 0.856976591, 0.8027932961, 0.7907936508, 0.7839779006, 0.7112676056, 0.8545023697, 0.8246427576, 0.8161329305, 0.8057225196, 0.7608298439, 0.7, 0.713487882, 0.7737362249, 0.7705891485, 0.782208589, 0.7664526485, 0.7610114192, 0.8283557995, 0.7780115937, 0.7568807339, 0.8161906357, 0.8668971671, 0.7, 0.8266820975, 0.8397260274, 0.8010404624, 0.8293413174, 0.8162966066, 0.8467278033, 0.8147164839, 0.5870792565, 0.7407948568, 0.8607502328, 0.788, 0.690260777, 0.7, 0.7541026998, 0.7713927952, 0.8358941528, 0.8592761382, 0.7, 0.7112676056, 0.6876350776, 0.752992728, 0.7352941176, 0.7544502618, 0.7690514987, 0.755820743, 0.7467357701, 0.7621819449, 0.820855615, 0.6519313305, 0.8146722164, 0.8608592306, 0.7909090909, 0.7352941176, 0.6884545001, 0.7925778132, 0.8188369678, 0.8558093565, 0.7775899632, 0.8252225519, 0.8609341826, 0.7287179487, 0.7871473354, 0.8475993917, 0.7930444698, 0.7778378378, 0.8126693227, 0.7909090909, 0.7524778299, 0.8668464934, 0.8139544513, 0.8542784703, 0.6879194631, 0.860087146, 0.8604187656, 0.6162671001, 0.8233814433, 0.830634715, 0.7725958345, 0.7731600718, 0.7618602914, 0.8548983364, 0.8309882747, 0.8290566038, 0.7947368421, 0.7112676056, 0.8222741117, 0.8229009453, 0.8625874126, 0.7553160607, 0.749851279, 0.6549731801, 0.7352941176, 0.737037037, 0.7800356824, 0.7505097582, 0.7285714286, 0.8587143424, 0.6831243469, 0.7616216216, 0.8120278908, 0.6912221561, 0.7, 0.6556420233, 0.8515095372, 0.761595824, 0.7701962931, 0.8110871075, 0.8363924579, 0.7, 0.6903646596, 0.796, 0.796, 0.7, 0.8420689655, 0.7482598608, 0.8204678363, 0.796, 0.8337453646, 0.7451371146, 0.8131147541, 0.8115354018, 0.6007673154, 0.7, 0.6319838057, 0.6792746114, 0.7, 0.8494330113, 0.7372844037, 0.6319838057, 0.7, 0.7799143775, 0.8494672755, 0.705991342, 0.7450105579, 0.7725463415]

sort4 = list(np.argsort(sp4))
sp4= [sp4[i] for i in sort4] 
fid4= [fid4[i] for i in sort4]

sp5 = [0.19262375, 0.19738875, 0.201134375, 0.210328125, 0.213920625, 0.205021875, 0.2261, 0.21871875, 0.22350625, 0.19148625, 0.20419875, 0.206715625, 0.20341875, 0.200338125, 0.28875, 0.19855, 0.22275625, 0.196214375, 0.20889375, 0.24088125, 0.198740625, 0.2212, 0.20346875, 0.202478125, 0.22638, 0.2472, 0.19674625, 0.2272, 0.24504375, 0.230634375, 0.2166, 0.20125, 0.19502375, 0.188861875, 0.197359375, 0.19836875, 0.19435625, 0.22001125, 0.201215625, 0.22723125, 0.238690625, 0.199409375, 0.19930875, 0.397375, 0.188046875, 0.228743125, 0.34425, 0.21299625, 0.21044375, 0.32446625, 0.20915625, 0.194578125, 0.23175, 0.21198125, 0.22261, 0.2347, 0.210659375, 0.2008, 0.19661, 0.19067875, 0.20321125, 0.20231875, 0.22692375, 0.22576875, 0.30974, 0.19183125, 0.196059375, 0.292134375, 0.21980625, 0.24123, 0.231184375, 0.19242, 0.20693625, 0.232734375, 0.247955625, 0.21902625, 0.21147875, 0.22400625, 0.352, 0.203571875, 0.23199, 0.2088725, 0.232671875, 0.19298125, 0.227760625, 0.19195, 0.19846875, 0.21934625, 0.2259, 0.196139375, 0.2409, 0.25130625, 0.23236, 0.19325625, 0.19854375, 0.22020625, 0.217359375, 0.256025, 0.2022, 0.188981875, 0.24215625, 0.193996875, 0.23708625, 0.28828125, 0.244921875, 0.19201125, 0.19194125, 0.244921875, 0.20073375, 0.20704, 0.25570625, 0.254118125, 0.237415, 0.19580625, 0.22977, 0.196390625, 0.20090625, 0.18376875, 0.20765625, 0.191065625, 0.22875, 0.20505625, 0.2070625, 0.27848125, 0.190909375, 0.239221875, 0.234188125, 0.206978125, 0.197463125, 0.2715, 0.201884375, 0.23103125, 0.198115625, 0.308125, 0.19301, 0.272171875, 0.25954, 0.21895625, 0.2675375, 0.22408625, 0.196680625, 0.21285625, 0.219796875, 0.1984125, 0.253025, 0.22836875, 0.20453125, 0.19963125, 0.19283125, 0.225403125, 0.211434375, 0.2363, 0.21702625, 0.3055, 0.21265625, 0.198380625, 0.21871875, 0.223865625, 0.19675, 0.19444375, 0.19725, 0.22486875, 0.202553125, 0.218946875, 0.19479375, 0.22101875, 0.240964375, 0.2292, 0.20300625, 0.2050425, 0.25103375, 0.22876875, 0.202671875, 0.190108125, 0.20319375, 0.238215, 0.25471875, 0.19363625, 0.19706875, 0.19123125, 0.2328375, 0.24745625, 0.21339375, 0.191946875, 0.23149375, 0.24928125, 0.192171875, 0.21487125, 0.196659375, 0.27457125, 0.20109625, 0.217459375, 0.193554375, 0.200284375, 0.199943125, 0.190940625, 0.19334375, 0.190539375, 0.22693125, 0.19034375, 0.26935, 0.4624, 0.2014, 0.2200575, 0.21945625, 0.24665625, 0.218609375, 0.21998125, 0.19854875, 0.20455625, 0.20851375, 0.18831, 0.19739375, 0.2227375, 0.23034375, 0.219984375, 0.2146, 0.2114, 0.191971875, 0.25125, 0.20683125, 0.284184375, 0.21220625, 0.22383125, 0.19706625, 0.22649125, 0.1931, 0.202159375, 0.224253125, 0.21555625, 0.192948125, 0.281165625, 0.202403125, 0.20896875, 0.222925, 0.22921125, 0.20384375, 0.20269375, 0.27421, 0.214325, 0.210715625, 0.22423, 0.196845625, 0.286184375, 0.238611875, 0.21448, 0.240909375, 0.224639375, 0.3105, 0.21401875, 0.20808125, 0.220996875, 0.191746875, 0.20721875, 0.196971875, 0.2535, 0.2035, 0.18370625, 0.24785, 0.19246625, 0.20568125, 0.20028125, 0.24707375, 0.22835, 0.253508125, 0.214021875, 0.261984375, 0.197346875, 0.20556625, 0.2429, 0.214828125, 0.25933125, 0.18893375, 0.192580625, 0.2561, 0.194391875, 0.20100625, 0.2691125, 0.2004175, 0.220455, 0.408, 0.20715625, 0.2847, 0.22222125, 0.249234375, 0.27556875, 0.284909375, 0.232675, 0.27115, 0.190265625, 0.20128125, 0.20683125, 0.20857875, 0.22833125, 0.19640125, 0.22163, 0.20585, 0.21204375, 0.21663625, 0.245809375, 0.23010625, 0.2405, 0.20686875, 0.19801, 0.224580625, 0.213153125, 0.226359375, 0.19390625, 0.22928125, 0.21913125, 0.202620625, 0.19366, 0.19259375, 0.22436875, 0.232030625, 0.25833625, 0.24554375, 0.19768875, 0.18990625, 0.206104375, 0.22334625, 0.2387, 0.23141, 0.18978, 0.23368875, 0.212028125, 0.18884, 0.204364375, 0.195369375, 0.23499375, 0.19816875, 0.25499, 0.23784375, 0.20019375, 0.25039875, 0.20859375, 0.282146875, 0.20831125, 0.227203125, 0.210065625, 0.2657175, 0.204453125, 0.19881, 0.27189375, 0.241049375, 0.23859375, 0.258453125, 0.233146875, 0.21734875, 0.227628125, 0.18406875, 0.193353125, 0.20673875, 0.211874375, 0.193188125, 0.2351, 0.22803125, 0.260856875, 0.206096875, 0.20259375, 0.19266375, 0.20205625, 0.21318125, 0.19947875, 0.19473125, 0.19281875, 0.19780625, 0.21167125, 0.236484375, 0.21856875, 0.21798125, 0.19410125, 0.20025, 0.22080875, 0.19210625, 0.20115625, 0.257, 0.276940625, 0.19565625, 0.1986375, 0.22431875, 0.189099375, 0.24696, 0.22450625, 0.198516875, 0.22378125, 0.22423375, 0.206674375, 0.192623125, 0.202449375, 0.22755, 0.2188625, 0.20102375, 0.208704375, 0.25415, 0.1919, 0.203521875, 0.19646, 0.231121875, 0.19881875, 0.196698125, 0.2023125, 0.2538, 0.19320625, 0.2193375, 0.19683, 0.19256, 0.196146875, 0.198734375, 0.247905, 0.229246875, 0.19678125, 0.19488625, 0.189509375, 0.22887125, 0.207005, 0.19849375, 0.20776125, 0.23876875, 0.194611875, 0.20093125, 0.19606875, 0.19685625, 0.19489375, 0.189561875, 0.20479125, 0.25786875, 0.22091875, 0.20167, 0.21854625, 0.22265, 0.19496, 0.19243, 0.23537, 0.204265625, 0.20134375, 0.23693125, 0.315190625, 0.22069875, 0.195784375, 0.24949375, 0.2205375, 0.20431, 0.195281875, 0.22968, 0.28479375, 0.20730625, 0.19743875, 0.24061875, 0.207015625, 0.224755, 0.20910625, 0.1988, 0.19580625, 0.19463125, 0.30224, 0.203003125, 0.2573125, 0.25509375, 0.22475625, 0.20754125, 0.289425, 0.306159375, 0.19103125, 0.194724375, 0.191684375, 0.22461875, 0.21813375, 0.19125875, 0.195059375, 0.23545125, 0.192698125, 0.196015625, 0.243121875, 0.20207375, 0.202559375, 0.21888125, 0.19308, 0.309375, 0.21118125, 0.2385, 0.18994, 0.193126875, 0.20295, 0.199035625, 0.21286875, 0.20987875, 0.20485625, 0.19650625, 0.21013125, 0.219665625, 0.231784375, 0.210421875, 0.19832875, 0.200858125, 0.21015625, 0.18423125, 0.19704, 0.21499375, 0.279703125, 0.257765625, 0.2592, 0.1957, 0.20129625, 0.227159375, 0.19205, 0.19379625, 0.23378, 0.21091875, 0.25605, 0.2253, 0.195683125, 0.2328, 0.24264, 0.19390625, 0.22584375, 0.2663625, 0.18989875, 0.193780625, 0.213734375, 0.230030625, 0.203790625, 0.26945, 0.202296875, 0.29, 0.2008375, 0.23974, 0.23933875, 0.26872, 0.219490625, 0.220584375, 0.190171875, 0.199063125, 0.23591625, 0.2503125, 0.19479375, 0.203621875, 0.19907375, 0.21605625, 0.20543625, 0.192616875, 0.21745875, 0.227865625, 0.23206875, 0.22524375, 0.191249375, 0.22437125, 0.21326875, 0.22180625, 0.2044, 0.213865, 0.22685, 0.22765625, 0.20629375, 0.195471875, 0.21396125, 0.23878125, 0.24531375, 0.22394, 0.20914625, 0.19564375, 0.204103125, 0.19108125, 0.2677925, 0.199546875, 0.20818, 0.222044375, 0.3961, 0.194048125, 0.21278125, 0.19458875, 0.210501875, 0.3328, 0.19841875, 0.252290625, 0.205534375, 0.23413125, 0.2872, 0.250408125, 0.197671875, 0.19973375, 0.22330625, 0.191996875, 0.26625, 0.197614375, 0.2054025, 0.192041875, 0.244575, 0.231155625, 0.193565625, 0.18888375, 0.21241875, 0.21494375, 0.22683125, 0.2215, 0.25845, 0.21378125, 0.202736875, 0.19314, 0.24777125, 0.226209375, 0.219105, 0.196246875, 0.247396875, 0.22896875, 0.19538375, 0.20469625, 0.272, 0.19901, 0.211303125, 0.2733, 0.197396875, 0.2224, 0.19751875, 0.26030625, 0.2291, 0.19395, 0.226853125, 0.21088, 0.222421875, 0.194844375, 0.23575, 0.21922, 0.21823125, 0.19380875, 0.4335, 0.191450625, 0.21193125, 0.192421875, 0.227505625, 0.20682125, 0.236671875, 0.21003875, 0.236565625, 0.22296625, 0.220746875, 0.206906875, 0.1938675, 0.19745, 0.19748, 0.20688125, 0.22771875, 0.23180625, 0.223778125, 0.192565625, 0.2187875, 0.2094, 0.22709625, 0.21211875, 0.238159375, 0.193896875, 0.21665, 0.189134375, 0.216575, 0.25361875, 0.239731875, 0.25064375, 0.31532875, 0.262395, 0.199705625, 0.19126875, 0.28606875, 0.239578125, 0.19659625, 0.21319375, 0.191764375, 0.20892625, 0.21013125, 0.199259375, 0.22138625, 0.23713125, 0.19321625, 0.211709375, 0.26746875, 0.204740625, 0.20953125, 0.20841875, 0.194649375, 0.2085875, 0.19606875, 0.20339375, 0.19827125, 0.213, 0.234328125, 0.221799375, 0.230758125, 0.199546875, 0.30192, 0.21008, 0.23420625, 0.22573, 0.224590625, 0.220728125, 0.222703125, 0.32596875, 0.24016875, 0.22014125, 0.200940625, 0.20524375, 0.20396875, 0.223546875, 0.20016875, 0.194656875, 0.21484, 0.19405625, 0.23572125, 0.19314375, 0.22761875, 0.19509625, 0.191909375, 0.21020875, 0.20133625, 0.19997875, 0.219271875, 0.27168125, 0.19561875, 0.24402, 0.21744, 0.2318275, 0.25679375, 0.24365625, 0.2544, 0.244389375, 0.21066875, 0.220590625, 0.21264375, 0.203115625, 0.23428125, 0.22605125, 0.20280625, 0.223866875, 0.19604375, 0.209, 0.188610625, 0.233240625, 0.2277, 0.19695, 0.19445375, 0.25453125, 0.22148125, 0.22397375, 0.2057625, 0.18855625, 0.201165625, 0.195291875, 0.27954375, 0.1967, 0.194534375, 0.28018125, 0.196853125, 0.2415, 0.20750625, 0.20851625, 0.200934375, 0.200934375, 0.21285, 0.20545, 0.19985625, 0.266913125, 0.20036875, 0.191265625, 0.21270625, 0.28528125, 0.241565625, 0.2082, 0.3, 0.237380625, 0.23213625, 0.193399375, 0.2363775, 0.212615625, 0.19644, 0.20279375, 0.191434375, 0.23064375, 0.210496875, 0.195628125, 0.1905875, 0.19499875, 0.22831875, 0.21066875, 0.2214575, 0.24599625, 0.19361125, 0.403696875, 0.2691, 0.22325625, 0.21470125, 0.207396875, 0.2375625, 0.200925, 0.1963, 0.202136875, 0.197303125, 0.23541, 0.19963125, 0.30294, 0.1930525, 0.22615, 0.2283625, 0.221428125, 0.19331, 0.2518, 0.21951875, 0.197803125, 0.22177125, 0.20295, 0.29484375, 0.272201875, 0.20584625, 0.23985, 0.218984375, 0.254625, 0.230915, 0.21230625, 0.205484375, 0.21096875, 0.204203125, 0.21020625, 0.20465, 0.19391875, 0.247296875, 0.194096875, 0.195546875, 0.19854375, 0.19420625, 0.22044, 0.22003125, 0.21534625, 0.21051875, 0.21418875, 0.206353125, 0.23275, 0.207896875, 0.2114875, 0.193403125, 0.21886875, 0.19869875, 0.208015625, 0.195596875, 0.2416, 0.195178125, 0.23111875, 0.2074, 0.19235625, 0.212859375, 0.19593125, 0.21322125, 0.22701, 0.2280875, 0.25228125, 0.20303125, 0.213418125, 0.21781875, 0.2175, 0.211380625, 0.20903875, 0.19601125, 0.21298125, 0.20959375, 0.26364875, 0.265765625, 0.22824625, 0.2185775, 0.191080625, 0.2686, 0.2015, 0.196890625, 0.205646875, 0.232675, 0.194814375, 0.21356, 0.2495, 0.22525, 0.197865625, 0.21990625, 0.22122375, 0.19728125, 0.195446875, 0.203983125, 0.19789625, 0.25245, 0.2237, 0.18991875, 0.216, 0.2724, 0.21738125, 0.23915, 0.18925125, 0.21324, 0.20438625, 0.206805, 0.25340625, 0.21346875, 0.226263125, 0.193478125, 0.24764, 0.229, 0.18390625, 0.23004875, 0.2215, 0.19479375, 0.2661, 0.201009375, 0.26139625, 0.24238, 0.2706, 0.2365425, 0.189128125, 0.210165625, 0.207278125, 0.19851875, 0.20795, 0.22130625, 0.210675, 0.19364125, 0.2202875, 0.2099, 0.1839875, 0.24435, 0.334698125, 0.19409375, 0.22945, 0.20639625, 0.21078125, 0.2653625, 0.20429375, 0.197610625, 0.190246875, 0.27125625, 0.21995625, 0.19335, 0.19321125, 0.19896625, 0.209840625, 0.23167375, 0.22195, 0.19723125, 0.228265625, 0.24165625, 0.22575, 0.19610625, 0.21532375, 0.248953125, 0.207884375, 0.20358125, 0.1903125, 0.200640625, 0.222140625, 0.26509375, 0.23745, 0.19155625, 0.19074375, 0.26180625, 0.22113875, 0.2271875, 0.196531875, 0.197390625, 0.25927125, 0.22093125, 0.19595625, 0.22385, 0.33309375, 0.192564375, 0.242446875, 0.230665625, 0.228475, 0.205590625, 0.198378125, 0.209715625, 0.2717, 0.234234375, 0.21099375, 0.22110875, 0.2330375, 0.21873125, 0.24023125, 0.23416875, 0.19001875, 0.25918625, 0.24793125, 0.214775, 0.26934375, 0.22428, 0.220326875, 0.234290625, 0.200415625, 0.19471875, 0.19470875, 0.210600625, 0.221203125, 0.196040625, 0.20604125, 0.196540625, 0.204140625, 0.28400625, 0.215390625, 0.22201875, 0.2037, 0.19037875, 0.20637, 0.206659375, 0.245578125, 0.2311, 0.22184375, 0.20459, 0.20368125, 0.231375625, 0.204534375, 0.20119375, 0.21524375, 0.24140625, 0.203159375, 0.2736, 0.3536, 0.2865, 0.19836875, 0.25350625, 0.23495625, 0.2005625, 0.21834375, 0.20316125, 0.19700625, 0.21836875, 0.191886875, 0.3315, 0.224921875, 0.214709375, 0.22418375, 0.200971875, 0.22213625, 0.236321875, 0.195058125, 0.19067125, 0.20676, 0.18965625, 0.269248125, 0.2531, 0.19089375, 0.20307375, 0.196064375, 0.203790625, 0.19799375, 0.24171875, 0.234478125, 0.2126, 0.224446875, 0.19715125, 0.19431875, 0.189016875, 0.21042875, 0.290615, 0.271496875, 0.249609375, 0.24854375, 0.24088, 0.211171875, 0.19556375, 0.18970375, 0.193303125, 0.216821875, 0.232228125, 0.2112, 0.1999875, 0.188746875, 0.22490375, 0.24144, 0.1961, 0.21715, 0.2140325, 0.20204375, 0.22546875, 0.2013, 0.200365625, 0.2343, 0.19475625, 0.21929625, 0.22838125, 0.229196875, 0.1972, 0.224766875, 0.21356375, 0.206915625, 0.23675625, 0.19527375, 0.237103125, 0.24299375, 0.19608125, 0.212334375, 0.3519, 0.202809375, 0.196028125, 0.230765625, 0.28366875, 0.2070275, 0.2534875, 0.19468125, 0.22438125, 0.22415, 0.194140625, 0.19238125, 0.22440625, 0.22745625, 0.2103175, 0.21385625, 0.20746, 0.19476125, 0.24034375, 0.201563125, 0.19311875, 0.22015, 0.27584625, 0.222476875, 0.196756875, 0.21130625, 0.20965625, 0.190734375, 0.23799875, 0.19343, 0.202765625, 0.20654375, 0.19338, 0.201053125, 0.21981875, 0.26071875, 0.197975, 0.210819375, 0.213021875, 0.18405625, 0.19553125, 0.19601, 0.19609375, 0.22945, 0.256, 0.192415625, 0.243556875, 0.209828125, 0.22465, 0.287671875, 0.24451875, 0.2076875, 0.240603125, 0.19117125, 0.18424375, 0.19055, 0.20196, 0.18421875, 0.265571875, 0.19151625, 0.259621875, 0.22981875, 0.2077, 0.21966, 0.22660625, 0.205364375, 0.21858375, 0.19723625, 0.19859375, 0.207328125, 0.24969375, 0.20076, 0.22540125, 0.192190625, 0.199584375, 0.246340625, 0.23746875, 0.196915625, 0.203425625, 0.214913125, 0.2294325, 0.193084375, 0.21160875, 0.20175, 0.196821875, 0.2673, 0.201683125, 0.189228125, 0.24505875, 0.216046875, 0.1982925, 0.28815, 0.189184375, 0.193503125, 0.19002125, 0.19463, 0.20052375, 0.240284375, 0.203484375, 0.19748625, 0.221984375, 0.2336375, 0.226, 0.23630625, 0.22309625, 0.264515625, 0.227715625, 0.277185, 0.20875, 0.21628125, 0.25994875, 0.291946875, 0.210290625, 0.19066, 0.19986375, 0.20379875, 0.210253125, 0.19609625, 0.224565625, 0.20413125, 0.218305, 0.23139375, 0.190251875, 0.20844375, 0.22826375, 0.213240625, 0.23459625, 0.19385625, 0.19904625, 0.19211875, 0.19112625, 0.205340625, 0.223078125, 0.199778125, 0.206044375, 0.21935, 0.26088, 0.19610625, 0.2425, 0.209175, 0.31099375, 0.195430625, 0.20435625, 0.19678375, 0.23057375, 0.20874375, 0.20921875, 0.204325, 0.24331875, 0.22626, 0.191153125, 0.2102375, 0.231771875, 0.190940625, 0.211378125, 0.2418, 0.201346875, 0.195428125, 0.26840625, 0.23595, 0.260386875, 0.23428125, 0.275625, 0.21862125, 0.21268125, 0.207225, 0.201588125, 0.19070625, 0.20051875, 0.19635625, 0.190999375, 0.22032, 0.20777375, 0.229715625, 0.19822625, 0.241640625, 0.20648125, 0.193265625, 0.2716875, 0.209953125, 0.20104375, 0.196653125, 0.19448, 0.20840625, 0.214725, 0.21996, 0.209696875, 0.233485625, 0.19124, 0.194911875, 0.21765625, 0.325125, 0.20827125, 0.209253125, 0.20313125, 0.224859375, 0.19486875, 0.20787, 0.23853125, 0.2843675, 0.1887, 0.205546875, 0.2105375, 0.19524375, 0.19416375, 0.190715625, 0.20518375, 0.25343, 0.20706875, 0.23929375, 0.263285625, 0.21759375, 0.24996375, 0.20180625, 0.20686875, 0.234471875, 0.22206875, 0.23135, 0.19598125, 0.189460625, 0.223875, 0.28273125, 0.2496, 0.19119375, 0.218234375, 0.27576125, 0.23199375, 0.199059375, 0.252675, 0.2017, 0.27795, 0.1995325, 0.24188125, 0.190839375, 0.20232, 0.251184375, 0.28215, 0.20778125, 0.19520125, 0.21761625, 0.222415625, 0.226446875, 0.195034375, 0.23410875, 0.296915625, 0.20053125, 0.2819625, 0.2203, 0.19252375, 0.27051125, 0.234, 0.251759375, 0.206605, 0.20576875, 0.20345625, 0.235540625, 0.1933875, 0.21635, 0.22846875, 0.20998125, 0.191684375, 0.288575, 0.21884375, 0.22150125, 0.21285, 0.206728125, 0.1891, 0.2146875, 0.20671875, 0.2242925, 0.19851, 0.23907875, 0.193215625, 0.201671875, 0.30796875, 0.19225625, 0.19234375, 0.21733375, 0.191983125, 0.20206125, 0.196658125, 0.200778125, 0.26602375, 0.18815, 0.1969875, 0.194871875, 0.207759375, 0.19948125, 0.20104375, 0.21628125, 0.22423625, 0.228045625, 0.25280625, 0.2354625, 0.19182875, 0.25488, 0.29308, 0.23930625, 0.193046875, 0.192653125, 0.26146875, 0.20125, 0.204515625, 0.241095, 0.2258, 0.200753125, 0.2268575, 0.27253125, 0.193448125, 0.2277, 0.21113125, 0.22686875, 0.23120625, 0.2044, 0.221175, 0.23025, 0.22445, 0.221525, 0.3553, 0.205240625, 0.2166, 0.23085375, 0.22359375, 0.194190625, 0.217115625, 0.22378125, 0.2091625, 0.20046375, 0.192585625, 0.228084375, 0.1996625, 0.2425, 0.202280625, 0.25115625, 0.20849375, 0.188830625, 0.2144, 0.197265625, 0.2074625, 0.19925, 0.200228125, 0.22825, 0.212678125, 0.2335, 0.19845625, 0.226440625, 0.21054375, 0.2864, 0.21189375, 0.195746875, 0.20501875, 0.193828125, 0.2907, 0.2414, 0.19273375, 0.26063125, 0.209915, 0.200235, 0.225909375, 0.22911, 0.2007, 0.2043125, 0.24055, 0.22775, 0.19447375, 0.211553125, 0.19543125, 0.2643, 0.192039375, 0.22640625, 0.1946, 0.2168, 0.22795625, 0.294046875, 0.22286875, 0.22100625, 0.19348125, 0.192958125, 0.2506875, 0.20403125, 0.19735625, 0.20265, 0.21515625, 0.2221375, 0.19018125, 0.22509375, 0.198390625, 0.25653, 0.20465625, 0.2111, 0.22494375, 0.22415, 0.19405375, 0.22215, 0.2198, 0.19031875, 0.194590625, 0.1944, 0.208784375, 0.19239875, 0.23420625, 0.24804, 0.22493, 0.192653125, 0.20178125, 0.206003125, 0.220521875, 0.198265625, 0.219360625, 0.206871875, 0.19823125, 0.248795, 0.194260625, 0.214296875, 0.19805625, 0.19431875, 0.19095, 0.20028125, 0.20492125, 0.202640625, 0.223746875, 0.19765625, 0.23781875, 0.23768125, 0.224315625, 0.21135, 0.1892, 0.28389, 0.22110125, 0.25205, 0.2109, 0.20346, 0.19816875, 0.19986875, 0.2712, 0.218748125, 0.20799375, 0.21965625, 0.211478125, 0.226721875, 0.22241875, 0.27635625, 0.19370125, 0.1952, 0.218528125, 0.2213825, 0.1925, 0.215534375, 0.22603125, 0.2124, 0.191493125, 0.2015625, 0.21590625, 0.206228125, 0.309, 0.190246875, 0.18814, 0.20109375, 0.239259375, 0.19023625, 0.193881875, 0.22765625, 0.21580125, 0.21563375, 0.194329375, 0.21759375, 0.22022125, 0.189265625, 0.1999, 0.193256875, 0.2225025, 0.19401, 0.210296875, 0.212371875, 0.190159375, 0.194159375, 0.2415, 0.225759375, 0.19220375, 0.1984925, 0.221275, 0.205765625, 0.24008875, 0.288065, 0.195971875, 0.24530625, 0.195926875, 0.215675, 0.19880625, 0.19301875, 0.197, 0.197728125, 0.192000625, 0.1992525, 0.21356875, 0.33946875, 0.22528, 0.227053125, 0.21331375, 0.24122, 0.19474375, 0.19685, 0.19974625, 0.2089875, 0.21745625, 0.19415625, 0.211225, 0.27705, 0.23971875, 0.19351875, 0.221808125, 0.191385625, 0.19374625, 0.21451125, 0.19361, 0.22935, 0.187964375, 0.20668125, 0.22759375, 0.274603125, 0.23509125, 0.2235, 0.232915625, 0.272600625, 0.194553125, 0.229186875, 0.18925, 0.19155625, 0.21799375, 0.20515, 0.194176875, 0.194878125, 0.26788125, 0.20365125, 0.196244375, 0.202828125, 0.30313125, 0.20909875, 0.20624375, 0.194410625, 0.23172875, 0.2008, 0.24091125, 0.1959, 0.24346875, 0.19501125, 0.208796875, 0.190623125, 0.30435625, 0.197465625, 0.26424375, 0.2557, 0.20129625, 0.21253125, 0.2715, 0.194399375, 0.26148125, 0.206975, 0.238499375, 0.20597125, 0.24325, 0.20125125, 0.19030625, 0.203599375, 0.26815, 0.198551875, 0.225, 0.265465625, 0.199015625, 0.204671875, 0.192753125, 0.209971875, 0.19804375, 0.195141875, 0.21284375, 0.199455625, 0.2083125, 0.21042125, 0.197705, 0.18779375, 0.198905625, 0.23342125, 0.227559375, 0.195921875, 0.203359375, 0.206984375, 0.28659375, 0.25489375, 0.20141875, 0.21019375, 0.22465, 0.25609375, 0.241920625, 0.19954625, 0.22459625, 0.21886875, 0.201886875, 0.198903125, 0.23109375, 0.22934375, 0.19298, 0.19976125, 0.20363625, 0.20420875, 0.20696375, 0.21693125, 0.21522625, 0.2175825, 0.19532125, 0.232846875, 0.23128125, 0.2160375, 0.24625, 0.39015, 0.20659375, 0.19515375, 0.32671875, 0.213096875, 0.32459375, 0.20506875, 0.220173125, 0.19112125, 0.222225, 0.254625, 0.19913625, 0.195540625, 0.193421875, 0.24137875, 0.220884375, 0.1923125, 0.26403125, 0.258609375, 0.2559, 0.235225625, 0.19033375, 0.201346875, 0.2202, 0.24021, 0.21019375, 0.209421875, 0.3723, 0.20660625, 0.286171875, 0.20181, 0.21644625, 0.22607125, 0.19623125, 0.2173825, 0.19569375, 0.22601, 0.19621875, 0.2871, 0.2190875, 0.18975, 0.22223375, 0.2703, 0.207158125, 0.190230625, 0.252575, 0.198753125, 0.25453125, 0.21855, 0.33415625, 0.196865625, 0.20683125, 0.22075625, 0.20728375, 0.227991875, 0.19456, 0.208125, 0.223633125, 0.217359375, 0.22315, 0.195540625, 0.196446875, 0.25284875, 0.2202, 0.194873125, 0.1916, 0.19097875, 0.18936, 0.23254375, 0.196359375, 0.19814625, 0.228446875, 0.3168375, 0.23796875, 0.223309375, 0.191836875, 0.2869, 0.19421625, 0.26067375, 0.290221875, 0.21925, 0.19937625, 0.201886875, 0.21321875, 0.197559375, 0.189709375, 0.213, 0.2518, 0.205653125, 0.2665125, 0.191058125, 0.199109375, 0.197346875, 0.2086, 0.187278125, 0.220858125, 0.23862, 0.2214575, 0.238201875, 0.193840625, 0.218721875, 0.19802625, 0.198478125, 0.214875, 0.22363625, 0.202190625, 0.199771875, 0.31231125, 0.28932, 0.192378125, 0.20461875, 0.19227375, 0.190211875, 0.22894375, 0.189384375, 0.19034875, 0.22918, 0.19903125, 0.21993875, 0.19535, 0.20994375, 0.2637, 0.20421875, 0.224203125, 0.205588125, 0.19282375, 0.19818875, 0.1910125, 0.20295625, 0.221271875, 0.220971875, 0.196424375, 0.206934375, 0.202565625, 0.21842375, 0.2055, 0.2900625, 0.198546875, 0.21721, 0.32909875, 0.264, 0.20180625, 0.31630625, 0.23890625, 0.2283875, 0.2414625, 0.23779125, 0.273859375, 0.20953125, 0.22484, 0.22085, 0.23915625, 0.22399375, 0.220959375, 0.24871, 0.225140625, 0.19905625, 0.190635625, 0.2501625, 0.19821875, 0.19384125, 0.214716875, 0.192203125, 0.211215625, 0.2325125, 0.23038125, 0.20804625, 0.196984375, 0.196843125, 0.2721, 0.235471875, 0.200865625, 0.217015625, 0.22525, 0.22284, 0.19525, 0.19127875, 0.20044375, 0.27662, 0.20409375, 0.202634375, 0.31875, 0.218221875, 0.26316, 0.20120625, 0.220465625, 0.19820625, 0.20644375, 0.2365125, 0.20459375, 0.233885, 0.202203125, 0.22848625, 0.19523, 0.22740375, 0.18977625, 0.22515, 0.26138, 0.204484375, 0.2280375, 0.208153125, 0.27614375, 0.2680475, 0.19621, 0.18731875, 0.2184, 0.221999375, 0.22256, 0.21168, 0.19470625, 0.22336875, 0.270034375, 0.2299, 0.20008125, 0.285440625, 0.22993, 0.208213125, 0.20753125, 0.252703125, 0.204953125, 0.22509375, 0.21745625, 0.194249375, 0.2098, 0.191093125, 0.2399, 0.245953125, 0.204825, 0.20026875, 0.20419875, 0.25214875, 0.19265, 0.2154375, 0.203371875, 0.216, 0.225059375, 0.21113125, 0.21114, 0.193828125, 0.195399375, 0.224151875, 0.220034375, 0.22674875, 0.191528125, 0.206184375, 0.27061875, 0.211765625, 0.205128125, 0.19914375, 0.196333125, 0.22704, 0.196359375, 0.24394, 0.21353125, 0.229803125, 0.22859375, 0.21135, 0.223508125, 0.23451, 0.18378125, 0.2086, 0.21376, 0.19352, 0.2192, 0.221271875, 0.21291875, 0.259196875, 0.31078125, 0.19740625, 0.23315, 0.20849375, 0.234721875, 0.2267, 0.200609375, 0.22769375, 0.20270375, 0.218091875, 0.19694375, 0.198546875, 0.18701875, 0.20916875, 0.25340625, 0.1979, 0.206, 0.192370625, 0.203715625, 0.25188125, 0.19405625, 0.217553125, 0.248625, 0.197715625, 0.2008, 0.19403125, 0.23277375, 0.198999375, 0.206606875, 0.19418125, 0.19166875, 0.19652625, 0.191283125, 0.19641375, 0.21801, 0.20595625, 0.21820375, 0.207471875, 0.228684375, 0.2012, 0.191370625, 0.20820625, 0.196534375, 0.19178625, 0.224765625, 0.2252, 0.22533125, 0.22687, 0.19274375, 0.25694, 0.21860625, 0.248671875, 0.22243125, 0.213121875, 0.32042875, 0.206834375, 0.282675, 0.21036875, 0.194788125, 0.22189375, 0.220618125, 0.18945625, 0.203450625, 0.192375625, 0.19465, 0.196846875, 0.201889375, 0.194898125, 0.218610625, 0.219996875, 0.22645, 0.2007375, 0.225975, 0.21646875, 0.221859375, 0.19156, 0.2132, 0.21190625, 0.19272375, 0.200329375, 0.19261, 0.20035625, 0.21696, 0.2781, 0.21055, 0.20006375, 0.244875, 0.19183125, 0.219375, 0.22076875, 0.20655, 0.25322, 0.18985, 0.21225, 0.2178, 0.21329375, 0.23490875, 0.197346875, 0.232246875, 0.282890625, 0.20178, 0.21014125, 0.21370625, 0.192946875, 0.19760625, 0.32187375, 0.194865625, 0.18934125, 0.187846875, 0.20531875, 0.22303875, 0.190936875, 0.212821875, 0.21268375, 0.19103375, 0.293090625, 0.196603125, 0.21223125, 0.291220625, 0.1938275, 0.208578125, 0.21288, 0.2044625, 0.2459, 0.19885, 0.23034375, 0.219506875, 0.22545, 0.28868125, 0.219625, 0.20090625, 0.24890625, 0.2058, 0.20234625, 0.22275, 0.18913625, 0.19899375, 0.192683125, 0.192665625, 0.2376, 0.21871125, 0.2203, 0.20464625, 0.26166, 0.208040625, 0.21616875, 0.2148, 0.2652, 0.222796875, 0.23053125, 0.249315625, 0.211925, 0.24075, 0.18989375, 0.24600875, 0.1931, 0.231203125, 0.272, 0.213196875, 0.192924375, 0.24995, 0.210915625, 0.23185, 0.21981875, 0.23614, 0.18953375, 0.22158125, 0.197440625, 0.30257875, 0.2952, 0.193034375, 0.22544, 0.246228125, 0.199974375, 0.21605625, 0.212634375, 0.19979875, 0.24508, 0.2167625, 0.200140625, 0.20805625, 0.237745, 0.207690625, 0.18799375, 0.189780625, 0.198, 0.197853125, 0.211371875, 0.2795625, 0.19973125, 0.205454375, 0.27, 0.196096875, 0.19301125, 0.2237375, 0.189321875, 0.223415625, 0.229078125, 0.25341, 0.18913375, 0.25466875, 0.208534375, 0.21200625, 0.18904875, 0.31141, 0.2148, 0.201345625, 0.22923, 0.221621875, 0.228083125, 0.20544875, 0.193634375, 0.227964375, 0.19638625, 0.284765625, 0.2254175, 0.22783125, 0.19926, 0.22582, 0.20723375, 0.20749375, 0.224459375, 0.18994, 0.221409375, 0.19213625, 0.21744375, 0.19394, 0.23861625, 0.208471875, 0.22235, 0.232465, 0.198960625, 0.29038125, 0.19001875, 0.20546625, 0.26138125, 0.200625, 0.22127, 0.18905, 0.2282, 0.19155, 0.23071875, 0.222646875, 0.222378125, 0.195076875, 0.24728375, 0.233885625, 0.22264125, 0.22756875, 0.249103125, 0.238955625, 0.227975, 0.19453125, 0.19430625, 0.19354125, 0.2194775, 0.2236, 0.20173125, 0.2417925, 0.19179375, 0.239476875, 0.2499, 0.19544625, 0.19733125, 0.232375625, 0.2218, 0.22603125, 0.190778125, 0.19122625, 0.1935875, 0.20112125, 0.21885625, 0.20477625, 0.193669375, 0.18964875, 0.247495, 0.204328125, 0.216685625, 0.188868125, 0.20004375, 0.21484, 0.2216, 0.236840625, 0.25334375, 0.234, 0.20534375, 0.27081, 0.20802625, 0.22448, 0.25510625, 0.19674375, 0.202315625, 0.18943125, 0.19474125, 0.2136, 0.208615625, 0.22559375, 0.2385, 0.19265, 0.22029625, 0.193505625, 0.2155, 0.21962, 0.26793125, 0.22015125, 0.22793375, 0.19615625, 0.2513, 0.2115875, 0.235125, 0.19075625, 0.18984625, 0.19358, 0.20015375, 0.21563625, 0.21497375, 0.21984625, 0.211421875, 0.2026, 0.22753125, 0.2568425, 0.24641125, 0.221265, 0.213815, 0.2443, 0.24320625, 0.23960625, 0.234219375, 0.226370625, 0.222684375, 0.20705625, 0.21046875, 0.211734375, 0.22743625, 0.22287, 0.19560625, 0.19398375, 0.18993, 0.228646875, 0.244715, 0.19414875, 0.204284375, 0.19515, 0.24635625, 0.20758125, 0.22734375, 0.202703125, 0.227946875, 0.196871875, 0.22815625, 0.19046125, 0.19080875, 0.20905625, 0.19510875, 0.20885, 0.19945625, 0.271734375, 0.21244375, 0.21306125, 0.22478125, 0.201525, 0.1994375, 0.21006875, 0.2638, 0.22172125, 0.25743, 0.189480625, 0.207974375, 0.19609375, 0.21623125, 0.2742375, 0.19675125, 0.22711125, 0.21672, 0.22275, 0.207528125, 0.22924, 0.21255, 0.20855625, 0.209340625, 0.23927, 0.24925, 0.22720625, 0.21673125, 0.24667875, 0.190053125, 0.224921875, 0.22684, 0.226259375, 0.19358125, 0.19226, 0.2066, 0.21101875, 0.221428125, 0.20547, 0.25111875, 0.19579375, 0.22799625, 0.25096875, 0.21796, 0.196798125, 0.20849875, 0.275771875, 0.195140625, 0.191796875, 0.197478125, 0.222190625, 0.219125, 0.19233125, 0.202145, 0.22176, 0.22735875, 0.2205, 0.223325, 0.19243125, 0.19191125, 0.26700625, 0.207965, 0.23893125, 0.21541875, 0.199313125, 0.22458, 0.19082375, 0.20463125, 0.2205, 0.22153125, 0.20961875, 0.21146875, 0.255375, 0.190665625, 0.199046875, 0.232258125, 0.2385025, 0.208734375, 0.19210625, 0.200415625, 0.233925, 0.19771875, 0.19794375, 0.2895, 0.21776875, 0.20215, 0.22013, 0.1923875, 0.23784375, 0.20106125, 0.19673125, 0.20229375, 0.2977, 0.19601, 0.21306875, 0.24609375, 0.298828125, 0.227564375, 0.25955, 0.1904, 0.269025, 0.19945, 0.20971875, 0.22775, 0.21040125, 0.2325, 0.24341875, 0.30158, 0.221881875, 0.24534, 0.22459875, 0.25106875, 0.207703125, 0.19336, 0.20911875, 0.212165625, 0.2044125, 0.203684375, 0.208640625, 0.21695, 0.22221125, 0.201370625, 0.3077, 0.2355, 0.23984875, 0.193695625, 0.2873, 0.1924, 0.207359375, 0.199609375, 0.22674375, 0.24108125, 0.189009375, 0.189793125, 0.22105875, 0.25625875, 0.219275, 0.21221375, 0.30767875, 0.19501875, 0.2559375, 0.20439125, 0.193589375, 0.20900625, 0.22456875, 0.2252, 0.193589375, 0.1927125, 0.196659375, 0.2152, 0.23498125, 0.192655625, 0.277409375, 0.25977125, 0.215465625, 0.2249, 0.27469875, 0.19525625, 0.3135, 0.24215625, 0.21851875, 0.205415625, 0.19925, 0.23547, 0.239, 0.23514625, 0.24618125, 0.29664, 0.191885625, 0.2465, 0.195728125, 0.262703125, 0.21351875, 0.39556875, 0.2004, 0.257, 0.20586875, 0.207059375, 0.25178, 0.194009375, 0.22684375, 0.29408, 0.26059875, 0.3043, 0.191299375, 0.203503125, 0.1949, 0.190134375, 0.22237, 0.23547625, 0.2132, 0.21907375, 0.2301625, 0.25050625, 0.26743125, 0.18818125, 0.24403125, 0.20134375, 0.221586875, 0.22429375, 0.22003125, 0.210946875, 0.20121125, 0.197640625, 0.192123125, 0.202165625, 0.19440625, 0.21914375, 0.220575, 0.23409, 0.2329, 0.208278125, 0.192128125, 0.20625, 0.21163125, 0.24821875, 0.19736625, 0.19069625, 0.31501, 0.190859375, 0.194871875, 0.253, 0.193196875, 0.21921, 0.23660625, 0.189496875, 0.19746875, 0.197015625, 0.23090625, 0.198135625, 0.21874375, 0.210328125, 0.29114625, 0.205209375, 0.2184, 0.20607875, 0.21725, 0.21348, 0.2848, 0.20469375, 0.198951875, 0.21365625, 0.220921875, 0.20980875, 0.220653125, 0.19465, 0.21953125, 0.19146, 0.19995625, 0.212390625, 0.2549, 0.193678125, 0.21143125, 0.19671875, 0.191121875, 0.25619, 0.20052375, 0.22195, 0.19269625, 0.23826875, 0.195674375, 0.19335625, 0.20891125, 0.21928125, 0.19874625, 0.2488125, 0.221813125, 0.198403125, 0.2173975, 0.218034375, 0.19268, 0.246809375, 0.198603125, 0.24480625, 0.22406875, 0.21842125, 0.24592375, 0.22674, 0.222228125, 0.276375, 0.279965625, 0.194521875, 0.277184375, 0.2104, 0.24531875, 0.26130625, 0.19115, 0.213609375, 0.20053125, 0.253149375, 0.21839375, 0.19743125, 0.19425625, 0.22801875, 0.21553375, 0.22115625, 0.21937, 0.224996875, 0.21051875, 0.22085625, 0.206184375, 0.22111, 0.1974, 0.19622125, 0.21913875, 0.26079375, 0.22013125, 0.203044375, 0.20948625, 0.30440625, 0.19521125, 0.32079, 0.21955, 0.202809375, 0.21586375, 0.200565625, 0.23456, 0.19189375, 0.226260625, 0.25235, 0.22484625, 0.20201875, 0.198733125, 0.19443875, 0.222928125, 0.268225, 0.19419375, 0.2301, 0.198174375, 0.198253125, 0.19173125, 0.222571875, 0.226071875, 0.21724375, 0.19950625, 0.2204875, 0.197378125, 0.194621875, 0.21726875, 0.199128125, 0.24704, 0.19593125, 0.22388375, 0.19589375, 0.20743875, 0.19925875, 0.2567, 0.19507875, 0.21289375, 0.19025, 0.21322875, 0.19335875, 0.199140625, 0.20583625, 0.201535, 0.248935, 0.19405625, 0.18876875, 0.19295125, 0.23255625, 0.20034375, 0.202925, 0.19327375, 0.19821875, 0.228939375, 0.23765625, 0.206675, 0.23198125, 0.19899375, 0.199421875, 0.321310625, 0.2312625, 0.232194375, 0.2059375, 0.198696875, 0.20008125, 0.203100625, 0.21441875, 0.194799375, 0.20297125, 0.21785625, 0.204549375, 0.41038, 0.19473125, 0.20058125, 0.2775, 0.197028125, 0.2201025, 0.22138625, 0.222578125, 0.24487375, 0.233703125, 0.19964375, 0.19823125, 0.23664375, 0.21965625, 0.287375, 0.20523125, 0.20040625, 0.189309375, 0.25061875, 0.23453125, 0.20094625, 0.242296875, 0.192421875, 0.20986875, 0.225628125, 0.20736, 0.220546875, 0.221440625, 0.193873125, 0.2124525, 0.19476875, 0.193684375, 0.19105, 0.20671125, 0.208896875, 0.192040625, 0.23735625, 0.322734375, 0.206003125, 0.195725, 0.205928125, 0.19365625, 0.25740625, 0.221071875, 0.203490625, 0.24261875, 0.193730625, 0.2144625, 0.211964375, 0.27280625, 0.19545375, 0.21061125, 0.206803125, 0.198005625, 0.22225625, 0.19966625, 0.20076, 0.193446875, 0.2196, 0.19796, 0.2148, 0.299784375, 0.25540625, 0.2109, 0.2496, 0.19555, 0.23236875, 0.200759375, 0.19455625, 0.206221875, 0.26114, 0.18854375, 0.207453125, 0.22443125, 0.192906875, 0.249725, 0.21593125, 0.204968125, 0.2143625, 0.202371875, 0.267803125, 0.25636, 0.19293625, 0.19584875, 0.204303125, 0.228634375, 0.26029375, 0.20061875, 0.19333125, 0.202371875, 0.195034375, 0.1923, 0.207995625, 0.22850875, 0.191733125, 0.215109375, 0.193863125, 0.200079375, 0.234, 0.21181875, 0.235078125, 0.191871875, 0.2045, 0.20892625, 0.215578125, 0.2511875, 0.193653125, 0.22771875, 0.23885625, 0.19176875, 0.21248125, 0.2160725, 0.205446875, 0.19281, 0.20990625, 0.23159, 0.21785375, 0.18949625, 0.248625, 0.199228125, 0.21900625, 0.199940625, 0.219796875, 0.22392625, 0.274135625, 0.193483125, 0.2186125, 0.231965625, 0.21205, 0.19404, 0.24518125, 0.20475, 0.22245, 0.2571975, 0.20331875, 0.228606875, 0.22419, 0.20524375, 0.20165625, 0.202671875, 0.23263125, 0.22843875, 0.190440625, 0.19493, 0.22096625, 0.210015625, 0.194409375, 0.19539875, 0.191503125, 0.222396875, 0.28614, 0.20716375, 0.27359375, 0.21240125, 0.22594375, 0.19182375, 0.205421875, 0.242558125, 0.19859375, 0.20483625, 0.19840625, 0.21854375, 0.216890625, 0.207390625, 0.1969, 0.27039375, 0.30538, 0.199946875, 0.312, 0.22528375, 0.23559375, 0.203546875, 0.22209875, 0.19335625, 0.19401875, 0.19185625, 0.189211875, 0.250303125, 0.191646875, 0.25626, 0.21073125, 0.1943375, 0.234264375, 0.2924, 0.20549375, 0.20919375, 0.19645625, 0.19249375, 0.2124, 0.218158125, 0.20030625, 0.220813125, 0.22268375, 0.192440625, 0.24944, 0.20674375, 0.19236875, 0.20560375, 0.18854875, 0.212539375, 0.21008625, 0.21585125, 0.19000625, 0.197153125, 0.2275, 0.21294375, 0.21140875, 0.23677125, 0.26990625, 0.19087375, 0.304353125, 0.20299375, 0.24602, 0.2359, 0.19988125, 0.21195, 0.188110625, 0.205278125, 0.19419875, 0.249846875, 0.2346, 0.20635125, 0.232103125, 0.19245625, 0.2925, 0.20801, 0.19493375, 0.231788125, 0.248, 0.19730875, 0.198959375, 0.1916675, 0.201509375, 0.22574, 0.203196875, 0.199874375, 0.19484875, 0.239353125, 0.22410125, 0.25046125, 0.21901875, 0.215196875, 0.234396875, 0.23505, 0.19787, 0.206578125, 0.30525625, 0.218875, 0.28081875, 0.21779375, 0.223194375, 0.19645625, 0.19592375, 0.2115, 0.21524375, 0.205808125, 0.194076875, 0.22918125, 0.19560625, 0.19545625, 0.22863125, 0.212296875, 0.23035875, 0.19242125, 0.21784375, 0.204959375, 0.22493125, 0.278215625, 0.20365625, 0.22869375, 0.20129375, 0.227321875, 0.22015375, 0.200784375, 0.21721875, 0.19888125, 0.194164375, 0.22460625, 0.19249375, 0.19356375, 0.2401, 0.249740625, 0.19629375, 0.19681125, 0.24375, 0.241845625, 0.19116625, 0.263998125, 0.20681625, 0.190381875, 0.24507, 0.24074125, 0.2115125, 0.2166375, 0.2, 0.19038625, 0.20782375, 0.33415625, 0.196148125, 0.228854375, 0.19443, 0.19851875, 0.19265, 0.209715, 0.193478125, 0.25808, 0.261640625, 0.24790625, 0.2195375, 0.218346875, 0.18968125, 0.211459375, 0.2089, 0.200615, 0.25284375, 0.226180625, 0.20687875, 0.20245, 0.20698, 0.21834375, 0.22318625, 0.2259, 0.259845, 0.1903, 0.27, 0.29424, 0.201245625, 0.2095625, 0.20149375, 0.27805625, 0.1979625, 0.27115, 0.19036, 0.20704, 0.237975, 0.19946875, 0.224440625, 0.21155625, 0.194780625, 0.21264375, 0.207953125, 0.196521875, 0.233365, 0.20908625, 0.19225, 0.250951875, 0.22499375, 0.20176, 0.190664375, 0.2392, 0.19284375, 0.21010625, 0.2490125, 0.19885, 0.2007, 0.212715625, 0.20246875, 0.20388125, 0.189261875, 0.19821875, 0.21990125, 0.22239, 0.18936875, 0.21725625, 0.22524375, 0.21182125, 0.21001, 0.194010625, 0.283676875, 0.243328125, 0.25261875, 0.266931875, 0.252025, 0.22314, 0.20326, 0.196634375, 0.24235, 0.18902125, 0.19184125, 0.19101875, 0.197071875, 0.23043375, 0.199403125, 0.20333125, 0.228890625, 0.22547, 0.288, 0.19213125, 0.25863375, 0.211918125, 0.25370375, 0.2007875, 0.224925, 0.203859375, 0.26780625, 0.210178125, 0.20288125, 0.23163, 0.19904625, 0.212703125, 0.20809375, 0.190560625, 0.21236875, 0.24046875, 0.2005875, 0.25791125, 0.2003625, 0.2177, 0.2486675, 0.22462625, 0.21061875, 0.194559375, 0.22260625, 0.23338625, 0.232601875, 0.19281875, 0.21046875, 0.257008125, 0.19278, 0.19281875, 0.2198125, 0.27402, 0.190655625, 0.194045625, 0.215575, 0.20236875, 0.26179375, 0.190718125, 0.225728125, 0.20205, 0.21561, 0.194271875, 0.21241875, 0.19508, 0.19076, 0.237946875, 0.224534375, 0.2456, 0.27322, 0.266146875, 0.2076625, 0.198151875, 0.209371875, 0.213490625, 0.198285625, 0.21204625, 0.19877125, 0.24530625, 0.199721875, 0.208353125, 0.216, 0.30402375, 0.20380625, 0.33487875, 0.1925, 0.20198125, 0.2265, 0.19962, 0.24649375, 0.189368125, 0.24690375, 0.25446875, 0.39358, 0.19909375, 0.22356, 0.283078125, 0.230324375, 0.190985625, 0.46059375, 0.193434375, 0.20115, 0.19226, 0.2185125, 0.196188125, 0.2304625, 0.24460625, 0.200315625, 0.194714375, 0.24205, 0.22944375, 0.21270625, 0.21740375, 0.1913, 0.205053125, 0.23725, 0.195465625, 0.210746875, 0.188683125, 0.301484375, 0.21008125, 0.195311875, 0.29839, 0.195015625, 0.31552, 0.202110625, 0.20470625, 0.2565, 0.20613125, 0.203959375, 0.24028125, 0.208893125, 0.19567875, 0.18903, 0.204134375, 0.23204375, 0.19603, 0.227303125, 0.2697, 0.24155625, 0.22951125, 0.22494375, 0.199414375, 0.20753125, 0.26745, 0.20768125, 0.22572125, 0.19934375, 0.23546, 0.28591875, 0.25495625, 0.216421875, 0.23381875, 0.20385, 0.2250875, 0.24666125, 0.253625, 0.20725, 0.23186875, 0.211903125, 0.188218125, 0.22754625, 0.20749375, 0.199875, 0.193021875, 0.23465, 0.237145, 0.23135625, 0.226626875, 0.19943625, 0.22381, 0.197, 0.22501875, 0.259934375, 0.199121875, 0.191646875, 0.22621875, 0.22896875, 0.19929375, 0.194726875, 0.23896875, 0.20633625, 0.191668125, 0.212540625, 0.25825, 0.19513, 0.2582, 0.26325, 0.20339375, 0.2063, 0.21936, 0.20421875, 0.29390875, 0.195280625, 0.22526, 0.27656875, 0.219821875, 0.21178125, 0.202884375, 0.209021875, 0.191134375, 0.217859375, 0.191808125, 0.2241, 0.192870625, 0.27630625, 0.20128125, 0.25956875, 0.2564, 0.22820625, 0.23052875, 0.2690875, 0.208534375, 0.222075, 0.213703125, 0.23449375, 0.21174875, 0.21024375, 0.22, 0.25616875, 0.213125, 0.2214, 0.22096625, 0.21724375, 0.20001375, 0.204646875, 0.211078125, 0.206109375, 0.207254375, 0.22180625, 0.19821625, 0.26641125, 0.229405, 0.19177875, 0.18391875, 0.203021875, 0.193919375, 0.19652625, 0.270926875, 0.238313125, 0.21700875, 0.201900625, 0.23491875, 0.2198025, 0.1978125, 0.19726875, 0.31263, 0.20961875, 0.192110625, 0.22669375, 0.221365625, 0.21313375, 0.18835, 0.25409125, 0.2339, 0.2792, 0.213825, 0.23934375, 0.242196875, 0.207434375, 0.20985125, 0.1947125, 0.20846, 0.225590625, 0.1914275, 0.2002, 0.18950875, 0.224065625, 0.19958625, 0.188733125, 0.19144375, 0.25235, 0.18968, 0.198146875, 0.237053125, 0.22490625, 0.2665, 0.19576, 0.22165625, 0.20730875, 0.19286, 0.25065, 0.20984375, 0.210584375, 0.21443125, 0.21935625, 0.24418125, 0.20421875, 0.24346875, 0.21886875, 0.19513125, 0.251953125, 0.191979375, 0.193065625, 0.22903125, 0.208, 0.188665625, 0.21175, 0.236268125, 0.21108, 0.206325, 0.2030125, 0.2231, 0.212428125, 0.2451875, 0.21433875, 0.211303125, 0.18993125, 0.28528125, 0.24027125, 0.27271875, 0.20856875, 0.195, 0.202540625, 0.219258125, 0.21737375, 0.19168125, 0.37750625, 0.22490625, 0.221498125, 0.224625, 0.191018125, 0.22421875, 0.2195, 0.24001875, 0.28315625, 0.2599, 0.20753375, 0.3825, 0.269269375, 0.20296875, 0.21073125, 0.200255, 0.25149375, 0.21809375, 0.286078125, 0.23295, 0.1985, 0.195358125, 0.24530625, 0.20779625, 0.22553, 0.19065625, 0.22866, 0.29665, 0.3074875, 0.219851875, 0.191965625, 0.216325, 0.262759375, 0.197665625, 0.23561875, 0.27459375, 0.25775, 0.23691875, 0.19998125, 0.189615625, 0.23994, 0.274390625, 0.23640625, 0.200248125, 0.194914375, 0.23378, 0.205453125, 0.19818375, 0.237230625, 0.20810625, 0.196959375, 0.26616, 0.19819625, 0.21888375, 0.198, 0.23338125, 0.26775, 0.23223625, 0.24732, 0.21361875, 0.21078125, 0.22559375, 0.19835, 0.191240625, 0.25962, 0.25575, 0.23546875, 0.19298125, 0.22795, 0.198661875, 0.19154625, 0.19661875, 0.201046875, 0.20980625, 0.219896875, 0.19844875, 0.1881125, 0.191821875, 0.246965625, 0.21726875, 0.193484375, 0.20303125, 0.2538, 0.282315, 0.2178, 0.20271875, 0.227455, 0.23934375, 0.19841875, 0.191489375, 0.19754, 0.200909375, 0.2286, 0.191246875, 0.200846875, 0.22625, 0.21744875, 0.26585625, 0.19621875, 0.26792, 0.22206875, 0.2202, 0.25553125, 0.238465625, 0.227215625, 0.1973, 0.224303125, 0.20135625, 0.236, 0.20893125, 0.199159375, 0.19005625, 0.21593, 0.21516875, 0.193274375, 0.22395375, 0.20268125, 0.19368125, 0.332169375, 0.188616875, 0.225503125, 0.19535625, 0.190208125, 0.28443125, 0.26555, 0.21354375, 0.25355, 0.210909375, 0.3312, 0.19848125, 0.246675, 0.19600125, 0.34912, 0.206796875, 0.19686125, 0.20275, 0.217185625, 0.21001875, 0.19348125, 0.26193, 0.21166875, 0.205621875, 0.21507, 0.23732875, 0.22141875, 0.19771375, 0.220356875, 0.21302625, 0.22679, 0.22466875, 0.20824, 0.2142, 0.19868125, 0.322840625, 0.27495, 0.19524, 0.27, 0.2233875, 0.26865, 0.22405, 0.2334375, 0.194115625, 0.194655625, 0.202643125, 0.222044375, 0.20577875, 0.20260125, 0.19047625, 0.250325, 0.222703125, 0.24979375, 0.19221375, 0.19055875, 0.21243125, 0.21034375, 0.197906875, 0.20815, 0.191421875, 0.230289375, 0.19078625, 0.22281125, 0.202678125, 0.2211, 0.2679, 0.203384375, 0.190065625, 0.253855, 0.25921875, 0.26578125, 0.21353125, 0.20766875, 0.22699375, 0.22476, 0.20748125, 0.21256875, 0.227685, 0.1933, 0.21143125, 0.23165, 0.232171875, 0.21543125, 0.19674375, 0.21832, 0.236475, 0.2252, 0.260578125, 0.209558125, 0.222375, 0.2540875, 0.239053125, 0.27440625, 0.207246875, 0.253725, 0.18809625, 0.237778125, 0.207771875, 0.212015625, 0.212523125, 0.20761375, 0.19740625, 0.2205, 0.282795, 0.21835, 0.22718, 0.2855575, 0.195803125, 0.22848125, 0.26865, 0.189949375, 0.19153125, 0.199559375, 0.221490625, 0.194434375, 0.2743375, 0.19728875, 0.196686875, 0.22466875, 0.22967875, 0.228353125, 0.187178125, 0.19618125, 0.206699375, 0.21298125, 0.200274375, 0.2388, 0.194440625, 0.24894375, 0.21274375, 0.221759375, 0.2201325, 0.235415625, 0.252375625, 0.1998, 0.21280625, 0.191288125, 0.20678125, 0.192443125, 0.190564375, 0.21871875, 0.20705625, 0.3162, 0.19658125, 0.203711875, 0.25621875, 0.208281875, 0.19435625, 0.22040625, 0.22388125, 0.206578125, 0.22050625, 0.20605, 0.189553125, 0.237995, 0.25830625, 0.27113125, 0.233821875, 0.23431875, 0.25978125, 0.31926, 0.2156575, 0.210510625, 0.19382125, 0.189899375, 0.19778125, 0.189079375, 0.2597, 0.20636875, 0.198071875, 0.27435, 0.2688, 0.193446875, 0.206571875, 0.19551, 0.19312125, 0.20970625, 0.26059375, 0.203604375, 0.234064375, 0.238124375, 0.2156825, 0.20556375, 0.209430625, 0.225253125, 0.2297625, 0.25363, 0.250734375, 0.20813125, 0.239166875, 0.21234375, 0.19865, 0.20574875, 0.22885625, 0.209009375, 0.226503125, 0.194090625, 0.189740625, 0.19956875, 0.25908, 0.22732125, 0.26385, 0.23445, 0.2153375, 0.264, 0.189366875, 0.1983, 0.191978125, 0.2283875, 0.2340375, 0.25690625, 0.25689375, 0.19850125, 0.23795, 0.240688125, 0.2190475, 0.27675, 0.22806, 0.21491875, 0.21567875, 0.21720375, 0.20443125, 0.199983125, 0.22940625, 0.193946875, 0.2153125, 0.25551125, 0.239371875, 0.200665625, 0.200304375, 0.32990625, 0.19084375, 0.191041875, 0.19079375, 0.20399375, 0.19374375, 0.19171375, 0.2364, 0.21572, 0.20385625, 0.20676875, 0.23315625, 0.197505625, 0.27771625, 0.22165625, 0.2132, 0.1904, 0.22433625, 0.194415625, 0.20615, 0.197663125, 0.192914375, 0.245258125, 0.19134375, 0.21268125, 0.231801875, 0.19029375, 0.28375, 0.20944, 0.2171, 0.21745, 0.193371875, 0.193409375, 0.21621875, 0.2031625, 0.23238, 0.22092875, 0.205603125, 0.2302875, 0.209921875, 0.24984375, 0.197103125, 0.20244125, 0.24511875, 0.24985, 0.233405, 0.1899125, 0.19170375, 0.22221, 0.22725, 0.21706, 0.208140625, 0.231499375, 0.19010625, 0.240225, 0.19278125, 0.226090625, 0.245484375, 0.19996, 0.205208125, 0.222, 0.202313125, 0.22286875, 0.30164375, 0.21024, 0.20028125, 0.231715625, 0.215784375, 0.27155375, 0.19156125, 0.212184375, 0.239611875, 0.18847875, 0.211965625, 0.26203375, 0.189108125, 0.22009375, 0.23218125, 0.19369875, 0.196850625, 0.19800625, 0.201491875, 0.204184375, 0.240046875, 0.20276875, 0.188696875, 0.21452125, 0.20305, 0.22795875, 0.21196875, 0.20787625, 0.22810625, 0.19605, 0.23984375, 0.18398125, 0.29835, 0.207003125, 0.19206, 0.19165, 0.26925, 0.203439375, 0.224030625, 0.20570625, 0.20640625, 0.21264, 0.19418125, 0.21759375, 0.255975, 0.23146, 0.3705, 0.19853375, 0.30258, 0.19675625, 0.21416875, 0.19646875, 0.1968525, 0.22976375, 0.2275, 0.19905625, 0.2265125, 0.21523125, 0.200265625, 0.190141875, 0.31875, 0.2278125, 0.1840125, 0.191834375, 0.1948875, 0.22598125, 0.254334375, 0.26634, 0.214409375, 0.20639375, 0.193996875, 0.22246875, 0.25614, 0.23017, 0.213655, 0.2463, 0.2066125, 0.21338125, 0.2102, 0.190389375, 0.22493125, 0.19721875, 0.20402375, 0.251859375, 0.211740625, 0.20938625, 0.21628125, 0.2090275, 0.221939375, 0.190540625, 0.245166875, 0.1929475, 0.199853125, 0.19656125, 0.21036, 0.200934375, 0.2186, 0.2083375, 0.23671875, 0.210803125, 0.199590625, 0.20283125, 0.19868625, 0.200511875, 0.2336125, 0.19172125, 0.20677375, 0.2222325, 0.21675, 0.215296875, 0.30249375, 0.232183125, 0.236131875, 0.18393125, 0.190021875, 0.21721875, 0.229525, 0.192246875, 0.19690625, 0.2245375, 0.19953, 0.2321625, 0.2219625, 0.23798125, 0.259665625, 0.20408125, 0.19056, 0.2496, 0.1992125, 0.27189375, 0.205896875, 0.2925, 0.194971875, 0.206478125, 0.19881875, 0.191060625, 0.20955, 0.231525, 0.1939, 0.22126, 0.18870375, 0.204725, 0.22230625, 0.22403, 0.204046875, 0.2696625, 0.30195, 0.2603, 0.210096875, 0.20167875, 0.19061875, 0.25446875, 0.20144625, 0.29378125, 0.21050625, 0.20599375, 0.2356875, 0.2345825, 0.229865625, 0.22185625, 0.26825625, 0.20295625, 0.19533125, 0.3145, 0.20693125, 0.19707625, 0.21784125, 0.216390625, 0.193576875, 0.1963025, 0.203495625, 0.22478125, 0.28692, 0.20675, 0.203458125, 0.203080625, 0.211009375, 0.18829375, 0.19666875, 0.20046875, 0.238371875, 0.1931875, 0.22976875, 0.26139375, 0.20510875, 0.191509375, 0.23704375, 0.202028125, 0.208265625, 0.21056875, 0.23555625, 0.197409375, 0.22365, 0.2927, 0.232128125, 0.20815625, 0.191268125, 0.195984375, 0.222196875, 0.2728, 0.231249375, 0.22128375, 0.20060625, 0.214903125, 0.2146, 0.20513125, 0.2044, 0.2085, 0.1945725, 0.267460625, 0.211721875, 0.263053125, 0.22353125, 0.21108375, 0.2192125, 0.20545, 0.23604375, 0.21328375, 0.19663875, 0.18848125, 0.25171875, 0.19953125, 0.23084875, 0.190440625, 0.22603, 0.211503125, 0.22398625, 0.21708125, 0.3180125, 0.194236875, 0.236503125, 0.2276625, 0.1994625, 0.27801375, 0.21468625, 0.197890625, 0.23755, 0.209315625, 0.19936875, 0.20506875, 0.212325, 0.190409375, 0.20202, 0.209896875, 0.25078125, 0.19653125, 0.21059375, 0.1943, 0.220533125, 0.20983125, 0.200445, 0.19160875, 0.23214375, 0.23015, 0.25089375, 0.28719375, 0.258969375, 0.21344875, 0.19452125, 0.19373, 0.19789375, 0.23125625, 0.22396875, 0.1979975, 0.26003, 0.23196, 0.190434375, 0.19755625, 0.20322375, 0.23196875, 0.20081875, 0.241859375, 0.21096, 0.21572125, 0.21336875, 0.22158125, 0.225735, 0.19479375, 0.205715625, 0.19241875, 0.19838125, 0.21691875, 0.237286875, 0.23975625, 0.19186, 0.19846, 0.24384, 0.197955, 0.21524375, 0.22539, 0.266475, 0.21372875, 0.19676875, 0.19238, 0.21865, 0.21650625, 0.223990625, 0.196651875, 0.19575625, 0.25558, 0.191193125, 0.1911, 0.22596875, 0.20701875, 0.22424375, 0.21194, 0.23998125, 0.206975, 0.21798125, 0.2068, 0.325135625, 0.198453125, 0.20166125, 0.240981875, 0.20418125, 0.204840625, 0.270895, 0.2717, 0.1933925, 0.20148125, 0.22982375, 0.209746875, 0.19844375, 0.27763125, 0.22206875, 0.22466875, 0.22724375, 0.187834375, 0.228046875, 0.27945, 0.243525, 0.237590625, 0.19563625, 0.19389375, 0.231414375, 0.207890625, 0.209656875, 0.24415, 0.20605625, 0.22264, 0.204728125, 0.220675, 0.234984375, 0.22899875, 0.201520625, 0.22710625, 0.191571875, 0.21438125, 0.213709375, 0.19711875, 0.200914375, 0.19309625, 0.22140125, 0.21234375, 0.20515625, 0.208950625, 0.267, 0.20860375, 0.27909375, 0.210771875, 0.27219125, 0.24065625, 0.22306, 0.21844, 0.1964, 0.193971875, 0.22769, 0.19116875, 0.20078375, 0.26632625, 0.277525, 0.2305125, 0.199575, 0.194565625, 0.232271875, 0.2224, 0.23158125, 0.23449375, 0.201571875, 0.21696125, 0.187929375, 0.228675, 0.26095, 0.243471875, 0.21190125, 0.24534, 0.254734375, 0.195360625, 0.19195, 0.271270625, 0.3, 0.2994, 0.21367125, 0.26775625, 0.2234025, 0.20157, 0.250434375, 0.193674375, 0.2608, 0.19794, 0.21021375, 0.22724375, 0.188834375, 0.27746125, 0.18966625, 0.21528125, 0.22808625, 0.207921875, 0.188321875, 0.21220625, 0.2198625, 0.24259, 0.22, 0.20952625, 0.22454, 0.25614375, 0.2150375, 0.215811875, 0.193214375, 0.28125, 0.20612375, 0.244194375, 0.20654375, 0.347475, 0.277578125, 0.208193125, 0.25275, 0.2315, 0.24005, 0.1938925, 0.24226875, 0.21306875, 0.19700875, 0.23264, 0.23265, 0.236353125, 0.20048375, 0.20794375, 0.2643275, 0.20265875, 0.23526, 0.240709375, 0.30284875, 0.20710875, 0.212771875, 0.23953125, 0.22750875, 0.222128125, 0.20878125, 0.26934375, 0.20020875, 0.192696875, 0.19191625, 0.191594375, 0.26211875, 0.217175, 0.2583, 0.20466875, 0.2247875, 0.207353125, 0.254224375, 0.203409375, 0.19836125, 0.233249375, 0.219609375, 0.20444375, 0.2189075, 0.26189375, 0.222953125, 0.19097125, 0.24320875, 0.21386875, 0.209975, 0.2277, 0.28655625, 0.200428125, 0.19206125, 0.253464375, 0.197916875, 0.19726, 0.195340625, 0.197234375, 0.21390625, 0.27069375, 0.223895625, 0.20095625, 0.29982375, 0.202415625, 0.19683125, 0.2727, 0.21311875, 0.20546875, 0.19975625, 0.20689625, 0.19844375, 0.23236875, 0.204371875, 0.22978, 0.220478125, 0.198190625, 0.28333125, 0.2162925, 0.19392875, 0.198928125, 0.19680625, 0.19113125, 0.190996875, 0.208350625, 0.223251875, 0.19729375, 0.26412, 0.215765625, 0.193510625, 0.233414375, 0.19509375, 0.266315625, 0.241884375, 0.22318, 0.20295, 0.21696, 0.210586875, 0.20668, 0.24195, 0.22048125, 0.192084375, 0.19920625, 0.22734375, 0.2344125, 0.26834, 0.190933125, 0.21731875, 0.2611625, 0.20978125, 0.196545625, 0.230246875, 0.20723125, 0.21089375, 0.195126875, 0.21195, 0.21100625, 0.189384375, 0.2385, 0.2356475, 0.20921125, 0.238395625, 0.25139375, 0.21503375, 0.19668125, 0.20515, 0.199973125, 0.196059375, 0.196471875, 0.19603625, 0.246064375, 0.2510625, 0.19216125, 0.21128125, 0.3026, 0.244698125, 0.28424, 0.2349375, 0.22794375, 0.19865125, 0.2433675, 0.207721875, 0.22318, 0.196784375, 0.23524875, 0.21626, 0.20366875, 0.24628125, 0.192478125, 0.19028875, 0.2088375, 0.1918, 0.23279375, 0.238308125, 0.196970625, 0.23390625, 0.225240625, 0.2841125, 0.2262, 0.22793125, 0.21890625, 0.19065, 0.22829375, 0.196070625, 0.218234375, 0.19332125, 0.20209375, 0.195484375, 0.245559375, 0.2098, 0.19113, 0.19847625, 0.20319375, 0.200575, 0.2195625, 0.27130875, 0.2225, 0.22675, 0.284, 0.217978125, 0.21365625, 0.2104, 0.196693125, 0.23211875, 0.23139375, 0.191284375, 0.192615625, 0.26, 0.227013125, 0.19873125, 0.22453125, 0.2085, 0.2446, 0.21775625, 0.19916875, 0.26175, 0.29678, 0.19180125, 0.2268875, 0.19614375, 0.20135625, 0.21045875, 0.2817, 0.27329625, 0.256, 0.289, 0.225305, 0.192096875, 0.190744375, 0.224013125, 0.1929925, 0.2110375, 0.20546875, 0.19880625, 0.2316, 0.20068125, 0.36898125, 0.25058, 0.2133, 0.19705, 0.220359375, 0.23321, 0.195721875, 0.21825, 0.24114375, 0.22093375, 0.2775, 0.2733, 0.28645, 0.235226875, 0.2537, 0.220055, 0.236548125, 0.209109375, 0.20715625, 0.258, 0.21254375, 0.207096875, 0.20965625, 0.3504, 0.19330625, 0.19875625, 0.21858375, 0.212625, 0.215563125, 0.199840625, 0.2262025, 0.213759375, 0.24655, 0.200059375, 0.22640625, 0.205958125, 0.198455625, 0.20183, 0.2085, 0.192365625, 0.238359375, 0.23785, 0.207234375, 0.204306875, 0.2144, 0.198109375, 0.232940625, 0.2645625, 0.187765625, 0.2108, 0.242, 0.191590625, 0.199046875, 0.2498, 0.20440375, 0.24959375, 0.194371875, 0.25752875, 0.19135, 0.19056875, 0.19702, 0.19318625, 0.2558, 0.2178875, 0.22736125, 0.19502875, 0.22453125, 0.205325, 0.23122125, 0.218775, 0.2561, 0.21005625, 0.22812375, 0.244555625, 0.18993125, 0.192058125, 0.198666875, 0.219290625, 0.2242, 0.258599375, 0.281765625, 0.211796875, 0.24528, 0.208260625, 0.22173125, 0.21221125, 0.20464375, 0.305521875, 0.29971125, 0.24464, 0.20456125, 0.219015, 0.25148375, 0.21711875, 0.20314, 0.193309375, 0.201953125, 0.26204625, 0.225409375, 0.2752, 0.198659375, 0.209859375, 0.21386375, 0.25353375, 0.21876875, 0.21084375, 0.2093, 0.201928125, 0.23428125, 0.192478125, 0.231008125, 0.19660625, 0.197, 0.204003125, 0.19698125, 0.201115, 0.19578875, 0.24958125, 0.19983, 0.206415625, 0.2457, 0.190916875, 0.19205625, 0.200190625, 0.199375, 0.19745, 0.213515, 0.2696, 0.25842, 0.195420625, 0.22354375, 0.200468125, 0.22905625, 0.2662625, 0.22320625, 0.23484375, 0.18730625, 0.20099125, 0.22453125, 0.23953, 0.2264, 0.22336625, 0.212690625, 0.2453, 0.228765625, 0.22481875, 0.18970625, 0.188803125, 0.19403, 0.19656875, 0.24525, 0.190173125, 0.195871875, 0.240219375, 0.21581875, 0.20494375, 0.282465, 0.259133125, 0.20915, 0.196950625, 0.210178125, 0.19034875, 0.218055, 0.22281, 0.219765625, 0.278321875, 0.25472375, 0.234938125, 0.232540625, 0.23908125, 0.191400625, 0.190784375, 0.19604875, 0.19664375, 0.219921875, 0.194640625, 0.3041475, 0.203665625, 0.196033125, 0.26363125, 0.19902125, 0.20318125, 0.241246875, 0.19608125, 0.201346875, 0.18825, 0.237840625, 0.19820875, 0.1937, 0.20098, 0.202809375, 0.196079375, 0.2862, 0.2216, 0.213853125, 0.23093125, 0.194506875, 0.19451875, 0.20761875, 0.23801125, 0.2296125, 0.221171875, 0.202775, 0.20009375, 0.25190625, 0.23515, 0.240921875, 0.211884375, 0.197984375, 0.2712375, 0.22503125, 0.26190625, 0.193538125, 0.284325, 0.212996875, 0.21005, 0.19402875, 0.211990625, 0.19227625, 0.217409375, 0.2867, 0.3108025, 0.21140625, 0.22976625, 0.24217875, 0.203453125, 0.20257375, 0.21070625, 0.222075, 0.27112, 0.320821875, 0.2496, 0.19655625, 0.19743, 0.2138375, 0.20102, 0.194723125, 0.22979375, 0.37389375, 0.191881875, 0.26955625, 0.266953125, 0.20816875, 0.23390625, 0.20676875, 0.25649375, 0.195853125, 0.2106125, 0.22718, 0.21046875, 0.24128, 0.19725, 0.23206875, 0.218075, 0.20890875, 0.210140625, 0.18408125, 0.19489375, 0.26743125, 0.19344, 0.26015625, 0.195078125, 0.198543125, 0.27231875, 0.19685625, 0.2524, 0.208788125, 0.210146875, 0.21111125, 0.20197375, 0.200353125, 0.212578125, 0.18848, 0.22696625, 0.21203125, 0.27001875, 0.207728125, 0.295321875, 0.20841875, 0.19271875, 0.21631875, 0.18938125, 0.19523, 0.25228, 0.20425, 0.19541875, 0.2214125, 0.235978125, 0.2556, 0.22336875, 0.28874, 0.22718125, 0.1982, 0.21162, 0.39304, 0.21775625, 0.20275625, 0.19315, 0.19971875, 0.20628125, 0.2199525, 0.19070875, 0.221859375, 0.202398125, 0.2299, 0.19803375, 0.26248, 0.21145, 0.23273875, 0.190808125, 0.233528125, 0.25423125, 0.21861875, 0.226915625, 0.20033125, 0.213159375, 0.25204, 0.2167825, 0.210153125, 0.19677375, 0.19303625, 0.26859375, 0.20529375, 0.243790625, 0.199215625, 0.21338125, 0.19925625, 0.19611875, 0.200589375, 0.25167125, 0.205315625, 0.19911125, 0.21076, 0.23054625, 0.23683125, 0.243, 0.26491875, 0.26510625, 0.191743125, 0.23390625, 0.19329375, 0.227146875, 0.233905, 0.206411875, 0.188809375, 0.2272125, 0.25489375, 0.20458375, 0.22183125, 0.212234375, 0.237775, 0.223453125, 0.208546875, 0.25062, 0.206, 0.22254875, 0.2319875, 0.20186, 0.192134375, 0.34, 0.21793125, 0.1889925, 0.20880625, 0.191491875, 0.2037525, 0.2487375, 0.19399375, 0.2118125, 0.26008125, 0.198939375, 0.252075, 0.233678125, 0.190421875, 0.2221775, 0.210701875, 0.24660625, 0.341200625, 0.22452875, 0.22705, 0.219603125, 0.2388, 0.192464375, 0.21626875, 0.19913125, 0.21386875, 0.30694, 0.22674875, 0.246803125, 0.22504375, 0.263765625, 0.193138125, 0.22983, 0.1988825, 0.19076, 0.21011875, 0.224203125, 0.247134375, 0.18926, 0.22435, 0.198609375, 0.18934375, 0.229621875, 0.2109, 0.26751625, 0.23619, 0.2808, 0.2381875, 0.20555, 0.210978125, 0.20811, 0.208778125, 0.24328125, 0.23743125, 0.245271875, 0.20019375, 0.19261625, 0.193036875, 0.217225, 0.196071875, 0.2781, 0.23095375, 0.243825, 0.19663, 0.223606875, 0.20035, 0.2186525, 0.189371875, 0.196061875, 0.21115625, 0.2137625, 0.3757, 0.203184375, 0.24096, 0.21457, 0.28285875, 0.2002625, 0.259409375, 0.19716875, 0.25574, 0.19193375, 0.201586875, 0.275825, 0.19748125, 0.22429, 0.19756875, 0.20967, 0.236525, 0.21784375, 0.18926125, 0.22271875, 0.205278125, 0.1902675, 0.19973375, 0.21946875, 0.204078125, 0.24778125, 0.23439375, 0.19141375, 0.250875, 0.192266875, 0.2115, 0.197565, 0.2633725, 0.23000625, 0.21098, 0.188839375, 0.21967375, 0.24048125, 0.193310625, 0.191995625, 0.2506, 0.265646875, 0.20120625, 0.24893, 0.20220625, 0.19938125, 0.25559375, 0.22378125, 0.21107125, 0.21963125, 0.2391375, 0.2567925, 0.190193125, 0.23035625, 0.193278125, 0.1919875, 0.1913125, 0.206696875, 0.252, 0.2243, 0.20025, 0.210609375, 0.218669375, 0.2108, 0.195619375, 0.19238125, 0.246340625, 0.19304375, 0.27593125, 0.1957125, 0.26065, 0.218, 0.191271875, 0.22623, 0.193809375, 0.19268125, 0.24936875, 0.18988125, 0.19336875, 0.218589375, 0.23465, 0.23986125, 0.22174875, 0.20115625, 0.21453125, 0.205915625, 0.254096875, 0.2514, 0.22432875, 0.22653, 0.22995, 0.24446, 0.229815, 0.24026125, 0.24013125, 0.251390625, 0.20408125, 0.23356125, 0.22157875, 0.21389375, 0.21077375, 0.189971875, 0.208755, 0.238286875, 0.21465625, 0.200140625, 0.193631875, 0.20609125, 0.20488125, 0.22032, 0.197623125, 0.252078125, 0.2114625, 0.237883125, 0.2199, 0.21443125, 0.22789625, 0.26896875, 0.225980625, 0.312, 0.203196875, 0.191821875, 0.206138125, 0.212825, 0.25021875, 0.213503125, 0.20046875, 0.2652, 0.207796875, 0.268228125, 0.2181775, 0.199811875, 0.19758125, 0.2325, 0.23018, 0.233060625, 0.3468, 0.26814375, 0.21763125, 0.192015625, 0.22741, 0.21674625, 0.198218125, 0.1963, 0.21394375, 0.2406, 0.30672, 0.204090625, 0.2221675, 0.20284375, 0.20970625, 0.194288125, 0.2583, 0.18696875, 0.19317875, 0.234, 0.2771, 0.27040625, 0.22643375, 0.202234375, 0.19756875, 0.33921375, 0.22446875, 0.203145625, 0.2395, 0.21013125, 0.261334375, 0.22001875, 0.207884375, 0.198493125, 0.233225, 0.197429375, 0.2429375, 0.2342, 0.195286875, 0.20635625, 0.262301875, 0.2362875, 0.23604375, 0.2118575, 0.2894, 0.21858125, 0.208784375, 0.21849875, 0.19700875, 0.214775, 0.221531875, 0.26624125, 0.201909375, 0.22684375, 0.209171875, 0.27299875, 0.19318125, 0.210203125, 0.210075, 0.20680875, 0.19939375, 0.21875625, 0.22765, 0.2396, 0.266015625, 0.26339375, 0.22188, 0.196666875, 0.21258625, 0.200684375, 0.197385625, 0.22369375, 0.211346875, 0.2313, 0.195819375, 0.233115, 0.22599375, 0.26605, 0.3621, 0.200224375, 0.270078125, 0.24158125, 0.28755, 0.19045375, 0.21625625, 0.2559, 0.23331875, 0.21383125, 0.222923125, 0.190721875, 0.20134375, 0.21185625, 0.29954, 0.21415125, 0.1946625, 0.194069375, 0.240156875, 0.195194375, 0.21112875, 0.217575, 0.25591, 0.20445, 0.25199375, 0.19631875, 0.25425, 0.19131875, 0.24660625, 0.20372125, 0.19598125, 0.235996875, 0.202371875, 0.26769, 0.19941375, 0.2753, 0.226905625, 0.21999625, 0.19580625, 0.31078125, 0.19138125, 0.212774375, 0.195621875, 0.1981625, 0.20276875, 0.258984375, 0.2288075, 0.19805625, 0.21288, 0.22115625, 0.23205625, 0.196445625, 0.20681875, 0.2253, 0.248640625, 0.189503125, 0.21773125, 0.24720125, 0.23273, 0.190109375, 0.207659375, 0.19097375, 0.207965625, 0.19006875, 0.246008125, 0.2617, 0.23011875, 0.2106, 0.38473125, 0.2552, 0.197374375, 0.19216875, 0.227815625, 0.21841875, 0.212275, 0.222578125, 0.20977125, 0.24537375, 0.244003125, 0.196090625, 0.224559375, 0.2212875, 0.229395, 0.25228125, 0.284165625, 0.26288125, 0.20999375, 0.218028125, 0.25861, 0.191043125, 0.20195, 0.2162, 0.2257625, 0.25310625, 0.200271875, 0.22444875, 0.243, 0.20205, 0.22221875, 0.23833125, 0.20978125, 0.187346875, 0.237920625, 0.21405875, 0.23094, 0.22033125, 0.206878125, 0.197346875, 0.242578125, 0.204440625, 0.32589, 0.29293125, 0.207375, 0.22996875, 0.193953125, 0.28335, 0.34903125, 0.2433375, 0.206685, 0.296, 0.1919, 0.20074375, 0.23923125, 0.2616, 0.22068125, 0.21005, 0.22420625, 0.2338875, 0.225208125, 0.294, 0.2635, 0.198659375, 0.20389375, 0.20296875, 0.226290625, 0.20314875, 0.21985625, 0.19969875, 0.1895125, 0.19815, 0.21309125, 0.193230625, 0.23146125, 0.21838, 0.1895, 0.20555625, 0.241580625, 0.19296125, 0.19537625, 0.214059375, 0.26536875, 0.24496875, 0.190044375, 0.188448125, 0.194278125, 0.26049375, 0.25846875, 0.191596875, 0.23134375, 0.20415625, 0.28910625, 0.21390625, 0.193194375, 0.21226125, 0.19445625, 0.190875625, 0.20334375, 0.19614375, 0.195829375, 0.196551875, 0.202583125, 0.2321, 0.23857375, 0.252821875, 0.207584375, 0.188740625, 0.207905, 0.20413125, 0.24669875, 0.19443125, 0.19478125, 0.23283625, 0.226075, 0.193281875, 0.206108125, 0.215128125, 0.2031, 0.193615625, 0.2231325, 0.21059375, 0.3136, 0.2269, 0.193646875, 0.19982625, 0.19413, 0.23865625, 0.20175, 0.27159375, 0.19329375, 0.2023, 0.19971875, 0.252421875, 0.257475, 0.22655, 0.20848375, 0.218753125, 0.2721225, 0.19484625, 0.29548125, 0.2233175, 0.2177775, 0.21145, 0.240703125, 0.208921875, 0.208434375, 0.211753125, 0.193934375, 0.252421875, 0.2128, 0.19264375, 0.22731375, 0.207495625, 0.222509375, 0.22921875, 0.20355, 0.21026875, 0.2124525, 0.288, 0.19573375, 0.18896875, 0.31399, 0.195384375, 0.20546875, 0.19266875, 0.193170625, 0.25993, 0.20094875, 0.29088, 0.26061, 0.21838125, 0.2526875, 0.218956875, 0.20403375, 0.2072375, 0.223878125, 0.24814, 0.1899, 0.21216875, 0.29552, 0.191103125, 0.194671875, 0.21344375, 0.19384375, 0.18915, 0.268546875, 0.258608125, 0.20690625, 0.220998125, 0.39918125, 0.203171875, 0.2488, 0.199771875, 0.21523875, 0.2018375, 0.2031875, 0.218730625, 0.21215, 0.22325, 0.27593125, 0.212234375, 0.251859375, 0.210819375, 0.22447, 0.2613, 0.25128625, 0.22727625, 0.233625, 0.2197125, 0.2381875, 0.228671875, 0.19804375, 0.191839375, 0.201515625, 0.28846875, 0.22870625, 0.19895, 0.23135625, 0.241828125, 0.20688625, 0.21310625, 0.28629375, 0.20698125, 0.25659375, 0.20360375, 0.21713, 0.21331125, 0.19060625, 0.22144375, 0.19176875, 0.26242875, 0.2427, 0.19573625, 0.2461, 0.19323, 0.194636875, 0.22921875, 0.198059375, 0.26955, 0.200484375, 0.252396875, 0.2112, 0.19172875, 0.20441, 0.23695, 0.200846875, 0.2853, 0.2020625, 0.220453125, 0.22190625, 0.204134375, 0.21329375, 0.220140625, 0.193655625, 0.198328125, 0.21379375, 0.20935, 0.22375, 0.2958, 0.213046875, 0.19475, 0.21865, 0.22030875, 0.198934375, 0.19887125, 0.2155425, 0.20784375, 0.19143125, 0.209238125, 0.190425625, 0.231871875, 0.239030625, 0.32765625, 0.205571875, 0.198153125, 0.225649375, 0.26413875, 0.277233125, 0.23665, 0.223346875, 0.205178125, 0.203653125, 0.2283175, 0.20624, 0.20749125, 0.2201625, 0.24244, 0.210255, 0.21825, 0.2560875, 0.19349375, 0.22915, 0.221489375, 0.272053125, 0.20796875, 0.191589375, 0.2395, 0.21750625, 0.324221875, 0.201196875, 0.19228, 0.36125, 0.271875, 0.19501875, 0.202718125, 0.23391875, 0.2087, 0.2545, 0.2674, 0.246234375, 0.22274375, 0.22483125, 0.206409375, 0.203565625, 0.414534375, 0.194830625, 0.203776875, 0.21059375, 0.217350625, 0.224724375, 0.21076125, 0.218484375, 0.2941, 0.22332375, 0.18904375, 0.20635, 0.195229375, 0.22318125, 0.24344, 0.196478125, 0.228515625, 0.2156575, 0.20323125, 0.198609375, 0.30461875, 0.22560625, 0.255711875, 0.21728375, 0.207, 0.21921875, 0.2689125, 0.207396875, 0.284683125, 0.21936125, 0.26114125, 0.2244625, 0.20108125, 0.2098675, 0.235659375, 0.21776, 0.29109375, 0.1917, 0.19331875, 0.224121875, 0.219218125, 0.251578125, 0.204359375, 0.20566875, 0.19575625, 0.19371375, 0.23688, 0.26195, 0.21160125, 0.21033125, 0.22085875, 0.26751625, 0.21391875, 0.22999375, 0.20765625, 0.29280375, 0.23749, 0.25595625, 0.21281875, 0.21576375, 0.2179, 0.191746875, 0.20703125, 0.190746875, 0.201190625, 0.32538, 0.22740375, 0.22779375, 0.20656125, 0.2133, 0.22954625, 0.19876, 0.19306, 0.192828125, 0.21646875, 0.20768125, 0.21311375, 0.20234375, 0.19860625, 0.207815625, 0.20938125, 0.21583125, 0.195323125, 0.205340625, 0.2137875, 0.202353125, 0.323158125, 0.20341875, 0.225940625, 0.19695, 0.20609125, 0.1954, 0.20346875, 0.23565, 0.26126875, 0.19291875, 0.198476875, 0.21465625, 0.22623625, 0.20134375, 0.19965625, 0.203046875, 0.21508125, 0.21276, 0.22677375, 0.201496875, 0.26409375, 0.19859375, 0.214, 0.2266, 0.23265625, 0.23990625, 0.25835, 0.2327, 0.231628125, 0.233671875, 0.19254375, 0.21300125, 0.207784375, 0.299, 0.23465, 0.22506, 0.226371875, 0.289153125, 0.2228, 0.199296875, 0.22071875, 0.21335625, 0.20814375, 0.211509375, 0.208459375, 0.2343, 0.19485625, 0.19825, 0.19842875, 0.22140875, 0.22268125, 0.22608, 0.192468125, 0.19409625, 0.205653125, 0.26904625, 0.20713125, 0.195709375, 0.2014, 0.21351875, 0.204369375, 0.209526875, 0.19822375, 0.213115625, 0.1924, 0.24165625, 0.2743125, 0.20646, 0.22625625, 0.196919375, 0.19584375, 0.21256875, 0.20951875, 0.19229375, 0.2201, 0.2547, 0.2426125, 0.21584, 0.21684, 0.1972, 0.20825, 0.2352, 0.22609375, 0.2024, 0.24260125, 0.21389375, 0.24139375, 0.2984125, 0.278300625, 0.22268, 0.22281, 0.26615625, 0.190684375, 0.21817125, 0.197431875, 0.23234375, 0.257765625, 0.242259375, 0.209215, 0.265640625, 0.296, 0.20678125, 0.19691875, 0.200151875, 0.23351625, 0.203890625, 0.22138125, 0.191565625, 0.259000625, 0.19051125, 0.19928125, 0.2665, 0.2174425, 0.22418125, 0.2612, 0.20671875, 0.29029625, 0.24339375, 0.21559375, 0.20785625, 0.229409375, 0.3018775, 0.21344, 0.196246875, 0.191370625, 0.19202875, 0.195975, 0.24578, 0.22718, 0.2083775, 0.27300625, 0.2072, 0.30144375, 0.20388125, 0.211053125, 0.25973875, 0.21006125, 0.27584, 0.19294, 0.2079, 0.19502875, 0.194961875, 0.22125, 0.19271875, 0.217734375, 0.20562125, 0.20440125, 0.205515625, 0.22413125, 0.19723125, 0.20065625, 0.19482875, 0.306796875, 0.25618, 0.20115, 0.28734375, 0.22864375, 0.198995625, 0.20641875, 0.199975, 0.21890625, 0.196834375, 0.21331, 0.19405, 0.200521875, 0.218903125, 0.19699375, 0.1977, 0.25361875, 0.20046, 0.207471875, 0.34499375, 0.192518125, 0.2805, 0.2244375, 0.2333, 0.24002, 0.22449875, 0.2215125, 0.2020375, 0.2757, 0.20824125, 0.1953875, 0.24040125, 0.19295, 0.21678, 0.23026875, 0.191401875, 0.226428125, 0.2177675, 0.22283125, 0.192404375, 0.19844875, 0.27148125, 0.21298125, 0.203375625, 0.2189125, 0.2584, 0.229440625, 0.207170625, 0.275995, 0.219599375, 0.196090625, 0.2050275, 0.198640625, 0.188355625, 0.206634375, 0.2227125, 0.209449375, 0.19199375, 0.19093, 0.19859125, 0.23305, 0.195090625, 0.22184375, 0.20074375, 0.19560625, 0.25076875, 0.22498125, 0.19828125, 0.26854, 0.19672, 0.194915625, 0.221714375, 0.208501875, 0.25493625, 0.27035, 0.2595125, 0.202415625, 0.20315, 0.19460375, 0.32149125, 0.208498125, 0.18981, 0.207534375, 0.26765625, 0.21458125, 0.26573125, 0.299115, 0.28305, 0.214998125, 0.22592125, 0.2703, 0.193751875, 0.19493125, 0.25875, 0.22359375, 0.19514, 0.257203125, 0.2004975, 0.227046875, 0.18955, 0.19054375, 0.201845, 0.21607375, 0.25074, 0.201055, 0.22407875, 0.193790625, 0.196041875, 0.195986875, 0.195373125, 0.194740625, 0.206071875, 0.2019375, 0.244534375, 0.21693875, 0.2224, 0.208121875, 0.22146125, 0.20464375, 0.20271875, 0.27227625, 0.208671875, 0.2316, 0.204343125, 0.19406875, 0.201103125, 0.21926875, 0.211703125, 0.20483125, 0.23206, 0.25415, 0.23091875, 0.196615625, 0.24238125, 0.22524625, 0.22490625, 0.198046875, 0.333614375, 0.21899375, 0.20159375, 0.21135, 0.2264875, 0.26015625, 0.20576875, 0.191340625, 0.194671875, 0.20636875, 0.19400125, 0.19649375, 0.21236125, 0.21243125, 0.19937, 0.20373625, 0.207115625, 0.21773125, 0.2045, 0.190070625, 0.19212125, 0.19255, 0.29252, 0.232495, 0.2076, 0.21795375, 0.2874375, 0.200639375, 0.30886875, 0.20313125, 0.2406, 0.19363625, 0.198918125, 0.19451875, 0.201034375, 0.19161625, 0.19135625, 0.19840375, 0.25502, 0.20636125, 0.30121875, 0.21617625, 0.20353125, 0.227940625, 0.245578125, 0.227571875, 0.210096875, 0.20125, 0.216640625, 0.207365625, 0.206128125, 0.201403125, 0.233375, 0.23626625, 0.18991875, 0.21916875, 0.194595625, 0.200625, 0.199690625, 0.20538625, 0.26690625, 0.2067875, 0.2433525, 0.21460625, 0.23363125, 0.19675625, 0.229175, 0.213, 0.229134375, 0.19948125, 0.20891375, 0.22635, 0.21278875, 0.23024375, 0.25546875, 0.222835, 0.22839, 0.22916875, 0.204885625, 0.2412375, 0.201215625, 0.22641875, 0.27999375, 0.288096875, 0.217465625, 0.203846875, 0.19677375, 0.21050875, 0.19935, 0.30175, 0.25722, 0.21723125, 0.240178125, 0.20033125, 0.219559375, 0.199159375, 0.21897375, 0.20604375, 0.192746875, 0.206950625, 0.2151625, 0.27571875, 0.19138125, 0.22575, 0.20903625, 0.19743625, 0.203384375, 0.19610125, 0.23323125, 0.19633875, 0.212515625, 0.256434375, 0.248225, 0.19504125, 0.21962625, 0.24052875, 0.20998, 0.24380625, 0.18931, 0.197146875, 0.19629375, 0.19107, 0.21035625, 0.20176875, 0.28185, 0.21205625, 0.201254375, 0.205315625, 0.266501875, 0.1974, 0.21208375, 0.21476, 0.23131375, 0.21944, 0.228386875, 0.195438125, 0.26420875, 0.2526, 0.191553125, 0.25096875, 0.20485, 0.2079875, 0.206190625, 0.21006875, 0.20982125, 0.213275, 0.208445625, 0.19055625, 0.215125, 0.38624, 0.21948, 0.23625, 0.20163625, 0.30515, 0.21281, 0.221175, 0.20625625, 0.19134375, 0.200465625, 0.204255, 0.252715625, 0.22903125, 0.19283125, 0.1964, 0.20797375, 0.192145625, 0.210440625, 0.194528125, 0.199, 0.21090625, 0.192735625, 0.22497, 0.23337, 0.22608125, 0.194333125, 0.20989875, 0.1959875, 0.21177, 0.1893, 0.2127875, 0.22535, 0.215609375, 0.20680625, 0.23041875, 0.1837125, 0.226684375, 0.222171875, 0.26425, 0.19498125, 0.212501875, 0.205371875, 0.18430625, 0.19135, 0.26925, 0.207075, 0.238265625, 0.195698125, 0.1993625, 0.236310625, 0.2485, 0.222465625, 0.203703125, 0.20476875, 0.18991875, 0.210901875, 0.21789375, 0.2677, 0.24224625, 0.1958, 0.23471375, 0.19521375, 0.231796875, 0.25766, 0.193878125, 0.230881875, 0.18826875, 0.18918, 0.193003125, 0.306, 0.193840625, 0.238860625, 0.211625, 0.21924, 0.19207125, 0.18759375, 0.19968125, 0.19391875, 0.23611875, 0.20811875, 0.244621875, 0.235123125, 0.21673125, 0.18986125, 0.23195625, 0.2214875, 0.327706875, 0.2087625, 0.224378125, 0.210078125, 0.2659, 0.20148125, 0.22155625, 0.19676125, 0.2627, 0.23396, 0.213134375, 0.21121875, 0.19705625, 0.21903125, 0.1958125, 0.21591875, 0.26699, 0.200809375, 0.19871875, 0.2806875, 0.24318125, 0.21993125, 0.21851, 0.25555, 0.230115625, 0.33524, 0.25980625, 0.22019625, 0.20827, 0.22410625, 0.32034375, 0.2703, 0.28416, 0.247225, 0.22938125, 0.25053125, 0.20016875, 0.24519, 0.229575, 0.19169375, 0.228320625, 0.195753125, 0.189885625, 0.33525, 0.246128125, 0.22530625, 0.20208125, 0.195059375, 0.194071875, 0.23296875, 0.23705625, 0.19356875, 0.23554875, 0.32113, 0.23255125, 0.21699125, 0.19021875, 0.20235875, 0.20115625, 0.242578125, 0.20751, 0.195055625, 0.200828125, 0.2619, 0.21635625, 0.208290625, 0.252425, 0.2363, 0.19584375, 0.257934375, 0.215753125, 0.22255625, 0.215475, 0.196584375, 0.188528125, 0.22115625, 0.21455, 0.23379375, 0.19508, 0.196903125, 0.25123, 0.2153075, 0.249975, 0.241623125, 0.20798625, 0.191699375, 0.19924, 0.19433375, 0.197895625, 0.2432625, 0.2448, 0.330873125, 0.23205625, 0.259453125, 0.19864375, 0.245175, 0.20915125, 0.206070625, 0.22286, 0.23515, 0.25976, 0.241198125, 0.26646875, 0.2150625, 0.24662, 0.2875125, 0.2054925, 0.206109375, 0.19462, 0.262188125, 0.218, 0.190918125, 0.20285625, 0.24839125, 0.20844, 0.2465, 0.1994125, 0.19276, 0.21185625, 0.21558125, 0.23908125, 0.192965625, 0.26503, 0.2036, 0.2151975, 0.232060625, 0.21783125, 0.2105475, 0.20756875, 0.19158125, 0.26193625, 0.20115625, 0.19232875, 0.19414375, 0.30843, 0.19061875, 0.1928675, 0.19993, 0.201714375, 0.20770625, 0.21120625, 0.23493125, 0.21846125, 0.2342625, 0.2102625, 0.208378125, 0.20709375, 0.338671875, 0.20946875, 0.19994, 0.19758125, 0.33415625, 0.26628375, 0.21138125, 0.23938125, 0.214909375, 0.22540625, 0.229484375, 0.21921, 0.19235, 0.24778, 0.1986, 0.195, 0.333253125, 0.21066875, 0.200125, 0.21504625, 0.24894375, 0.19589375, 0.2029425, 0.21488, 0.2140175, 0.20648125, 0.26569375, 0.21458125, 0.19158375, 0.211059375, 0.209359375, 0.21558625, 0.26168125, 0.242664375, 0.194346875, 0.26857125, 0.18998125, 0.2898, 0.199433125, 0.24980375, 0.196270625, 0.2672, 0.19651, 0.205161875, 0.21025625, 0.207389375, 0.24428125, 0.19959875, 0.30589375, 0.2129125, 0.22431875, 0.19858875, 0.234353125, 0.23125875, 0.20745625, 0.218228125, 0.26713, 0.2336, 0.2268, 0.19281875, 0.20925, 0.24384375, 0.1915, 0.19051875, 0.19161, 0.204653125, 0.19579375, 0.29152, 0.20401, 0.25246875, 0.209559375, 0.2413, 0.384, 0.2112, 0.189775625, 0.2184075, 0.23898, 0.193071875, 0.20708625, 0.2543125, 0.244321875, 0.19663, 0.21376, 0.226493125, 0.20605625, 0.2262, 0.3344, 0.234, 0.211959375, 0.2089875, 0.197351875, 0.25775625, 0.190794375, 0.22886, 0.27593125, 0.19203125, 0.28130625, 0.19039875, 0.19272625, 0.20733125, 0.19608875, 0.216028125, 0.199034375, 0.22080625, 0.19044625, 0.19999375, 0.2565, 0.18914875, 0.21022, 0.30741, 0.2268725, 0.20153125, 0.2504, 0.191484375, 0.23000625, 0.22353375, 0.2099825, 0.22454375, 0.200775, 0.230899375, 0.204089375, 0.24128, 0.19923, 0.2652, 0.23671875, 0.199628125, 0.23944875, 0.189264375, 0.302546875, 0.24, 0.198228125, 0.22582875, 0.21775125, 0.21859125, 0.19437125, 0.221909375, 0.23294375, 0.194690625, 0.223115625, 0.2329875, 0.208214375, 0.23925, 0.219683125, 0.208021875, 0.19020625, 0.1950725, 0.205755625, 0.2508, 0.26551875, 0.20067375, 0.19600375, 0.1888125, 0.188483125, 0.208665, 0.205590625, 0.20242125, 0.19584, 0.197496875, 0.19806, 0.19159375, 0.224553125, 0.2646, 0.21183, 0.221478125, 0.226771875, 0.204735, 0.216758125, 0.19583125, 0.194449375, 0.2685, 0.194, 0.19063125, 0.230970625, 0.240325, 0.2399, 0.21773125, 0.21451875, 0.2467, 0.32331875, 0.24094, 0.2248, 0.189773125, 0.2035, 0.2114625, 0.2625, 0.214115625, 0.19543125, 0.27795, 0.20429125, 0.196524375, 0.215755625, 0.24012, 0.19475125, 0.32540625, 0.19005625, 0.2367, 0.20534375, 0.205865625, 0.203069375, 0.3145, 0.20648875, 0.21033125, 0.23475, 0.2237225, 0.246725, 0.20090625, 0.18364375, 0.20983125, 0.226855, 0.214975, 0.28953125, 0.200235, 0.30345, 0.18764375, 0.20259375, 0.247615625, 0.23660625, 0.212753125, 0.21995375, 0.208159375, 0.21478125, 0.20928125, 0.20668125, 0.241890625, 0.191721875, 0.26975625, 0.2024775, 0.222609375, 0.20240125, 0.26326625, 0.27749375, 0.21208125, 0.29589375, 0.21235875, 0.20747625, 0.232794375, 0.194755625, 0.30668, 0.20251875, 0.232690625, 0.19654625, 0.207859375, 0.1919125, 0.19466875, 0.259675, 0.226961875, 0.19001625, 0.19978375, 0.254575, 0.3195, 0.2126125, 0.23126125, 0.357, 0.192485625, 0.22295375, 0.21463125, 0.2158125, 0.206593125, 0.20163625, 0.225234375, 0.206945, 0.2803125, 0.19569625, 0.239625625, 0.193033125, 0.26177875, 0.2509125, 0.204868125, 0.20001125, 0.2303625, 0.192169375, 0.216021875, 0.22645625, 0.223046875, 0.20560625, 0.227696875, 0.191021875, 0.22420625, 0.312375, 0.22256, 0.214390625, 0.21951625, 0.2907, 0.26562, 0.2444, 0.194870625, 0.30375, 0.20956, 0.22940125, 0.22024375, 0.19646375, 0.19972125, 0.19667875, 0.20911625, 0.220059375, 0.2099, 0.22734375, 0.20886875, 0.22104, 0.233745, 0.246503125, 0.244359375, 0.197690625, 0.210525, 0.210325, 0.248965, 0.19043625, 0.19338125, 0.21863, 0.229753125, 0.20846875, 0.23739375, 0.21149375, 0.21514375, 0.2112125, 0.193009375, 0.222421875, 0.21983125, 0.235528125, 0.223799375, 0.22214375, 0.2528, 0.232, 0.19159875, 0.224949375, 0.2436, 0.20631875, 0.22068, 0.19475, 0.20652375, 0.22163125, 0.1898725, 0.1922525, 0.284855, 0.22016875, 0.19753375, 0.19715, 0.22470625, 0.246750625, 0.2257025, 0.2013, 0.23319375, 0.196299375, 0.24735, 0.219534375, 0.23263125, 0.22429375, 0.201509375, 0.200971875, 0.23043875, 0.1987, 0.204776875, 0.20619, 0.226938125, 0.19209625, 0.19931125, 0.2447, 0.22395, 0.20005875, 0.192765625, 0.2076375, 0.19335625, 0.251015625, 0.20389375, 0.24241875, 0.2307, 0.21058125, 0.201186875, 0.21050625, 0.25446875, 0.283825625, 0.244525, 0.21598625, 0.23129125, 0.206565, 0.205473125, 0.1988775, 0.19706875, 0.19826, 0.20814625, 0.21960625, 0.22643125, 0.2017, 0.19157375, 0.19259375, 0.19711125, 0.20956, 0.19620625, 0.19803375, 0.234128125, 0.24788125, 0.19318, 0.23895, 0.210334375, 0.22502, 0.232009375, 0.243375, 0.21471875, 0.232745625, 0.209635, 0.2027, 0.2374125, 0.19423125, 0.25999375, 0.23553125, 0.26216625, 0.2206875, 0.23938125, 0.211218125, 0.2211, 0.22046375, 0.2445, 0.2594375, 0.2861125, 0.245390625, 0.22197875, 0.20384125, 0.21203625, 0.204343125, 0.1900675, 0.24316375, 0.220225, 0.22554375, 0.26389, 0.24424375, 0.20030625, 0.22013, 0.199026875, 0.195965625, 0.27114375, 0.219459375, 0.20626875, 0.27844375, 0.252078125, 0.228103125, 0.18929375, 0.21728125, 0.23451625, 0.19279625, 0.2664, 0.25496, 0.229515, 0.24653, 0.191603125, 0.203765625, 0.261, 0.197665625, 0.25808125, 0.20638125, 0.19045, 0.223745625, 0.1915875, 0.22026125, 0.28535625, 0.189709375, 0.193678125, 0.255125, 0.199418125, 0.20165, 0.1975, 0.233224375, 0.23585625, 0.27593875, 0.22046875, 0.20895, 0.205184375, 0.25922, 0.24144, 0.21323125, 0.20435625, 0.21045, 0.2321225, 0.21420625, 0.22479375, 0.2109625, 0.210021875, 0.23036375, 0.21206, 0.3267, 0.2409375, 0.29484375, 0.2128, 0.192186875, 0.20608, 0.21654875, 0.196049375, 0.22605, 0.2519625, 0.2418, 0.23643125, 0.19638, 0.248720625, 0.20535625, 0.217996875, 0.226755, 0.2597975, 0.2478125, 0.19163125, 0.20111875, 0.22426, 0.212903125, 0.227391875, 0.219796875, 0.200684375, 0.2079, 0.25185, 0.25149375, 0.2601, 0.22339375, 0.25765625, 0.20248125, 0.203434375, 0.269481875, 0.227040625, 0.2631, 0.3079125, 0.194903125, 0.212740625, 0.190696875, 0.23780125, 0.21375625, 0.198524375, 0.20481125, 0.225625, 0.195379375, 0.194815625, 0.22118, 0.213234375, 0.206171875, 0.202374375, 0.2510625, 0.209578125, 0.21455625, 0.22765, 0.24940625, 0.192196875, 0.187665625, 0.20163125, 0.19204625, 0.255, 0.266659375, 0.218328125, 0.22816875, 0.18934125, 0.2445, 0.29390625, 0.21825625, 0.19782, 0.206965625, 0.24873125, 0.212746875, 0.207546875, 0.237384375, 0.23449375, 0.20188125, 0.22752875, 0.19661875, 0.21801125, 0.23758125, 0.212153125, 0.19635, 0.20606, 0.28958, 0.24609375, 0.21676875, 0.315, 0.20355, 0.20930625, 0.200853125, 0.20355875, 0.26636125, 0.221380625, 0.20914375, 0.20975, 0.198976875, 0.25260625, 0.29818, 0.26109375, 0.2034125, 0.2274, 0.2784, 0.19868125, 0.223925, 0.256425, 0.19954, 0.212, 0.199839375, 0.19314125, 0.26190625, 0.1965725, 0.22448125, 0.189446875, 0.193578125, 0.209609375, 0.19196, 0.19275, 0.20751875, 0.3456, 0.2754, 0.21354375, 0.194768125, 0.200878125, 0.19644375, 0.193886875, 0.207753125, 0.230021875, 0.274773125, 0.21199375, 0.316996875, 0.21814375, 0.20436875, 0.20628125, 0.22055625, 0.286078125, 0.23342125, 0.202228125, 0.216171875, 0.22139375, 0.229125, 0.214103125, 0.2256, 0.3075, 0.224304375, 0.27324375, 0.25512, 0.2006, 0.20098125, 0.31518, 0.19756875, 0.193428125, 0.2411, 0.2586, 0.286875, 0.20638125, 0.242048125, 0.254409375, 0.192121875, 0.23221875, 0.23870625, 0.22074125, 0.221105625, 0.21198875, 0.21330625, 0.21240625, 0.22694375, 0.307125, 0.24640375, 0.22267875, 0.21928, 0.20765, 0.241003125, 0.204, 0.19667625, 0.23124125, 0.2127375, 0.199953125, 0.2304375, 0.218933125, 0.29484375, 0.319334375, 0.2095125, 0.20415625, 0.21050625, 0.250378125, 0.222421875, 0.2548, 0.2588875, 0.22264375, 0.27773625, 0.24065625, 0.27464, 0.31681625, 0.197685625, 0.27444375, 0.201134375, 0.21014375, 0.28446875, 0.212653125, 0.227196875, 0.22349375, 0.19107375, 0.201946875, 0.2134, 0.224560625, 0.191230625, 0.190589375, 0.27435, 0.237525, 0.2140375, 0.23248, 0.237571875, 0.2197175, 0.19039375, 0.19576625, 0.3181125, 0.24489375, 0.201725, 0.241734375, 0.274496875, 0.30055, 0.286884375, 0.194204375, 0.231, 0.20507, 0.21974875, 0.1979125, 0.2025, 0.19911875, 0.22227, 0.2839, 0.2694, 0.2563, 0.2162, 0.2267, 0.222375, 0.278109375, 0.220065, 0.234703125, 0.2423, 0.23506875, 0.21898, 0.1917475, 0.230428125, 0.201203125, 0.26265, 0.198596875, 0.24946875, 0.25835625, 0.285440625, 0.191794375, 0.22655, 0.19436375, 0.2559375, 0.20748, 0.256125, 0.198305, 0.19800625, 0.2676, 0.23391, 0.24958125, 0.22710625, 0.203203125, 0.199998125, 0.23016125, 0.197728125, 0.28059375, 0.29394, 0.200565625, 0.2309, 0.19174125, 0.21956, 0.201059375, 0.32480625, 0.24936, 0.20858125, 0.206274375, 0.20883125, 0.24067, 0.19630625, 0.247786875, 0.2343, 0.3720875, 0.19388625, 0.207976875, 0.21388, 0.18935875, 0.19915, 0.22542875, 0.238680625, 0.19149375, 0.2004, 0.192853125, 0.20375125, 0.249271875, 0.22427, 0.20064375, 0.24463625, 0.191178125, 0.197580625, 0.23376, 0.2086125, 0.19944375, 0.196653125, 0.22393, 0.234, 0.20028125, 0.19447125, 0.2607875, 0.19859375, 0.19703125, 0.254175, 0.231750625, 0.202724375, 0.201045625, 0.229414375, 0.22138125, 0.276281875, 0.2125, 0.19677125, 0.23857375, 0.29767, 0.25089875, 0.22498125, 0.22483125, 0.2064375, 0.206405, 0.20146875, 0.2725, 0.250763125, 0.2009, 0.202664375, 0.189121875, 0.28298625, 0.19357, 0.20258125, 0.19308625, 0.21928125, 0.23357125, 0.216, 0.24703125, 0.392221875, 0.19726875, 0.205609375, 0.2272275, 0.1934, 0.205790625, 0.19025125, 0.25426875, 0.23975, 0.2578875, 0.21706375, 0.206675, 0.188334375, 0.22774125, 0.197675, 0.25638125, 0.20370625, 0.22258125, 0.241264375, 0.19743125, 0.253575, 0.2115, 0.205603125, 0.25216, 0.22926, 0.20841875, 0.19910625, 0.20788, 0.2332, 0.211, 0.20819625, 0.19194625, 0.2107725, 0.227835, 0.26698125, 0.2163875, 0.22385625, 0.222025, 0.20763125, 0.324328125, 0.203180625, 0.21263, 0.22844375, 0.22965, 0.21518125, 0.196085, 0.21920625, 0.20578625, 0.23181375, 0.23341875, 0.20195625, 0.209446875, 0.20761875, 0.20050625, 0.23185, 0.19968375, 0.25502125, 0.21228125, 0.25572, 0.227318125, 0.19403625, 0.23598, 0.21206875, 0.3015, 0.2333925, 0.191263125, 0.19751, 0.22175, 0.2692, 0.189948125, 0.201635625, 0.23359375, 0.2366425, 0.19075, 0.201221875, 0.23975625, 0.197480625, 0.27714, 0.197059375, 0.2191875, 0.195508125, 0.206328125, 0.19693625, 0.19314375, 0.20095625, 0.22370625, 0.1992175, 0.20096875, 0.19724375, 0.198509375, 0.202178125, 0.201675, 0.20247, 0.19203125, 0.1911, 0.2074625, 0.19362125, 0.22674125, 0.206065, 0.25786875, 0.3495, 0.286875, 0.2025875, 0.22076, 0.18439375, 0.21916875, 0.191120625, 0.19082875, 0.1952, 0.1983, 0.218796875, 0.23229375, 0.24710625, 0.2381875, 0.19878125, 0.19395875, 0.23564125, 0.19330625, 0.22630625, 0.2049, 0.24145, 0.2360625, 0.286396875, 0.31197125, 0.272763125, 0.191846875, 0.220860625, 0.248955, 0.192851875, 0.22658, 0.35859375, 0.18885, 0.27444375, 0.20680625, 0.205078125, 0.19624, 0.196253125, 0.23935, 0.2536875, 0.2012625, 0.1992, 0.210671875, 0.196453125, 0.24571875, 0.242203125, 0.2051, 0.253125, 0.22490625, 0.208446875, 0.2458, 0.18379375, 0.196578125, 0.21053125, 0.298828125, 0.224759375, 0.3283125, 0.2429, 0.205543125, 0.2077, 0.18754375, 0.25198125, 0.21575625, 0.301059375, 0.20001875, 0.189545625, 0.21306, 0.2845375, 0.2211, 0.2788, 0.23509875, 0.224758125, 0.19315625, 0.23385, 0.2252625, 0.2151, 0.198246875, 0.26091875, 0.20075, 0.233655, 0.195796875, 0.24165625, 0.19086875, 0.20359625, 0.25188625, 0.202776875, 0.2004975, 0.20096875, 0.190423125, 0.2682, 0.2382, 0.189683125, 0.20264375, 0.36864375, 0.25809375, 0.2929, 0.2863, 0.2154, 0.231184375, 0.312375, 0.20265, 0.191428125, 0.2046375, 0.2948, 0.41770625, 0.22629375, 0.19505, 0.24839125, 0.197199375, 0.20288125, 0.20609375, 0.217011875, 0.2254, 0.294365625, 0.26125, 0.192714375, 0.2526, 0.20420625, 0.24901875, 0.2847875, 0.207903125, 0.225490625, 0.3285, 0.21356, 0.236373125, 0.210728125, 0.2431025, 0.21475875, 0.21256875, 0.27755625, 0.3074025, 0.20073875, 0.2403, 0.20825, 0.20679, 0.20695, 0.19296875, 0.263569375, 0.231975, 0.22314375, 0.20771875, 0.2215, 0.2144, 0.23528125, 0.2595, 0.1995875, 0.200794375, 0.22750625, 0.22, 0.198270625, 0.21319625, 0.197895625, 0.21051375, 0.20910375, 0.24534375, 0.27039, 0.21774625, 0.19126875, 0.24299625, 0.21525625, 0.20655625, 0.2517, 0.232578125, 0.19943125, 0.22598125, 0.23990625, 0.222984375, 0.22950125, 0.2487, 0.215184375, 0.23401125, 0.22805625, 0.22355, 0.19405875, 0.2325, 0.21266875, 0.221355, 0.227364375, 0.224359375, 0.19931875, 0.393805, 0.26316375, 0.22481875, 0.1944, 0.24314, 0.207603125, 0.19193125, 0.19278125, 0.202153125, 0.191659375, 0.20115625, 0.236953125, 0.230971875, 0.203668125, 0.24628125, 0.31826125, 0.21315625, 0.22166875, 0.21003125, 0.247734375, 0.284859375, 0.25933, 0.2109, 0.20397375, 0.205934375, 0.2625, 0.192495625, 0.205088125, 0.1981875, 0.26071875, 0.31683125, 0.22335, 0.221225, 0.214105, 0.220128125, 0.2217375, 0.235546875, 0.27315, 0.20682375, 0.198871875, 0.23394375, 0.2485825, 0.20041875, 0.19378625, 0.222690625, 0.217161875, 0.21607875, 0.20385, 0.20535, 0.199471875, 0.187799375, 0.2288, 0.2165, 0.2114, 0.192678125, 0.25064375, 0.198265625, 0.188946875, 0.22246875, 0.236725, 0.197496875, 0.21119875, 0.202171875, 0.27542125, 0.251440625, 0.23876875, 0.2048, 0.23000625, 0.19879375, 0.190883125, 0.22026, 0.19604375, 0.19923125, 0.2153375, 0.194228125, 0.193049375, 0.2302775, 0.200234375, 0.235725, 0.213534375, 0.222991875, 0.20571875, 0.225703125, 0.20673125, 0.231360625, 0.21567, 0.20815625, 0.251685, 0.2210125, 0.194371875, 0.248011875, 0.205484375, 0.249525, 0.354025, 0.25092375, 0.2144, 0.2105625, 0.20775625, 0.22416375, 0.2499, 0.223759375, 0.1907, 0.21785875, 0.2102, 0.268698125, 0.223128125, 0.199140625, 0.2239, 0.21487125, 0.192115625, 0.239561875, 0.212796875, 0.213390625, 0.23588, 0.2529125, 0.26800625, 0.19932625, 0.225518125, 0.19646625, 0.20153875, 0.2111375, 0.231840625, 0.25854, 0.20132125, 0.23604375, 0.21632125, 0.21818125, 0.27095875, 0.2247375, 0.207671875, 0.223840625, 0.18808125, 0.22021875, 0.20585875, 0.213638125, 0.22583, 0.20649375, 0.22105, 0.2349375, 0.19052875, 0.20111625, 0.2165425, 0.2379375, 0.196965625, 0.1884, 0.21059375, 0.256959375, 0.2269, 0.23261875, 0.19865, 0.231375, 0.234015625, 0.20036875, 0.227234375, 0.236625, 0.244321875, 0.35221875, 0.19291875, 0.21716875, 0.22312625, 0.20875, 0.22490625, 0.3273, 0.2248, 0.253828125, 0.20321375, 0.228, 0.219704375, 0.201609375, 0.25296, 0.27593125, 0.20202125, 0.286025, 0.2420625, 0.3332, 0.211143125, 0.28653, 0.21061875, 0.23590625, 0.210271875, 0.42085625, 0.19118125, 0.21074875, 0.2643125, 0.23934375, 0.1989375, 0.210046875, 0.29648, 0.2182125, 0.221521875, 0.21045625, 0.28064, 0.18993125, 0.21705, 0.21861, 0.21898125, 0.20955625, 0.43061, 0.24018875, 0.238403125, 0.23278125, 0.21665625, 0.228828125, 0.2836875, 0.21036875, 0.24978625, 0.20577625, 0.255, 0.20784375, 0.218784375, 0.22119875, 0.312, 0.19738125, 0.21370625, 0.24661875, 0.1963625, 0.235948125, 0.3315, 0.230558125, 0.224753125, 0.19104875, 0.194239375, 0.191438125, 0.21210625, 0.2621, 0.32693125, 0.21528125, 0.23147, 0.19270625, 0.19766, 0.21200625, 0.213176875, 0.1990025, 0.26902, 0.20247125, 0.227928125, 0.20665625, 0.2375, 0.2231125, 0.18998, 0.22751375, 0.24721875, 0.196271875, 0.19608125, 0.25059375, 0.2341, 0.24809375, 0.23430625, 0.36992, 0.23495625, 0.2239475, 0.202335, 0.2448, 0.19671375, 0.207234375, 0.21950625, 0.203596875, 0.19999125, 0.2097, 0.23695, 0.23427375, 0.2084, 0.21049375, 0.28112, 0.200954375, 0.21161875, 0.24125, 0.238275, 0.21894375, 0.28464375, 0.28152, 0.22665625, 0.20194375, 0.226261875, 0.19830375, 0.212825, 0.195115625, 0.19813625, 0.214353125, 0.20045, 0.203978125, 0.20165625, 0.1897, 0.196640625, 0.36915, 0.197409375, 0.24189, 0.222, 0.20651875, 0.19614375, 0.21408125, 0.20831875, 0.20129125, 0.19045875, 0.26003625, 0.204939375, 0.22203125, 0.225825, 0.20234375, 0.2490125, 0.3502, 0.196455625, 0.210053125, 0.19176875, 0.2257125, 0.21883875, 0.25155625, 0.237, 0.21774375, 0.22595625, 0.24481875, 0.191403125, 0.1980625, 0.203940625, 0.201825, 0.230315625, 0.21134375, 0.206721875, 0.2215625, 0.256078125, 0.227984375, 0.2351375, 0.2181, 0.202970625, 0.206609375, 0.254203125, 0.19114875, 0.21918, 0.205725, 0.20556375, 0.198303125, 0.25765625, 0.20073875, 0.22681125, 0.2630875, 0.19870875, 0.30483125, 0.2218, 0.2142, 0.235425, 0.19873, 0.243, 0.22472375, 0.20553, 0.23188, 0.19925, 0.25990625, 0.250764375, 0.270140625, 0.2429775, 0.209484375, 0.215778125, 0.20978875, 0.201099375, 0.29038125, 0.20369875, 0.228040625, 0.19379125, 0.23855875, 0.236240625, 0.2896, 0.207405, 0.27823125, 0.20319375, 0.25361, 0.245634375, 0.2003, 0.225109375, 0.21969375, 0.23208125, 0.285575, 0.24710125, 0.224290625, 0.2330625, 0.25246875, 0.20107375, 0.3332, 0.3070125, 0.221983125, 0.222540625, 0.26818125, 0.256328125, 0.21954375, 0.27172375, 0.22248125, 0.22783125, 0.206203125, 0.348075, 0.23066, 0.22123125, 0.20089375, 0.2321125, 0.19154, 0.257225, 0.247791875, 0.19193125, 0.2849625, 0.226225, 0.240375, 0.2180675, 0.23339125, 0.21271875, 0.2355, 0.207585, 0.198475, 0.222375, 0.21099, 0.29998125, 0.2326375, 0.253875, 0.24274, 0.226265625, 0.25340625, 0.19235125, 0.28192, 0.21741375, 0.19221875, 0.19451875, 0.210703125, 0.22575875, 0.28384, 0.20226875, 0.2634, 0.2163125, 0.21428125, 0.25215, 0.30048, 0.19631875, 0.24389, 0.23526375, 0.21787125, 0.271790625, 0.23671, 0.1907, 0.235714375, 0.28475, 0.22426875, 0.2054, 0.20548125, 0.2635425, 0.18864, 0.19852, 0.2075, 0.205765625, 0.2173, 0.20885, 0.22237, 0.2670125, 0.20871625, 0.204995625, 0.19981, 0.2601, 0.21540625, 0.211809375, 0.19787875, 0.2118, 0.316814375, 0.242334375, 0.31120625, 0.26878125, 0.23165625, 0.269578125, 0.24, 0.2039, 0.221328125, 0.3276, 0.222775, 0.19401, 0.2033, 0.19832625, 0.19781875, 0.22642375, 0.21792125, 0.26199, 0.2116525, 0.218485625, 0.28985, 0.25635, 0.2879, 0.2135375, 0.20699875, 0.23596875, 0.191903125, 0.285005, 0.24925625, 0.216703125, 0.21077, 0.21584375, 0.19141, 0.20896875, 0.2329, 0.21599375, 0.193360625, 0.199938125, 0.26265, 0.19646875, 0.19884375, 0.231475, 0.21893625, 0.197765, 0.197601875, 0.206078125, 0.19763125, 0.2975, 0.195203125, 0.19389375, 0.21154375, 0.233391875, 0.267455625, 0.22635625, 0.2301, 0.194755625, 0.265878125, 0.25835625, 0.20645625, 0.2595, 0.201015625, 0.2317, 0.26724375, 0.21926, 0.205641875, 0.19320625, 0.254391875, 0.21872625, 0.20739375, 0.197915625, 0.3136, 0.2284025, 0.241775, 0.20462125, 0.19841875, 0.22732, 0.191121875, 0.20762125, 0.191440625, 0.2333375, 0.19982875, 0.222, 0.2982, 0.227979375, 0.21043875, 0.232646875, 0.23294375, 0.20797875, 0.19536, 0.2266975, 0.21664375, 0.2218, 0.194996875, 0.20935625, 0.2737875, 0.19684375, 0.2202125, 0.254825, 0.2126125, 0.20808, 0.198126875, 0.306, 0.199628125, 0.20363625, 0.226628125, 0.2128625, 0.26185, 0.20912625, 0.25228125, 0.217625, 0.25269375, 0.2495475, 0.19624375, 0.21348, 0.19161125, 0.23753, 0.2224625, 0.302005, 0.1903, 0.20782125, 0.23499375, 0.20533125, 0.20325, 0.267675625, 0.19279125, 0.22488125, 0.206846875, 0.27214875, 0.193386875, 0.20648125, 0.2385, 0.19352125, 0.27376375, 0.24395, 0.20745625, 0.217996875, 0.20881875, 0.21804125, 0.20073, 0.22446375, 0.211846875, 0.2551, 0.2248375, 0.23219375, 0.235445625, 0.28640625, 0.21736875, 0.20634375, 0.22695, 0.26859375, 0.265225, 0.20010625, 0.235306875, 0.195875, 0.26840875, 0.2142625, 0.20065625, 0.236259375, 0.231, 0.23936, 0.22396, 0.225671875, 0.22123125, 0.209605, 0.37134375, 0.19415625, 0.219284375, 0.193034375, 0.22408125, 0.198009375, 0.217675, 0.2208275, 0.2153625, 0.22248125, 0.235105, 0.20026875, 0.19877375, 0.215284375, 0.21012125, 0.28871, 0.2536, 0.231496875, 0.205359375, 0.208540625, 0.198821875, 0.204921875, 0.37094, 0.284625, 0.19918, 0.1939875, 0.218290625, 0.2064425, 0.20960125, 0.21328125, 0.2805, 0.22405, 0.20955625, 0.21694375, 0.20039375, 0.192714375, 0.22078125, 0.26764, 0.264828125, 0.233325, 0.19946375, 0.22285625, 0.2133625, 0.273, 0.198803125, 0.23401875, 0.226746875, 0.2193525, 0.20864375, 0.231595, 0.19671, 0.2113875, 0.239709375, 0.24682, 0.19686125, 0.20826, 0.30886875, 0.1992, 0.233766875, 0.202296875, 0.271521875, 0.21505875, 0.25886875, 0.28325, 0.193440625, 0.279, 0.19488625, 0.19830875, 0.23851625, 0.209071875, 0.26871375, 0.2134, 0.241665625, 0.201978125, 0.233671875, 0.19488125, 0.193103125, 0.1904, 0.197625, 0.19605375, 0.22559375, 0.23544, 0.222978125, 0.21374375, 0.203330625, 0.2189875, 0.2146, 0.2312, 0.2253125, 0.19992, 0.20671875, 0.2140125, 0.19455, 0.23613, 0.20576875, 0.255796875, 0.21984375, 0.20915625, 0.20765625, 0.226334375, 0.25913125, 0.22289375, 0.20932, 0.246475, 0.199525, 0.23775, 0.206384375, 0.204816875, 0.20898125, 0.22823125, 0.19479375, 0.19884, 0.21109375, 0.31246, 0.19746, 0.26169375, 0.2511, 0.220053125, 0.1993775, 0.211084375, 0.23746875, 0.2419, 0.2506875, 0.20241875, 0.200605625, 0.200821875, 0.40640625, 0.28577, 0.29355, 0.247175, 0.19562625, 0.23158125, 0.21146, 0.22918875, 0.21045, 0.24825, 0.250696875, 0.2567, 0.218871875, 0.24778125, 0.275625, 0.200634375, 0.194384375, 0.20688375, 0.225441875, 0.2221425, 0.2074125, 0.218673125, 0.244285625, 0.202284375, 0.204576875, 0.25575625, 0.2481, 0.2331, 0.211801875, 0.1942, 0.263925, 0.22153, 0.27571875, 0.285365, 0.197243125, 0.192280625, 0.23737375, 0.2624, 0.20867875, 0.21809625, 0.24625, 0.212425, 0.212685625, 0.2136625, 0.2235375, 0.1968375, 0.232921875, 0.2232, 0.2553, 0.25160625, 0.21811875, 0.236775, 0.197044375, 0.33198875, 0.252515625, 0.29328125, 0.18966875, 0.19588125, 0.20892125, 0.244875, 0.197915625, 0.19870625, 0.193815625, 0.2199125, 0.251, 0.21224, 0.2289, 0.200793125, 0.20815, 0.24140625, 0.193090625, 0.25186, 0.206875, 0.209615625, 0.19479375, 0.209175, 0.2085475, 0.21550625, 0.19295625, 0.21384375, 0.21888125, 0.2161625, 0.205725, 0.213459375, 0.233443125, 0.20180625, 0.20320625, 0.27070375, 0.22078125, 0.19765, 0.22048125, 0.2572875, 0.26950375, 0.203705625, 0.203755, 0.23951875, 0.23266, 0.204416875, 0.209846875, 0.210143125, 0.222925, 0.19461, 0.20718125, 0.218270625, 0.2999875, 0.2065, 0.20808875, 0.23246875, 0.199628125, 0.2856, 0.24159375, 0.23833875, 0.224299375, 0.239025, 0.1961775, 0.24110625, 0.21823125, 0.20099375, 0.1995125, 0.203571875, 0.19716875, 0.22734375, 0.22730875, 0.18839875, 0.20803, 0.326623125, 0.2006125, 0.28438125, 0.238421875, 0.26049125, 0.212259375, 0.225125, 0.19498125, 0.2225, 0.19851625, 0.22911625, 0.19110625, 0.2102125, 0.3043, 0.20205625, 0.21775625, 0.2271, 0.255265625, 0.311896875, 0.26625, 0.324, 0.24137875, 0.19646875, 0.2021, 0.27805625, 0.24373125, 0.22829, 0.2154075, 0.19811875, 0.37995, 0.20796875, 0.217240625, 0.21764375, 0.209465625, 0.224109375, 0.22381875, 0.20651, 0.21359375, 0.3176875, 0.22543625, 0.307965625, 0.2331825, 0.22448875, 0.19535625, 0.203190625, 0.21197, 0.202371875, 0.20628125, 0.2115625, 0.189620625, 0.19843375, 0.197380625, 0.2031, 0.2066, 0.2188525, 0.209054375, 0.20534375, 0.209340625, 0.21765, 0.20541875, 0.229155, 0.2635125, 0.206028125, 0.19458125, 0.23385625, 0.221934375, 0.223671875, 0.18908125, 0.20514375, 0.237885, 0.217096875, 0.194821875, 0.2064425, 0.23914875, 0.2150125, 0.197690625, 0.19875, 0.26756875, 0.217390625, 0.21819, 0.23598125, 0.29087, 0.22224875, 0.23854375, 0.218203125, 0.21304375, 0.2233, 0.21078125, 0.22499625, 0.20438125, 0.2244375, 0.29548125, 0.222715625, 0.3672, 0.21058125, 0.24648125, 0.2312, 0.2523125, 0.22849875, 0.195878125, 0.191891875, 0.2310925, 0.210509375, 0.246796875, 0.21099375, 0.22534375, 0.22028125, 0.19879875, 0.195209375, 0.216795625, 0.204763125, 0.28115, 0.23725, 0.24474, 0.228125, 0.2381, 0.24328125, 0.263671875, 0.19010625, 0.24511875, 0.20675625, 0.2186, 0.20515625, 0.24046875, 0.35221875, 0.20244375, 0.1928, 0.21872625, 0.2171375, 0.22642625, 0.21189625, 0.221825625, 0.1959125, 0.2206125, 0.21456875, 0.226465, 0.20415375, 0.20481875, 0.19984, 0.22595, 0.22263125, 0.221270625, 0.20440625, 0.25245, 0.2049375, 0.21714875, 0.23754375, 0.22753125, 0.19592125, 0.207859375, 0.26439375, 0.2091175, 0.191731875, 0.27585, 0.225509375, 0.20890125, 0.284420625, 0.28484, 0.2072625, 0.24296625, 0.21971125, 0.21028125, 0.194658125, 0.209278125, 0.22421875, 0.19934125, 0.22751875, 0.266475, 0.30906, 0.2065, 0.2860875, 0.240508125, 0.24851875, 0.217171875, 0.23968125, 0.232740625, 0.190559375, 0.27890625, 0.21156875, 0.25725, 0.2197925, 0.247246875, 0.2191875, 0.21994375, 0.210646875, 0.23375, 0.19554125, 0.19040625, 0.200128125, 0.20175625, 0.24664375, 0.20095625, 0.24609375, 0.19325625, 0.213559375, 0.21614, 0.217971875, 0.2175875, 0.25109375, 0.197101875, 0.191803125, 0.19007375, 0.2187875, 0.212578125, 0.228703125, 0.202609375, 0.19235125, 0.2231125, 0.23484375, 0.24609375, 0.19246875, 0.19736875, 0.21448125, 0.1985, 0.20898125, 0.22722, 0.22148, 0.2093475, 0.2124, 0.24605625, 0.248445, 0.36943125, 0.208765625, 0.30246625, 0.21804375, 0.2865, 0.218359375, 0.196696875, 0.26775, 0.22515625, 0.21126625, 0.23353875, 0.243778125, 0.211296875, 0.22723875, 0.24725, 0.250959375, 0.261, 0.22012375, 0.21390625, 0.272576875, 0.280921875, 0.25275, 0.21975375, 0.25315625, 0.19952875, 0.2158025, 0.205415625, 0.3179, 0.23221875, 0.24, 0.2010275, 0.260514375, 0.24225, 0.196471875, 0.21008125, 0.23020625, 0.2234625, 0.22236875, 0.1964125, 0.2472, 0.25762875, 0.2201625, 0.24346875, 0.26977375, 0.20955375, 0.21252, 0.20059375, 0.1969, 0.1917, 0.26067375, 0.224026875, 0.3408, 0.226171875, 0.24665625, 0.30419375, 0.221015625, 0.196450625, 0.223984375, 0.210241875, 0.2378, 0.27409, 0.203215625, 0.22737, 0.203584375, 0.203925, 0.21814625, 0.222846875, 0.21496875, 0.2186575, 0.194080625, 0.20307375, 0.234395, 0.27911875, 0.22035, 0.196440625, 0.1989, 0.30700625, 0.19361875, 0.22280625, 0.22782, 0.205909375, 0.1953, 0.25636, 0.202665625, 0.2225, 0.2200875, 0.191671875, 0.20325, 0.24342, 0.25599375, 0.200534375, 0.25075, 0.19665, 0.22821875, 0.261110625, 0.22021875, 0.27210625, 0.19603125, 0.22502875, 0.266803125, 0.28251875, 0.19713875, 0.2087, 0.20025625, 0.297765625, 0.22334, 0.198346875, 0.2170125, 0.19161875, 0.27911875, 0.22729375, 0.194408125, 0.1981, 0.21245625, 0.27253125, 0.21628125, 0.26015625, 0.3247, 0.21614625, 0.27089375, 0.223484375, 0.2202125, 0.241240625, 0.3058125, 0.2553125, 0.2138, 0.20518125, 0.203675, 0.20411875, 0.2009125, 0.2307, 0.2534125, 0.19178125, 0.19655, 0.1947, 0.2475, 0.2063, 0.19176125, 0.25801, 0.20930625, 0.22829375, 0.2399, 0.25629625, 0.1843125, 0.2499, 0.19888125, 0.2061, 0.24485625, 0.199028125, 0.26626625, 0.208284375, 0.2479125, 0.2577, 0.24009375, 0.27421875, 0.202003125, 0.25659375, 0.20725625, 0.220103125, 0.21158125, 0.19055875, 0.277625, 0.2597, 0.240384375, 0.22216875, 0.36352, 0.21265625, 0.207078125, 0.233105, 0.24626875, 0.227040625, 0.198665625, 0.30300625, 0.20553125, 0.1943, 0.21355625, 0.22640625, 0.2115, 0.20966875, 0.32612375, 0.23878, 0.26709375, 0.20541625, 0.206690625, 0.19561875, 0.202596875, 0.21726875, 0.23249625, 0.26275875, 0.230859375, 0.25592875, 0.43602875, 0.200345, 0.198221875, 0.2598125, 0.19979625, 0.21374375, 0.266616875, 0.22836625, 0.2390875, 0.21881875, 0.25761875, 0.24745625, 0.245185, 0.2278, 0.203314375, 0.26750625, 0.20436875, 0.32331875, 0.2095, 0.19732, 0.25471875, 0.23423125, 0.29590625, 0.301633125, 0.2096125, 0.19755625, 0.19138625, 0.30385, 0.263225, 0.19575, 0.24136875, 0.27881625, 0.195813125, 0.24463125, 0.21039, 0.21375, 0.20035625, 0.2716, 0.21282, 0.25524625, 0.2641375, 0.20741875, 0.204245, 0.273, 0.20538, 0.20597625, 0.22085875, 0.252, 0.2154, 0.31955625, 0.232, 0.209365, 0.237146875, 0.33, 0.22171875, 0.20371875, 0.205395625, 0.24058125, 0.19730875, 0.19902875, 0.2849, 0.274921875, 0.27629375, 0.205109375, 0.188753125, 0.238828125, 0.19526125, 0.20428625, 0.2648, 0.207825, 0.218515625, 0.22461, 0.246525, 0.2169, 0.28875, 0.258225, 0.23454, 0.19752375, 0.20835625, 0.22745, 0.19729375, 0.22860625, 0.23110625, 0.21040625, 0.23665, 0.20306875, 0.2896875, 0.249, 0.271904375, 0.201276875, 0.211748125, 0.22390625, 0.2067725, 0.24024375, 0.23988, 0.23671375, 0.21744375, 0.225, 0.225, 0.224859375, 0.40528, 0.21775625, 0.22995, 0.199996875, 0.21094375, 0.2059, 0.21083125, 0.22946, 0.2108675, 0.2133975, 0.23998, 0.27635625, 0.207825, 0.22450625, 0.25270125, 0.21108125, 0.2224, 0.22556875, 0.237373125, 0.207215625, 0.2193125, 0.21501375, 0.22118625, 0.205643125, 0.225865625, 0.234153125, 0.22548, 0.20665625, 0.2459125, 0.20909375, 0.19234125, 0.198621875, 0.22010625, 0.2505, 0.20536625, 0.206859375, 0.2385775, 0.28222125, 0.22272, 0.356203125, 0.274, 0.19326625, 0.2144325, 0.20048125, 0.23948125, 0.19636875, 0.20585, 0.2106775, 0.215248125, 0.2685, 0.2427, 0.23194875, 0.19505, 0.21766875, 0.24135, 0.21345625, 0.23225625, 0.30678625, 0.244784375, 0.21182875, 0.21603125, 0.256540625, 0.3145, 0.26340625, 0.194836875, 0.22389, 0.22491875, 0.24885, 0.24838125, 0.20075, 0.20624, 0.210471875, 0.267, 0.2065375, 0.2976, 0.2544, 0.242109375, 0.2204875, 0.3296, 0.27352, 0.21515, 0.20734375, 0.20713625, 0.23613, 0.210675, 0.28041875, 0.25808125, 0.2627, 0.2143975, 0.2169875, 0.2237875, 0.20706, 0.20237125, 0.202684375, 0.222306875, 0.25478, 0.20622375, 0.21456, 0.22435625, 0.22161875, 0.273325, 0.2428125, 0.21084625, 0.22380625, 0.23185, 0.2671125, 0.249103125, 0.238875, 0.34565625, 0.239765625, 0.2349, 0.2117475, 0.24384375, 0.24760625, 0.306375, 0.21026, 0.3057, 0.259734375, 0.28512, 0.2768, 0.2054, 0.22788125, 0.2124125, 0.2716375, 0.2096, 0.21336625, 0.2515575, 0.22231125, 0.2411, 0.21751125, 0.39365625, 0.197659375, 0.20130625, 0.28846875, 0.21903125, 0.21096875, 0.2542, 0.21565625, 0.20746875, 0.193015625, 0.21019375, 0.20971875, 0.24660875, 0.22182, 0.2302125, 0.22768125, 0.22930875, 0.202828125, 0.2090625, 0.32625, 0.21811875, 0.2693, 0.29696, 0.2765, 0.21200625, 0.241118125, 0.21241125, 0.238565, 0.244, 0.22455, 0.232265, 0.257496875, 0.3701, 0.245873125, 0.190990625, 0.19740625, 0.284496875, 0.26602, 0.191290625, 0.20035625, 0.20765125, 0.2085, 0.22336875, 0.207171875, 0.200340625, 0.21454375, 0.22165625, 0.199721875, 0.2226125, 0.212, 0.233296875, 0.218375625, 0.24658125, 0.25616875, 0.190739375, 0.2264, 0.2220875, 0.220096875, 0.215, 0.24165, 0.2614, 0.3039, 0.208296875, 0.204784375, 0.206231875, 0.21805625, 0.252375, 0.204725, 0.19365625, 0.242915625, 0.2562875, 0.20480625, 0.19947375, 0.21746, 0.346640625, 0.200996875, 0.28025, 0.19743, 0.22588125, 0.20791875, 0.209703125, 0.27045, 0.216015625, 0.2559875, 0.25066, 0.20755, 0.2274625, 0.22982125, 0.27832, 0.203290625, 0.27380625, 0.200555, 0.210803125, 0.19599875, 0.21313875, 0.251953125, 0.29930625, 0.2173125, 0.233948125, 0.223359375, 0.22737625, 0.26522, 0.2464875, 0.19399375, 0.27558, 0.21984375, 0.31834125, 0.268, 0.25499375, 0.227, 0.2480625, 0.210425, 0.2243775, 0.2536875, 0.24046875, 0.21271875, 0.19678, 0.2038, 0.225025, 0.2313875, 0.280383125, 0.2179, 0.228234375, 0.253045, 0.21893125, 0.228128125, 0.2708, 0.207273125, 0.21985, 0.196796875, 0.234256875, 0.22615, 0.2199125, 0.212355, 0.2467375, 0.241084375, 0.31248125, 0.20597375, 0.365765625, 0.2174325, 0.22578375, 0.194396875, 0.208696875, 0.191528125, 0.220465625, 0.20191875, 0.38144, 0.22101875, 0.30440625, 0.20734625, 0.20165, 0.20738125, 0.233784375, 0.2325375, 0.2631175, 0.19861875, 0.234015, 0.4096, 0.24259, 0.23837625, 0.23145, 0.2083375, 0.204400625, 0.19149375, 0.224955, 0.221015625, 0.23041875, 0.21643125, 0.391884375, 0.201078125, 0.26656, 0.1977575, 0.24193125, 0.20196875, 0.374, 0.22640625, 0.21056, 0.25665, 0.283846875, 0.318256875, 0.24184625, 0.21204375, 0.20289375, 0.22985625, 0.21696, 0.24681375, 0.2333, 0.21085, 0.246, 0.25541, 0.2039025, 0.23198, 0.209390625, 0.2403, 0.19126875, 0.236296875, 0.20835625, 0.20402625, 0.23790625, 0.19268125, 0.26105625, 0.20015625, 0.189983125, 0.251440625, 0.281503125, 0.287725, 0.20993625, 0.2353125, 0.200851875, 0.236184375, 0.20775, 0.257753125, 0.218421875, 0.206871875, 0.204403125, 0.198528125, 0.2339625, 0.229513125, 0.20216, 0.2222, 0.23053125, 0.21260875, 0.2045125, 0.19971875, 0.31365, 0.4199, 0.191371875, 0.196759375, 0.20765625, 0.197221875, 0.2035625, 0.20339375, 0.21465, 0.26671625, 0.19656875, 0.28265625, 0.24761125, 0.20826875, 0.27084, 0.22005625, 0.23190625, 0.22285, 0.2281, 0.19806, 0.2363825, 0.23899, 0.195213125, 0.2514375, 0.202175, 0.20410125, 0.22013125, 0.20419375, 0.261635, 0.4913, 0.19825625, 0.296, 0.25298125, 0.261938125, 0.314296875, 0.270625, 0.250484375, 0.22035625, 0.218246875, 0.268566875, 0.29250625, 0.21405625, 0.2992, 0.27421875, 0.231445, 0.25262, 0.242165, 0.2250125, 0.30064, 0.24070625, 0.23555, 0.24154125, 0.28016, 0.19398125, 0.21221875, 0.20237, 0.36, 0.21985625, 0.22921, 0.2713125, 0.211471875, 0.2372, 0.21069375, 0.203648125, 0.20871625, 0.243525, 0.197495625, 0.19886125, 0.25925, 0.291496875, 0.199825, 0.223828125, 0.216275, 0.2199925, 0.23676, 0.20648125, 0.253075, 0.2097975, 0.26085125, 0.28864, 0.3264, 0.19456, 0.21148, 0.23171875, 0.3215125, 0.2345, 0.204678125, 0.2329875, 0.26008, 0.216946875, 0.23904375, 0.2816, 0.22621625, 0.24155, 0.21638125, 0.253078125, 0.31943, 0.21185, 0.2328275, 0.21974125, 0.2136625, 0.2678125, 0.22363125, 0.20116875, 0.21774, 0.196853125, 0.350625, 0.219760625, 0.20501875, 0.21725, 0.19279875, 0.2004, 0.24885625, 0.21530625, 0.208609375, 0.203746875, 0.21335, 0.2361, 0.243159375, 0.26987625, 0.242445, 0.30525, 0.24301875, 0.2478875, 0.19435, 0.1972, 0.3084, 0.19163625, 0.2428125, 0.253825, 0.21686875, 0.25915375, 0.22321875, 0.3477, 0.201696875, 0.212, 0.285721875, 0.28775625, 0.22321625, 0.23148125, 0.22263125, 0.23447625, 0.223575, 0.26254375, 0.18935625, 0.20364375, 0.2638875, 0.30624, 0.2212, 0.26851375, 0.21421875, 0.21671, 0.27075, 0.2365775, 0.22803625, 0.18996125, 0.2611125, 0.275538125, 0.20489875, 0.26876, 0.2316, 0.2397075, 0.3375, 0.24599, 0.20100375, 0.238671875, 0.22164, 0.2265125, 0.238696875, 0.23565625, 0.2835, 0.23765625, 0.23700625, 0.211703125, 0.20148125, 0.22864375, 0.201975, 0.19235125, 0.2597, 0.2387025, 0.206890625, 0.24614875, 0.30270625, 0.2407525, 0.368475, 0.2736, 0.204, 0.24142125, 0.28125, 0.231215, 0.23796, 0.2138125, 0.277575, 0.2221475, 0.190469375, 0.2418, 0.211508125, 0.23568125, 0.18726875, 0.2141625, 0.239915625, 0.21348125, 0.20937375, 0.2304, 0.20921875, 0.26859375, 0.22402625, 0.20472375, 0.2026075, 0.233521875, 0.21203125, 0.2848, 0.393175, 0.27376, 0.24800625, 0.195103125, 0.3258475, 0.21852375, 0.220135, 0.23896, 0.22721875, 0.19939375, 0.2195125, 0.31609375, 0.229921875, 0.23083125, 0.20125875, 0.233465, 0.217296875, 0.25970625, 0.32, 0.2295, 0.3346875, 0.19219375, 0.19115625, 0.20914375, 0.25968125, 0.2775, 0.22981875, 0.2042375, 0.347025, 0.22496875, 0.2432, 0.21189375, 0.21613125, 0.2314, 0.336, 0.276335, 0.270946875, 0.25270875, 0.19178625, 0.2002125, 0.23939625, 0.291805, 0.2293, 0.21203125, 0.28412, 0.2154, 0.27083125, 0.26895, 0.2559125, 0.23025, 0.253, 0.19955, 0.22659, 0.2178, 0.232125, 0.2258275, 0.19991875, 0.24176, 0.2137375, 0.26040625, 0.1993875, 0.2113, 0.26109375, 0.2596, 0.233455625, 0.20838625, 0.2047, 0.22416125, 0.28, 0.294, 0.26296875, 0.26224625, 0.2186, 0.23915625, 0.280930625, 0.235225, 0.23445, 0.3281, 0.3076, 0.20033125, 0.2145125, 0.26896, 0.2132075, 0.322753125, 0.209385, 0.24278125, 0.2082125, 0.2014375, 0.2666875, 0.22019875, 0.22578, 0.21030625, 0.2413, 0.26760125, 0.278270625, 0.19955625, 0.22506, 0.209015625, 0.2347, 0.20274375, 0.21713125, 0.2591, 0.202325, 0.346078125, 0.238843125, 0.22681875, 0.25449, 0.22835, 0.2115, 0.318459375, 0.2081875, 0.2295125, 0.2122, 0.23229375, 0.1949125, 0.220825, 0.2603975, 0.199996875, 0.2270625, 0.2338125, 0.21384375, 0.205443125, 0.346258125, 0.20483, 0.1971875, 0.191015625, 0.25904375, 0.19718125, 0.25376, 0.203445625, 0.44614375, 0.25953875, 0.277303125, 0.21668125, 0.22521375, 0.198, 0.2079875, 0.2547025, 0.24208, 0.20936875, 0.25178, 0.2597625, 0.306, 0.21823125, 0.2112, 0.21141875, 0.259675, 0.18926875, 0.33245625, 0.457523125, 0.21147875, 0.1974125, 0.23578125, 0.1969, 0.270375, 0.20326875, 0.2089375, 0.21296625, 0.23665, 0.19991875, 0.222578125, 0.2330625, 0.2691, 0.25828125, 0.21455, 0.21435625, 0.2387, 0.20689375, 0.24975, 0.205253125, 0.24863625, 0.209005, 0.198984375, 0.240990625, 0.25427, 0.24592125, 0.19957875, 0.21096875, 0.2451, 0.2678, 0.207153125, 0.2240875, 0.22084, 0.288196875, 0.2177, 0.215578125, 0.29953125, 0.205535, 0.19945, 0.2311, 0.24524375, 0.29334375, 0.20565625, 0.20106, 0.24640625, 0.29781875, 0.24384375, 0.252333125, 0.20316875, 0.2292375, 0.22838625, 0.211578125, 0.213421875, 0.27104375, 0.197871875, 0.21301875, 0.2421225, 0.26278125, 0.235135, 0.21625, 0.30601, 0.2267925, 0.19903125, 0.22115, 0.29362, 0.3088, 0.26622, 0.2635875, 0.2024375, 0.23934375, 0.21755, 0.338671875, 0.241878125, 0.2599, 0.27890625, 0.1944925, 0.224371875, 0.3728, 0.2486, 0.210093125, 0.188296875, 0.2498375, 0.25270625, 0.210328125, 0.210148125, 0.21346875, 0.2321, 0.41817875, 0.25094875, 0.261065625, 0.20551875, 0.240135625, 0.21156, 0.1948875, 0.20655625, 0.199959375, 0.21858125, 0.285046875, 0.20804375, 0.20710625, 0.260925, 0.3576, 0.23306, 0.191478125, 0.3328, 0.305859375, 0.2263475, 0.247075, 0.22614375, 0.217625, 0.243355, 0.2296875, 0.214425, 0.21553125, 0.23530125, 0.222301875, 0.2204875, 0.22098125, 0.4352, 0.20550625, 0.275203125, 0.25888125, 0.23509, 0.23209375, 0.2144025, 0.2381875, 0.25193, 0.201333125, 0.21735, 0.23333625, 0.2646, 0.22058125, 0.19411875, 0.27274375, 0.2226275, 0.3056, 0.20508125, 0.21377625, 0.23032875, 0.21540625, 0.28696, 0.2120375, 0.3484, 0.2156, 0.231575, 0.18763125, 0.2285, 0.30515625, 0.19946375, 0.3757, 0.2334125, 0.2575, 0.22681, 0.20438125, 0.2431, 0.26622, 0.270940625, 0.239525, 0.24025, 0.23345625, 0.23574375, 0.23006875, 0.21745, 0.25058, 0.255, 0.229325, 0.22000125, 0.22739375, 0.21304875, 0.22484375, 0.2597625, 0.20085, 0.23216, 0.265313125, 0.27056, 0.21554, 0.268106875, 0.2745325, 0.28829375, 0.2428125, 0.2803, 0.32293875, 0.2682, 0.193229375, 0.21108, 0.284409375, 0.2462625, 0.2154125, 0.21736875, 0.2030075, 0.21675, 0.2253275, 0.22146, 0.24426, 0.28820625, 0.3793125, 0.26965, 0.20807, 0.24633, 0.259681875, 0.18926, 0.25276, 0.32264375, 0.24215625, 0.204281875, 0.2696, 0.237915625, 0.22009375, 0.221415, 0.3129, 0.2355125, 0.198196875, 0.2103875, 0.34734375, 0.20844875, 0.37064, 0.304895, 0.20933125, 0.251825, 0.220390625, 0.20478125, 0.22301875, 0.2145, 0.2101375, 0.20552875, 0.27526875, 0.25621875, 0.2475, 0.19962625, 0.2093, 0.241096875, 0.24416875, 0.270703125, 0.28471875, 0.220000625, 0.2198, 0.235859375, 0.2086125, 0.2209475, 0.247059375, 0.2925, 0.3952, 0.234134375, 0.20156875, 0.22746875, 0.293285625, 0.22125, 0.3026, 0.208809375, 0.2395325, 0.20145, 0.28370625, 0.30444, 0.212775, 0.217115625, 0.229285625, 0.23559375, 0.262734375, 0.22625, 0.25427875, 0.233331875, 0.216945, 0.263175, 0.19699625, 0.23008125, 0.28870625, 0.3536, 0.1961125, 0.240985625, 0.255935, 0.321513125, 0.228296875, 0.25599875, 0.32896, 0.2856, 0.195076875, 0.24667, 0.239, 0.21371, 0.2126225, 0.24426875, 0.212725, 0.216575, 0.2421575, 0.227988125, 0.284134375, 0.250995, 0.2047, 0.27135625, 0.20266875, 0.2414, 0.27004, 0.34816, 0.1953125, 0.22830625, 0.3315, 0.238840625, 0.254475, 0.263671875, 0.22078125, 0.3241875, 0.1967875, 0.2885, 0.39163125, 0.255371875, 0.23453125, 0.265, 0.18403125, 0.245, 0.26496, 0.2345125, 0.20540625, 0.232053125, 0.2559475, 0.313225, 0.2953125, 0.195584375, 0.26953125, 0.21828625, 0.238025, 0.240234375, 0.417605, 0.22995, 0.22390625, 0.2303125, 0.2296875, 0.2281125, 0.2329325, 0.29204, 0.25390625, 0.222635625, 0.444135625, 0.231175, 0.212634375, 0.23972, 0.2058125, 0.32163125, 0.20414375, 0.25625, 0.2678, 0.2494125, 0.23459375, 0.41801, 0.383828125, 0.259825, 0.19441625, 0.31640625, 0.32122625, 0.250925, 0.28413125, 0.34952, 0.272265625, 0.24451, 0.237975, 0.24445, 0.52200625, 0.23828125, 0.328, 0.44378125] 
fid5 = [0.8825008598, 0.8779470715, 0.8705980144, 0.8667483842, 0.8421829943, 0.8563644123, 0.862406015, 0.8082440349, 0.7878498923, 0.8881022789, 0.8659302518, 0.8476273262, 0.8727471042, 0.8775492059, 0.6745454545, 0.8649458575, 0.7755534356, 0.8730613519, 0.8392095264, 0.7689343816, 0.8645958143, 0.7985533454, 0.8475656581, 0.8504699582, 0.8589539712, 0.7964401294, 0.8774468382, 0.8591549296, 0.7670441503, 0.7745471051, 0.8111726685, 0.8567701863, 0.8768964036, 0.900517574, 0.8654247486, 0.8602570969, 0.8884747725, 0.7830821151, 0.8691562223, 0.8088538658, 0.8660805698, 0.8640751595, 0.8705933633, 0.6745454545, 0.902359784, 0.7699206807, 0.737037037, 0.8087436985, 0.8302901607, 0.6876350776, 0.8389870013, 0.8779587248, 0.7964401294, 0.8128637556, 0.7777054041, 0.8353216873, 0.8324798623, 0.8722111554, 0.8758710137, 0.890388284, 0.8623876631, 0.8647709369, 0.7930444698, 0.7910001938, 0.7799121844, 0.8871175838, 0.870500008, 0.7467357701, 0.7906252666, 0.8594080338, 0.7747590532, 0.8858226796, 0.859923648, 0.8161329305, 0.8496593392, 0.8012481381, 0.8485929437, 0.7899974889, 0.6745454545, 0.852393964, 0.7882882883, 0.8653544627, 0.765105097, 0.8837484212, 0.7713375062, 0.8881219067, 0.87426547, 0.8066441756, 0.8625498008, 0.8752433697, 0.8295143213, 0.7, 0.7634704768, 0.8880922351, 0.8605187773, 0.7919138308, 0.846430882, 0.7285714286, 0.865677547, 0.8982260204, 0.7799070848, 0.8817716136, 0.8697230438, 0.6789268293, 0.7581333333, 0.8908950647, 0.8874948715, 0.788, 0.8657518977, 0.8249613601, 0.7, 0.7350973588, 0.7454773287, 0.8705767819, 0.7882882883, 0.8884095791, 0.8538030798, 0.9199775533, 0.8229435666, 0.8911744983, 0.8131147541, 0.857371453, 0.8401992152, 0.7784433422, 0.8959994107, 0.864023984, 0.7650967038, 0.8513535549, 0.8733481884, 0.7839779006, 0.8466092906, 0.7911348573, 0.8611274981, 0.7063448276, 0.8871302005, 0.6833492164, 0.7639284889, 0.789195901, 0.7908657665, 0.7755244911, 0.8725837255, 0.8078192442, 0.8359351674, 0.8677313677, 0.7279616639, 0.7706259066, 0.8636424752, 0.8628753013, 0.8844877321, 0.7715911769, 0.8607502328, 0.8330512061, 0.8032870448, 0.7, 0.8076385011, 0.8689876469, 0.8538791256, 0.7732889429, 0.8709275731, 0.8811835042, 0.866539924, 0.7730426082, 0.8745066881, 0.8170517962, 0.885016203, 0.7901054775, 0.8598042242, 0.762565445, 0.8649210308, 0.8575965958, 0.7135121274, 0.8069420539, 0.8671821756, 0.8937890608, 0.8612254314, 0.7459962639, 0.7615384615, 0.8793517484, 0.8694269132, 0.8930123868, 0.7874859076, 0.8464147703, 0.8258530299, 0.88814125, 0.7829396042, 0.7646859722, 0.8883892999, 0.8107965817, 0.8697764218, 0.7861233651, 0.8610460165, 0.8383390576, 0.8820115149, 0.8630837403, 0.8742939398, 0.8959346001, 0.8867657993, 0.8925697772, 0.7682805916, 0.8937284518, 0.7898830518, 0.7, 0.8699106256, 0.7957295252, 0.7844359639, 0.7246902319, 0.8370395254, 0.7889393982, 0.8697769439, 0.8497570962, 0.8552643411, 0.9010408369, 0.8658867112, 0.8121387283, 0.7996336996, 0.8532069039, 0.8017707363, 0.8139072848, 0.8861893832, 0.8271641791, 0.8436678452, 0.6667931251, 0.8084793685, 0.7870968643, 0.8799547107, 0.7724222238, 0.8837389953, 0.8523952327, 0.7829726453, 0.824375888, 0.8839244356, 0.6638891668, 0.880935324, 0.8587079408, 0.802792419, 0.7648212075, 0.8583872451, 0.876121612, 0.7687538748, 0.8182083285, 0.8508994646, 0.7773937475, 0.8738970577, 0.7898830518, 0.7590713476, 0.8114509511, 0.7429959399, 0.7746624451, 0.7, 0.8065093596, 0.8577328568, 0.7774812992, 0.8878941965, 0.8572010255, 0.877020831, 0.7915187377, 0.8532432432, 0.920242915, 0.7185999597, 0.8859442889, 0.8406302227, 0.8732750819, 0.7193626397, 0.7635428071, 0.746786962, 0.8604187656, 0.737037037, 0.8698183718, 0.8618115814, 0.8239193084, 0.8520837879, 0.769467139, 0.899280171, 0.8839430888, 0.7849668098, 0.8841595746, 0.8669662013, 0.7, 0.8718113438, 0.8096323513, 0.7, 0.8243988535, 0.713487882, 0.8009669417, 0.7259720394, 0.7112676056, 0.752992728, 0.7524551413, 0.7871473354, 0.8951318059, 0.8531439218, 0.8567219654, 0.853050107, 0.8207511017, 0.8787972327, 0.7783919145, 0.8512266213, 0.8341704247, 0.8114033316, 0.8457964124, 0.7701415107, 0.8297297297, 0.8632194326, 0.8652340791, 0.7769606684, 0.8354278761, 0.8247049078, 0.8804995971, 0.7679678343, 0.7964490459, 0.8685705737, 0.8806671486, 0.8892065552, 0.7774896237, 0.7667751294, 0.8067368594, 0.8460839463, 0.8705762215, 0.8957808129, 0.8694244724, 0.7939885939, 0.8256807708, 0.8496823819, 0.8962482875, 0.7839621077, 0.8302509985, 0.9003389112, 0.8640556237, 0.8790768128, 0.8092635442, 0.8758539124, 0.7737362249, 0.7800945999, 0.8536511504, 0.7424133507, 0.8301093633, 0.7577292412, 0.824424389, 0.833897256, 0.8533895658, 0.7112676056, 0.8461062285, 0.8728182687, 0.7690894881, 0.8596773483, 0.7716699411, 0.8211510791, 0.7505609393, 0.8106470592, 0.8076179624, 0.9188584428, 0.8827323711, 0.8497288244, 0.8483196116, 0.8807816862, 0.8347511697, 0.7734548445, 0.6938323458, 0.869935255, 0.8623939534, 0.8837969, 0.8688886139, 0.8351021724, 0.8684790987, 0.8773469846, 0.8843959677, 0.8634933173, 0.8433626437, 0.7654720846, 0.8239198193, 0.7972790091, 0.8793896227, 0.8675655431, 0.809855248, 0.8902072421, 0.8724530061, 0.7, 0.7615000959, 0.8709631049, 0.8715310553, 0.7823827701, 0.8976745032, 0.7993197279, 0.7711784193, 0.8706359976, 0.8162966066, 0.7698913522, 0.8545674204, 0.8838453976, 0.8674992822, 0.7682487365, 0.8015706208, 0.8652242583, 0.8538018789, 0.7352941176, 0.8921834289, 0.8598507531, 0.8802046218, 0.8285604186, 0.869174185, 0.8872812717, 0.8773493976, 0.785106383, 0.88115356, 0.8556163447, 0.8732154651, 0.8853344412, 0.869200376, 0.8627958173, 0.7216978278, 0.8076923077, 0.8722979196, 0.8778678588, 0.8980805699, 0.7682731011, 0.8260790802, 0.8713844894, 0.8231654122, 0.7587388425, 0.8784070217, 0.8698839777, 0.8735360683, 0.8689049751, 0.8767116698, 0.8969020669, 0.8676914907, 0.8337453646, 0.8097009647, 0.866638568, 0.8019097788, 0.8722434314, 0.8790521133, 0.8848672244, 0.7810043761, 0.8610984472, 0.8512338973, 0.7929858345, 0.811747851, 0.7985007844, 0.8711991828, 0.7544502618, 0.8373462563, 0.8589887915, 0.8828440299, 0.776907001, 0.7, 0.8379601435, 0.8766167989, 0.7510143121, 0.8492489999, 0.7958777335, 0.8285470903, 0.861971831, 0.8710874908, 0.876076555, 0.7541026998, 0.8439479072, 0.7, 0.7458654906, 0.8135897222, 0.8577903188, 0.7408223201, 0.8209439528, 0.8934500245, 0.8825920612, 0.8902248162, 0.7874147862, 0.8390351103, 0.892655238, 0.8777311396, 0.7799783395, 0.8919991437, 0.875480271, 0.7713145413, 0.8675823802, 0.8521930298, 0.8311213272, 0.8835715765, 0.6745454545, 0.8636686496, 0.7880503145, 0.896335685, 0.8852260334, 0.8640305494, 0.8763145417, 0.8126750639, 0.8179871712, 0.8544985813, 0.8783149391, 0.8334166146, 0.8203960565, 0.7596378639, 0.8352194253, 0.8701871262, 0.8727133269, 0.8286185874, 0.9182854429, 0.8688591149, 0.8297712143, 0.8076923077, 0.7467357701, 0.7682098765, 0.870260603, 0.8719952557, 0.7763918504, 0.8873991148, 0.8831923992, 0.8488750107, 0.8149020654, 0.7722905682, 0.769418553, 0.8754156113, 0.8436426117, 0.776129245, 0.8876164384, 0.8295143213, 0.7183584401, 0.8960202476, 0.8849714077, 0.831720155, 0.7764849246, 0.8536963489, 0.7826498423, 0.8594593342, 0.7063448276, 0.869739217, 0.8135897222, 0.8636686496, 0.6973801727, 0.7852741433, 0.8190615836, 0.895129406, 0.875370878, 0.7635734079, 0.7898876404, 0.8771104052, 0.8490661305, 0.8798529439, 0.805852064, 0.8299381195, 0.8840655187, 0.8178476378, 0.7641030212, 0.7577630551, 0.7997585949, 0.8905895117, 0.7753527914, 0.8255165138, 0.7765532982, 0.8414872798, 0.8192668272, 0.7910293145, 0.7715607412, 0.8612567031, 0.8754120638, 0.8109967342, 0.7800942285, 0.7534611289, 0.8000803787, 0.848470269, 0.8711784813, 0.8441520065, 0.8925031891, 0.7947151246, 0.8635361366, 0.8223172255, 0.778142601, 0.6519313305, 0.8817480844, 0.8305624908, 0.8795633098, 0.8523127763, 0.7615384615, 0.8642737896, 0.8396083386, 0.8628590108, 0.772074958, 0.8008356546, 0.7524778299, 0.8751798277, 0.8705870941, 0.7757703826, 0.8936652615, 0.7163380282, 0.8788185956, 0.8596596926, 0.8893031142, 0.7637227844, 0.7712458328, 0.8872588431, 0.8981218607, 0.8118309942, 0.8056555494, 0.7649381423, 0.7835214447, 0.7651189785, 0.8198655167, 0.8681496028, 0.8866625246, 0.7214996696, 0.7754969815, 0.8048766573, 0.8705409322, 0.7690514987, 0.7707738501, 0.8757669202, 0.8555573197, 0.7376470588, 0.8714386212, 0.8439726696, 0.750640322, 0.8643294758, 0.7785971223, 0.8710027529, 0.7, 0.8575731122, 0.8829853055, 0.7770845674, 0.8181904401, 0.8397260274, 0.8804061575, 0.8366277837, 0.8017060487, 0.8096772346, 0.8788206157, 0.7, 0.8871494282, 0.81369253, 0.8854161592, 0.7712020857, 0.8591507159, 0.820855615, 0.8173640578, 0.8722434314, 0.8024729079, 0.777795552, 0.8564767966, 0.8875714083, 0.868498354, 0.8726959692, 0.850134437, 0.8057225196, 0.7717894794, 0.8137147565, 0.8857386281, 0.7895846426, 0.8199617956, 0.7680856685, 0.8620613454, 0.8668971671, 0.8777136687, 0.8003000231, 0.8999273004, 0.8239524414, 0.8162966066, 0.7568807339, 0.7152806523, 0.7112676056, 0.7993197279, 0.8769548304, 0.8900205862, 0.6744292237, 0.7693797691, 0.875191541, 0.8084459559, 0.8884976029, 0.8222249478, 0.8173314298, 0.8774226432, 0.7781620358, 0.7463509133, 0.8850672498, 0.8415058969, 0.7143007361, 0.8457056947, 0.8176674124, 0.8515998441, 0.8836642167, 0.8199496614, 0.869073348, 0.8559536613, 0.8716184268, 0.8362676056, 0.8122224445, 0.7804336126, 0.7654836097, 0.8617320492, 0.7882882883, 0.8179741051, 0.7573853174, 0.7727151907, 0.8315365457, 0.7775473221, 0.778027082, 0.6344262295, 0.7711374815, 0.7840960065, 0.8629927995, 0.8453211121, 0.8516224912, 0.7738687356, 0.8671901833, 0.8809909103, 0.8593371812, 0.8802054817, 0.7799121844, 0.8831537391, 0.772665367, 0.8730530443, 0.8876829884, 0.821103427, 0.861491038, 0.8671327579, 0.8216896832, 0.7854125929, 0.877692578, 0.8069420539, 0.8094186902, 0.7839934865, 0.7, 0.7843401308, 0.7880503145, 0.7541026998, 0.8316047112, 0.7810636218, 0.8087734768, 0.8539855686, 0.7918767507, 0.7963100182, 0.8754445437, 0.7750630257, 0.8686868365, 0.8292822967, 0.9008257753, 0.7538151319, 0.803030303, 0.8705509012, 0.8821924236, 0.7839779006, 0.7772977397, 0.7829740092, 0.8544013122, 0.900742484, 0.8616920137, 0.8767565855, 0.7930444698, 0.8725470259, 0.8825544971, 0.7584755404, 0.8669868081, 0.8139544513, 0.8555615795, 0.8592106131, 0.8661892098, 0.8604161807, 0.8594080338, 0.859892918, 0.8630421866, 0.691663486, 0.8651392745, 0.8921248264, 0.81395998, 0.8027932961, 0.737037037, 0.8241114313, 0.7581333333, 0.7687666696, 0.7642467516, 0.884592763, 0.7648940783, 0.8598042242, 0.873040114, 0.8528646716, 0.8912567949, 0.8005446712, 0.8495657596, 0.8704860945, 0.8942414901, 0.881124238, 0.7859981933, 0.8342807132, 0.7947631035, 0.7226101008, 0.882293126, 0.6879194631, 0.7615384615, 0.7812967162, 0.8070371039, 0.8506200371, 0.7839779006, 0.8694911036, 0.8689251146, 0.8668924831, 0.8758556789, 0.7637738414, 0.8668639053, 0.7861952862, 0.8845715544, 0.860822463, 0.8010235919, 0.8480248952, 0.8838394289, 0.7, 0.8190615836, 0.8709622889, 0.8051849597, 0.8639566396, 0.7778378378, 0.7712049651, 0.8613437213, 0.7388159266, 0.8154334641, 0.740353461, 0.7636467964, 0.8074862375, 0.8589567333, 0.8323833506, 0.8581605326, 0.8278744091, 0.8564133887, 0.8821123538, 0.8562620838, 0.8791019304, 0.8805609269, 0.8782478673, 0.8813278409, 0.8033932136, 0.7906007669, 0.8049101157, 0.8641658997, 0.8028111887, 0.8467962988, 0.8562620838, 0.8358275587, 0.8270997104, 0.8849860234, 0.7829264114, 0.8725174416, 0.8326673177, 0.8761690978, 0.8364238411, 0.8776934531, 0.7740528408, 0.8240115718, 0.8835656497, 0.8593922044, 0.878276819, 0.8455624146, 0.8564600678, 0.8049049159, 0.737037037, 0.8430814222, 0.842812964, 0.8090585636, 0.8420689655, 0.8604871544, 0.8565786248, 0.8792703225, 0.8078263932, 0.8487878336, 0.8042637221, 0.6849750132, 0.7881153579, 0.7989214352, 0.8918764658, 0.8012658228, 0.8518610422, 0.8662947385, 0.8375567949, 0.7812506715, 0.8822099884, 0.8161640757, 0.7923446894, 0.7811986681, 0.8674210718, 0.8311524797, 0.8011374231, 0.8671725012, 0.8747181939, 0.8586693752, 0.8747544484, 0.8286195286, 0.8691551185, 0.8953466943, 0.8472222222, 0.7408223201, 0.8100600903, 0.8249843195, 0.8968071545, 0.8126993059, 0.8715269496, 0.8563018302, 0.7402515723, 0.8542380325, 0.7786910152, 0.8811412789, 0.799628493, 0.7765938865, 0.9194324554, 0.7720238645, 0.7783747178, 0.8820386948, 0.7874859076, 0.8585902399, 0.8005446712, 0.8088538658, 0.7815964523, 0.7800448122, 0.8988698138, 0.8496290128, 0.834924392, 0.8715455089, 0.8368117336, 0.8012172047, 0.8156520707, 0.8805038957, 0.7886568689, 0.8481657932, 0.9191453224, 0.8354205034, 0.7568807339, 0.8798035743, 0.7765744171, 0.8273375849, 0.8261082283, 0.7112676056, 0.8556582127, 0.8739677459, 0.8973915472, 0.7946337642, 0.7945187964, 0.8826221877, 0.8811436963, 0.8709234606, 0.8480736869, 0.8198355446, 0.7774498761, 0.8672909339, 0.775556164, 0.7778378378, 0.7763233666, 0.8699078943, 0.8157977232, 0.7577292412, 0.8539227034, 0.8419580634, 0.8944170772, 0.8583521533, 0.8403249631, 0.7923446894, 0.8396083386, 0.8918235505, 0.8936367509, 0.708250376, 0.7851264195, 0.7757083906, 0.875295356, 0.8732700071, 0.8069420539, 0.7888738012, 0.8701910503, 0.8688184052, 0.7, 0.883767766, 0.8129036, 0.7765908444, 0.7858956122, 0.8557555214, 0.868803264, 0.8331177636, 0.7, 0.8125075045, 0.8473029414, 0.7967713851, 0.7839779006, 0.7969911707, 0.862406015, 0.7839779006, 0.8944018682, 0.8070427154, 0.7541026998, 0.8063671284, 0.7094674556, 0.803905832, 0.7846792975, 0.7500313446, 0.8524924766, 0.8740972557, 0.8775831851, 0.8519457741, 0.8487179487, 0.8711475619, 0.8691311813, 0.8692875201, 0.8473096058, 0.7861952862, 0.8321407327, 0.7812318779, 0.8488463427, 0.8915287289, 0.8578039444, 0.8549802664, 0.7335814723, 0.8460839463, 0.7763149739, 0.868541962, 0.859293013, 0.7645913976, 0.8474232632, 0.8619272467, 0.8388019397, 0.7581333333, 0.8519404408, 0.8204678363, 0.7615384615, 0.7, 0.8720281042, 0.7242992037, 0.7974223925, 0.8706699907, 0.8486474882, 0.8778279569, 0.8769487009, 0.7842153467, 0.8864907384, 0.7615384615, 0.788, 0.83009737, 0.7824413009, 0.8589432601, 0.7805597916, 0.8116194808, 0.878156511, 0.8940617686, 0.8714451538, 0.8977854671, 0.7751272641, 0.7278150928, 0.8924630848, 0.8702041746, 0.8802832616, 0.8637066229, 0.8654850216, 0.8547252747, 0.755706156, 0.8095014111, 0.7690948582, 0.8759613495, 0.8765141038, 0.898258092, 0.8176312647, 0.7456858731, 0.6839339771, 0.7572018779, 0.7250660095, 0.8121886416, 0.8458675546, 0.8797163329, 0.895557547, 0.8820036536, 0.8201591168, 0.8008356546, 0.8143939394, 0.8793987124, 0.900443716, 0.7730223484, 0.8114645461, 0.8754207037, 0.7996085655, 0.813048252, 0.8613016983, 0.8297297297, 0.8612767014, 0.8674282951, 0.7699103713, 0.8797631655, 0.7830930761, 0.7725979037, 0.7635398061, 0.869979716, 0.7690676729, 0.8121257704, 0.8401386435, 0.7912753095, 0.8795633054, 0.7518629816, 0.7595025592, 0.8734995059, 0.8467172944, 0.7, 0.8797654817, 0.8705463183, 0.7620840951, 0.6912221561, 0.8261281714, 0.7, 0.8860477062, 0.7694576753, 0.8668971671, 0.8870390342, 0.8891036678, 0.7693162512, 0.7624735526, 0.8234989956, 0.8270479586, 0.8244962884, 0.883225616, 0.7696086335, 0.8708887724, 0.8853522768, 0.7888030888, 0.7638471612, 0.7801302383, 0.8804568455, 0.8485994853, 0.8390371143, 0.8907266323, 0.7799121844, 0.8856692343, 0.8725483548, 0.8373558871, 0.8800806702, 0.8528545005, 0.7916891757, 0.737037037, 0.879946963, 0.8505370415, 0.8353528834, 0.9188733064, 0.8773373821, 0.8771236161, 0.8744063745, 0.8514491175, 0.7725, 0.8871826937, 0.8571958295, 0.8318787698, 0.8660805698, 0.7374145891, 0.737037037, 0.8678964791, 0.8576948554, 0.8888463877, 0.9182706333, 0.8922330097, 0.8660130719, 0.9183002545, 0.8122224445, 0.8878326252, 0.8354205034, 0.7562182154, 0.8459797785, 0.8365200765, 0.8088396723, 0.8593612573, 0.8001080821, 0.869005444, 0.8781526357, 0.8566206949, 0.7571900578, 0.86807631, 0.7920181232, 0.8860750232, 0.8551129691, 0.846430882, 0.8693064877, 0.871732817, 0.8643592099, 0.8497110751, 0.7861952862, 0.8810510302, 0.8105672615, 0.8679801735, 0.8868599463, 0.7861952862, 0.8750313766, 0.898474064, 0.7469560871, 0.8428075504, 0.871484045, 0.8209439528, 0.8990815838, 0.8844769949, 0.8945940257, 0.8877870832, 0.8665588241, 0.860822463, 0.8600276434, 0.8735006868, 0.7777672978, 0.7629714836, 0.7783185841, 0.7452802243, 0.7806636149, 0.737037037, 0.7753434245, 0.7637227844, 0.8219161677, 0.8470307759, 0.7112676056, 0.6879194631, 0.8503009228, 0.890852827, 0.8675441394, 0.8615269966, 0.8665224952, 0.8802278218, 0.7751979516, 0.8557576314, 0.8059939076, 0.7554898306, 0.8936754894, 0.8384846032, 0.7880823171, 0.8300496798, 0.7580710476, 0.8802946771, 0.8745925884, 0.8876964117, 0.8925219587, 0.86410233, 0.8279680605, 0.8746281031, 0.8567192747, 0.7976065649, 0.7637227844, 0.8734773879, 0.8336082474, 0.8283375164, 0.7, 0.8781393014, 0.8611401658, 0.8761425931, 0.776690213, 0.8480762897, 0.8293383122, 0.8673559281, 0.7639284889, 0.8620613454, 0.8909038892, 0.8281883584, 0.8082233338, 0.8911163483, 0.8620880827, 0.7615384615, 0.8487637938, 0.8809968499, 0.7, 0.8487179487, 0.83312931, 0.7572175537, 0.7285714286, 0.8015317587, 0.8069440771, 0.8241886838, 0.8713388375, 0.8937010455, 0.8724339993, 0.8742241462, 0.8905771943, 0.7815904139, 0.8610598066, 0.7645880096, 0.8759088415, 0.7656838021, 0.8501346975, 0.8823752931, 0.7, 0.8674034383, 0.861488482, 0.8767166171, 0.8763883176, 0.823988604, 0.8176155548, 0.7823240589, 0.8363784034, 0.7681749679, 0.8911315624, 0.8788250459, 0.7937717157, 0.7352941176, 0.8531752462, 0.8357775422, 0.8428140673, 0.7689681051, 0.8759389333, 0.8521431664, 0.7593423294, 0.777238081, 0.9001059883, 0.840931965, 0.827340735, 0.8743205608, 0.8783340093, 0.8933605335, 0.8664215612, 0.7752633863, 0.8369110501, 0.786588137, 0.72380637, 0.8585523481, 0.8439598742, 0.8503422218, 0.8649596665, 0.8008356546, 0.7769356337, 0.8457964124, 0.8739133208, 0.8992386281, 0.8309882747, 0.7874859076, 0.7923076923, 0.8918472753, 0.7845793656, 0.7639284889, 0.7724372963, 0.8674280601, 0.7220243396, 0.8689638076, 0.7568807339, 0.8736358739, 0.7615384615, 0.8912190553, 0.8654606564, 0.737037037, 0.737037037, 0.833433599, 0.8772516826, 0.7866341553, 0.7796566114, 0.778486952, 0.8770617359, 0.7700400989, 0.737037037, 0.8712201964, 0.6744292237, 0.788061734, 0.8863354521, 0.6805516406, 0.8012820513, 0.8403249631, 0.8267103894, 0.8386264921, 0.8697170768, 0.8169092381, 0.888533385, 0.8095909406, 0.785635344, 0.8339405304, 0.8880337143, 0.740353461, 0.8048693417, 0.8012229051, 0.8548038525, 0.8368796578, 0.8987308302, 0.8136069869, 0.8669659864, 0.7716207631, 0.8643141403, 0.75549113, 0.8826812661, 0.8592841094, 0.6744292237, 0.8897987712, 0.8857611698, 0.8105764751, 0.8862969727, 0.8659162754, 0.8748764512, 0.8601114414, 0.6853922817, 0.9023385597, 0.8716860207, 0.8738995173, 0.8469398192, 0.8710687095, 0.8780209532, 0.8594278283, 0.7757861407, 0.7813472085, 0.7, 0.7615384615, 0.8879436737, 0.7736974262, 0.7482598608, 0.754913944, 0.8872764063, 0.8864847767, 0.7026963069, 0.8572670807, 0.8466727787, 0.739075883, 0.8013286094, 0.8522859233, 0.7917139614, 0.8169590643, 0.8822084086, 0.7909090909, 0.8152600574, 0.8235295738, 0.7996512854, 0.8450097847, 0.7902678874, 0.8061889251, 0.7704388505, 0.8208667193, 0.7, 0.8595779344, 0.8034164358, 0.7678698093, 0.8290566038, 0.8791120194, 0.8523468198, 0.8190615836, 0.8427000538, 0.8654463712, 0.8839331207, 0.7651650294, 0.8766355725, 0.8233814433, 0.863850962, 0.7447928331, 0.8434095746, 0.9008883623, 0.8354477612, 0.8688427723, 0.8607398928, 0.8782183187, 0.8596772431, 0.7860021906, 0.8335674508, 0.8451820128, 0.8740622933, 0.7684147334, 0.8645026271, 0.8027932961, 0.8511075716, 0.8797634062, 0.8572356187, 0.8794212011, 0.8169590643, 0.8591549296, 0.8924455369, 0.8001222992, 0.8177476598, 0.8700651734, 0.7690459393, 0.7700667801, 0.8635276532, 0.8678800857, 0.8604240283, 0.7756531284, 0.8819474351, 0.8443224958, 0.8717995459, 0.7917139614, 0.8892951641, 0.788, 0.8864337102, 0.8034132841, 0.7618978422, 0.7579945799, 0.7745281696, 0.8006306383, 0.8801983396, 0.8847337343, 0.7777860883, 0.852396998, 0.8679703582, 0.8472982976, 0.860087146, 0.7885487592, 0.8941568898, 0.8306122449, 0.8624163188, 0.8114645461, 0.8594380821, 0.8127427759, 0.8388347087, 0.8173098372, 0.8784454049, 0.7799909971, 0.8099181074, 0.8946996815, 0.8843967303, 0.8783950617, 0.8441828441, 0.8847153373, 0.7700877965, 0.7994678278, 0.7751300405, 0.8848497137, 0.8698126065, 0.836997315, 0.7782603767, 0.8800961463, 0.7828219513, 0.8598241665, 0.8692719992, 0.7352941176, 0.8806226815, 0.8207582938, 0.8715169302, 0.8817632112, 0.8952867243, 0.8531096895, 0.8584852718, 0.8601449611, 0.7725289459, 0.8752094862, 0.7615384615, 0.8691551185, 0.8410373219, 0.8619588361, 0.899154334, 0.7500968685, 0.8079188833, 0.7766117834, 0.8201517307, 0.8617418657, 0.8657678115, 0.8699427749, 0.8209439528, 0.7836024309, 0.8292346525, 0.8386256936, 0.8354602279, 0.7693277832, 0.7827043583, 0.7830449827, 0.8801730758, 0.8724385246, 0.8373364412, 0.7943435005, 0.8871168831, 0.8335894796, 0.8104935711, 0.8145009416, 0.8880948729, 0.8650604651, 0.8324649009, 0.8526313397, 0.7, 0.8937909624, 0.9022536409, 0.8645003885, 0.7394054569, 0.8948577098, 0.8825138374, 0.7655813315, 0.8057500825, 0.8032862435, 0.8799734343, 0.8545023697, 0.8022471719, 0.8993329481, 0.8697848924, 0.8858206856, 0.830074943, 0.8793618886, 0.8531317334, 0.857568534, 0.8951422327, 0.8791730376, 0.8245548654, 0.811747851, 0.8871867745, 0.8675831581, 0.7884193876, 0.8591464804, 0.7600652884, 0.7502213042, 0.8694535249, 0.721279523, 0.8738001742, 0.800359337, 0.86438115, 0.8833759674, 0.8719796954, 0.8728225309, 0.8874206139, 0.8696942322, 0.8115565831, 0.7112676056, 0.8622159091, 0.8076923077, 0.8204610579, 0.8384462317, 0.8804935974, 0.8707391415, 0.8677728618, 0.8292900293, 0.7906159285, 0.8794946081, 0.8422890283, 0.7598808879, 0.7854125929, 0.8824112651, 0.7760155878, 0.889344811, 0.891620483, 0.8103858189, 0.8823149631, 0.8384565075, 0.9027358908, 0.8545616741, 0.7649402719, 0.7651189785, 0.7694964189, 0.8693064877, 0.7647310587, 0.6834918757, 0.8768764958, 0.7693762459, 0.8972523118, 0.8882997814, 0.7861721953, 0.8474530831, 0.8833489441, 0.8782748833, 0.6925526703, 0.8583454558, 0.8807497667, 0.8634481165, 0.7143007361, 0.8214689232, 0.8382587351, 0.8834152583, 0.7874859076, 0.8545816733, 0.8585604657, 0.8754977029, 0.8005005776, 0.8778307662, 0.8306443164, 0.8913612265, 0.6519313305, 0.8695833136, 0.8154804986, 0.7854125929, 0.8649409763, 0.8122423173, 0.7533333333, 0.8801452551, 0.7991466883, 0.8405242179, 0.8663741257, 0.8613914563, 0.8211510791, 0.8656188471, 0.8938717199, 0.8742644102, 0.752992728, 0.8718045977, 0.8243333333, 0.8125075045, 0.861278166, 0.8558012062, 0.8859729901, 0.8659656799, 0.8618329283, 0.882683112, 0.8444956688, 0.8690043838, 0.8531353135, 0.8168940875, 0.8740927139, 0.9033081506, 0.8695219781, 0.7671618372, 0.7621733339, 0.8757492623, 0.8466769113, 0.8341315015, 0.7, 0.8264693622, 0.8649114097, 0.8536915346, 0.8087246828, 0.7, 0.856976591, 0.8691437449, 0.7754331384, 0.789848368, 0.8801262464, 0.8609923172, 0.821703854, 0.7702384521, 0.8868794694, 0.8764493865, 0.8607474111, 0.8571957617, 0.8257514903, 0.8262612003, 0.8455956232, 0.8059184447, 0.8761047505, 0.7571754506, 0.7982569923, 0.8066018631, 0.8222741117, 0.737037037, 0.8345636061, 0.881948848, 0.6789268293, 0.8436817175, 0.7, 0.8536557862, 0.7824027683, 0.8897067961, 0.8005962425, 0.7083063328, 0.8705063744, 0.8751905774, 0.8858793117, 0.8576899375, 0.8186282416, 0.8856353591, 0.8039839034, 0.7610114192, 0.7690894881, 0.7649371215, 0.8946685756, 0.8687293384, 0.7915531335, 0.8609341826, 0.8160833755, 0.8241184809, 0.6744292237, 0.853244396, 0.6793939394, 0.8709429662, 0.8426312537, 0.7719496619, 0.8817307386, 0.8038998079, 0.8715946472, 0.8592761382, 0.8734607422, 0.6831243469, 0.8010555143, 0.8983135705, 0.776280043, 0.7716981132, 0.8576857051, 0.8925761165, 0.7530733446, 0.8789751733, 0.7533333333, 0.7972317547, 0.7616216216, 0.8742654412, 0.8372616565, 0.77796495, 0.8251521164, 0.7809858904, 0.8799342105, 0.8392192192, 0.7708118486, 0.8450506793, 0.7800806632, 0.8717897496, 0.877010324, 0.7112676056, 0.7796548592, 0.8801319448, 0.8881002088, 0.8896703167, 0.8976552598, 0.7750396431, 0.8884713933, 0.8707425702, 0.7644050176, 0.7112676056, 0.7618778726, 0.7726262612, 0.8869824297, 0.7621819449, 0.8791714133, 0.8012146409, 0.7508511807, 0.7845153934, 0.869257873, 0.8647711125, 0.8076593874, 0.8712301681, 0.8981880179, 0.8372769953, 0.7908657665, 0.8416553967, 0.7, 0.8915954896, 0.8602040336, 0.8679941727, 0.8200383509, 0.9053730247, 0.7831568909, 0.8471628531, 0.8063228836, 0.8666532852, 0.8801060794, 0.7850309325, 0.8812752097, 0.87012895, 0.8014776033, 0.7753634529, 0.8644085872, 0.8625009777, 0.7112676056, 0.7102861883, 0.8849580091, 0.8471761508, 0.8869677998, 0.8951514594, 0.7704485272, 0.896019999, 0.8951628261, 0.7700279256, 0.8777296279, 0.8106262539, 0.8721525467, 0.8169360841, 0.7584755404, 0.8462892119, 0.8386326573, 0.865597478, 0.8869790417, 0.8706504532, 0.892343433, 0.8568010347, 0.7786730126, 0.7838483404, 0.8827427222, 0.8375949501, 0.8702503818, 0.7846702797, 0.8367396594, 0.7615384615, 0.8598016841, 0.8020118779, 0.7799121844, 0.7909090909, 0.8541082102, 0.7, 0.7587678221, 0.7739970445, 0.7388880261, 0.7600652884, 0.7656838021, 0.8693064877, 0.7926525529, 0.8022866199, 0.7959623677, 0.7892882056, 0.7810471382, 0.7573278115, 0.7717412728, 0.8679424786, 0.8941550799, 0.7243441763, 0.8607315151, 0.879904948, 0.8488430986, 0.8878318836, 0.8497847283, 0.7898876404, 0.7812783158, 0.8612718326, 0.8852082177, 0.8873008011, 0.7458654906, 0.7454539422, 0.8559593634, 0.8195161639, 0.7671698113, 0.8006641537, 0.8777720871, 0.8932415389, 0.8709706589, 0.769467139, 0.8685744909, 0.8517665746, 0.788, 0.7863255574, 0.7565739474, 0.8532755568, 0.7793179209, 0.8644751364, 0.8270564016, 0.7941176471, 0.8472430121, 0.7608119375, 0.8507487829, 0.7723249211, 0.8831378374, 0.7751878542, 0.8951034442, 0.864023984, 0.7670441503, 0.8603209292, 0.7896782327, 0.8610923449, 0.7843401308, 0.7941176471, 0.8823454462, 0.9051850122, 0.7968864469, 0.7761269035, 0.7758806614, 0.8174603175, 0.8796295702, 0.8169590643, 0.7719653748, 0.7909090909, 0.8785524631, 0.7525218686, 0.7639933893, 0.8513827478, 0.8445264267, 0.7396586904, 0.8421392087, 0.7694821602, 0.823932975, 0.8839394593, 0.8193517636, 0.8890380672, 0.8264693622, 0.7397306397, 0.8478091053, 0.8653278407, 0.8637571239, 0.7201825311, 0.8867116533, 0.8105192921, 0.8588623058, 0.812962963, 0.7705063941, 0.8366685415, 0.8508572511, 0.8801176945, 0.8746173702, 0.7780369337, 0.7817741546, 0.8057073082, 0.8883633278, 0.8460010003, 0.7800942285, 0.8437762857, 0.8450404473, 0.8694567367, 0.8726447078, 0.8594080338, 0.8767581762, 0.8070427154, 0.8324162154, 0.7712648055, 0.796, 0.842181216, 0.7774955049, 0.7671527867, 0.9199625914, 0.821093001, 0.8443113772, 0.8819760232, 0.8219890511, 0.8350516192, 0.8327971351, 0.8199016192, 0.7287179487, 0.8725122685, 0.7652369719, 0.8314188075, 0.7500938611, 0.7697838553, 0.8548189111, 0.7633251901, 0.8672292693, 0.7861294122, 0.8676113103, 0.862445896, 0.9063696822, 0.8511937132, 0.8290566038, 0.8783729156, 0.8509708738, 0.8861296391, 0.8554955591, 0.7197910722, 0.8820606139, 0.8201172127, 0.7249170437, 0.8645640045, 0.8581673307, 0.8815944597, 0.7861952862, 0.8788956624, 0.8602856875, 0.8771605137, 0.8887892523, 0.8749181089, 0.8902199946, 0.8734145395, 0.8464061282, 0.8404667253, 0.785690553, 0.8504932898, 0.7631574632, 0.8666749503, 0.8913871316, 0.8361330412, 0.8717415846, 0.8925268365, 0.8216684046, 0.7747557726, 0.775594264, 0.8596773483, 0.8840267194, 0.7689343816, 0.7860908597, 0.718269557, 0.7823400489, 0.8392307805, 0.7874859076, 0.8495550486, 0.6704430884, 0.8155352209, 0.8858423094, 0.7755316452, 0.7834649238, 0.9007323591, 0.8643144375, 0.884144301, 0.8846904701, 0.8873188233, 0.8704999334, 0.8779538669, 0.786422492, 0.7844699499, 0.8576948554, 0.8716171617, 0.7756057086, 0.8037794139, 0.8299387281, 0.8887032783, 0.8151969981, 0.8337383867, 0.8823023888, 0.8699906716, 0.8874149836, 0.8625604392, 0.802359882, 0.737037037, 0.8186891475, 0.8702694766, 0.7553139357, 0.8902192682, 0.8514529915, 0.8017580613, 0.8371580731, 0.7919990522, 0.89483803, 0.8604240283, 0.8027548209, 0.8076449732, 0.7971313329, 0.8692103055, 0.7556008558, 0.7572018779, 0.8693874517, 0.8509901078, 0.8070628491, 0.8840564922, 0.866593921, 0.7861952862, 0.8848806068, 0.8975692038, 0.9037464025, 0.8491157042, 0.7802203653, 0.8961649629, 0.8564600678, 0.8126649309, 0.8926989341, 0.7610114192, 0.8732265192, 0.8589539712, 0.7556824401, 0.8830080871, 0.8414667765, 0.8119128147, 0.8584031302, 0.8031313542, 0.8709077194, 0.8076923077, 0.7822576742, 0.7702151253, 0.7615384615, 0.8021513944, 0.8562918028, 0.7199271814, 0.8591836735, 0.8660124663, 0.8286195286, 0.8999365537, 0.8642513898, 0.8877999176, 0.8845003487, 0.8286195286, 0.783850282, 0.7814344076, 0.8556441909, 0.7664526485, 0.8497393838, 0.803183277, 0.8057728119, 0.7923076923, 0.7734216986, 0.754439474, 0.8532069039, 0.8263418662, 0.8188369678, 0.8972616266, 0.7509768453, 0.8815639565, 0.7704872609, 0.7725, 0.8558330768, 0.8860580085, 0.8122224445, 0.8503770761, 0.846430882, 0.7893235905, 0.8442025917, 0.8957012933, 0.7762699913, 0.8715547396, 0.6392653979, 0.7579945799, 0.8827297599, 0.8619588361, 0.8004632391, 0.8740479249, 0.8293413174, 0.8389856414, 0.8761785296, 0.8066753713, 0.8093823886, 0.8556499336, 0.8579861215, 0.753380723, 0.8503287642, 0.9038631603, 0.8973008309, 0.8631313131, 0.8723848183, 0.8422285959, 0.7112676056, 0.8635948306, 0.8598852543, 0.7, 0.8721757422, 0.8817458827, 0.792295659, 0.9000726276, 0.7767935322, 0.8354205034, 0.7751272641, 0.8999154038, 0.7081085724, 0.8456504473, 0.8432976622, 0.8976685908, 0.6879194631, 0.8039106145, 0.8678252259, 0.8571958295, 0.7777816946, 0.7741873166, 0.8704269313, 0.8840227232, 0.7701299271, 0.8795645062, 0.737037037, 0.7932225759, 0.7917126162, 0.8790023085, 0.8083872111, 0.8248445294, 0.8372390735, 0.7702493491, 0.8942297568, 0.8209439528, 0.8871276243, 0.821807364, 0.8840362999, 0.8645026271, 0.872532266, 0.7756914774, 0.7563826813, 0.8716313215, 0.750640322, 0.8958490938, 0.8584104445, 0.7030797924, 0.8704423676, 0.8037013603, 0.8983073261, 0.7860648554, 0.8877838684, 0.7991466883, 0.8515095372, 0.778973033, 0.878504628, 0.722087986, 0.767716325, 0.7761449192, 0.8085605998, 0.8359351674, 0.7598808879, 0.7707314399, 0.8805879518, 0.8772202387, 0.8842914624, 0.8003383035, 0.7862254025, 0.8540167921, 0.7534611289, 0.8876234236, 0.8632814233, 0.7918767507, 0.8805713847, 0.8727552022, 0.7660347874, 0.7781785392, 0.8426793861, 0.8919605563, 0.8885253724, 0.8834829212, 0.866035414, 0.8048633521, 0.8632318201, 0.879512765, 0.8958831788, 0.7328935938, 0.8599235299, 0.7889670231, 0.8983599668, 0.8669134877, 0.8103705083, 0.8083032491, 0.7459037591, 0.7245910941, 0.7923076923, 0.8475878862, 0.7736974262, 0.8231383059, 0.8000712758, 0.8306122449, 0.8788938658, 0.8556818708, 0.8970899733, 0.8778411097, 0.8134831461, 0.8317105322, 0.8054328854, 0.8435639413, 0.8838048274, 0.7844408382, 0.8891359747, 0.8019025522, 0.8011565431, 0.6959504537, 0.834684677, 0.8434495769, 0.8736880675, 0.7967767608, 0.8211791812, 0.7947368421, 0.8928639298, 0.8954845039, 0.8794813514, 0.8746057718, 0.804963799, 0.8052407561, 0.7832945752, 0.8421476609, 0.8642645607, 0.8337453646, 0.7112676056, 0.7227221768, 0.8019004361, 0.8077426747, 0.8277118297, 0.7568807339, 0.7703654433, 0.7682100915, 0.7681443319, 0.8435355534, 0.829481723, 0.8306072754, 0.8348535163, 0.7749836493, 0.7967649302, 0.8720803911, 0.886726981, 0.8941452114, 0.8370371889, 0.7352941176, 0.8817626948, 0.859988374, 0.8860876249, 0.7326762564, 0.8669767862, 0.8242556701, 0.8544962615, 0.7701396981, 0.8707106462, 0.7863662512, 0.8942074831, 0.8911901315, 0.8212592305, 0.8798842953, 0.819655255, 0.8708332028, 0.8190615836, 0.8493983702, 0.8465552746, 0.8003864869, 0.871988587, 0.8700407396, 0.8459938711, 0.7, 0.7961432429, 0.7109311269, 0.8968743506, 0.8635063214, 0.8752478088, 0.8266930659, 0.7112676056, 0.8795528618, 0.7722949656, 0.8407161314, 0.7815937149, 0.8470493457, 0.8074507067, 0.8111738414, 0.8531061764, 0.8679171207, 0.8410373219, 0.8134202608, 0.7864192776, 0.812281916, 0.7231191377, 0.8955867603, 0.7976033345, 0.79536237, 0.8092689529, 0.879588674, 0.8844793509, 0.8639883833, 0.8310606285, 0.7796279831, 0.8598578868, 0.7536598721, 0.8771251636, 0.780975016, 0.7539708629, 0.785465223, 0.8764136472, 0.8199959232, 0.7713927952, 0.8758315317, 0.8898004073, 0.8730159986, 0.776339292, 0.8024757558, 0.8867351249, 0.8685473299, 0.7835272367, 0.7930115951, 0.7917006803, 0.7883913579, 0.8834161551, 0.8914113946, 0.7967767608, 0.8245738466, 0.7563969761, 0.8451127165, 0.8770825429, 0.8645026271, 0.8899868334, 0.8451879906, 0.8033106576, 0.8330512061, 0.8490324697, 0.8149342397, 0.7408223201, 0.8976595152, 0.8653285187, 0.7693662493, 0.7739698745, 0.8722434314, 0.8875004067, 0.8624467903, 0.7833386769, 0.8831895053, 0.8741403808, 0.6792746114, 0.792873748, 0.8582488251, 0.8043656021, 0.8858748619, 0.7457811063, 0.8735988411, 0.8766623249, 0.863129113, 0.7, 0.8841640733, 0.8118183685, 0.7325892063, 0.788, 0.7764053029, 0.7713927952, 0.893907563, 0.7620574296, 0.8685134119, 0.8691551185, 0.7619538968, 0.8156884762, 0.7518709677, 0.8575731122, 0.7874859076, 0.7797997809, 0.8012146409, 0.7921599074, 0.8330512061, 0.8342150004, 0.885912288, 0.8542425058, 0.8427510936, 0.8514095273, 0.849364059, 0.8332045233, 0.8209956211, 0.807280797, 0.8728448477, 0.7839779006, 0.8328237792, 0.8628067688, 0.886482658, 0.7094674556, 0.8852390852, 0.8309230653, 0.8612054795, 0.785360677, 0.8568973116, 0.8991336409, 0.8944426333, 0.7967480364, 0.7185242455, 0.7954281154, 0.8093991318, 0.719835624, 0.8757587411, 0.7287179487, 0.8651697419, 0.8796695971, 0.8433210729, 0.8011048955, 0.791651865, 0.8836987438, 0.8861840825, 0.8679458454, 0.8157063197, 0.7827512833, 0.887634672, 0.6809599982, 0.7112676056, 0.8550450333, 0.7970208982, 0.7858358475, 0.8727473512, 0.7, 0.8009678668, 0.7838371993, 0.8455433344, 0.8588456713, 0.7808850384, 0.7594142259, 0.8087652897, 0.846741476, 0.7578209277, 0.8876006866, 0.8420689655, 0.8699902607, 0.8195146613, 0.8112548664, 0.6744292237, 0.863498004, 0.7108171206, 0.8514071465, 0.842896814, 0.7768289777, 0.8780487412, 0.8074528172, 0.7600652884, 0.6934151641, 0.682122905, 0.8893373933, 0.8734709234, 0.8743458184, 0.8958253209, 0.872532266, 0.7641508432, 0.8061913696, 0.7847385869, 0.784244827, 0.7, 0.6856972586, 0.9029426417, 0.7617748751, 0.8659289151, 0.7762686192, 0.7739209184, 0.8353216873, 0.8429180925, 0.8725251446, 0.8653095106, 0.8855730537, 0.8666300836, 0.8767175695, 0.8039956649, 0.8275076505, 0.8472168824, 0.8486474882, 0.8457996369, 0.8869423074, 0.8468606061, 0.8628067688, 0.7615384615, 0.8739304466, 0.8941412062, 0.7568807339, 0.8923307409, 0.8809554355, 0.7, 0.8885285412, 0.8007162082, 0.7532530311, 0.8988588202, 0.8714954898, 0.8669759695, 0.8221274868, 0.8694439099, 0.7897625646, 0.8457573732, 0.755820743, 0.8596326922, 0.7989010989, 0.8282179749, 0.82, 0.8067266254, 0.7898876404, 0.8370828372, 0.872766027, 0.806760275, 0.8369403777, 0.8546980286, 0.847955643, 0.8796557924, 0.8055430605, 0.8892196804, 0.8651923858, 0.8601853895, 0.7750098078, 0.8833034836, 0.8542877413, 0.8751485306, 0.8898167073, 0.7712049651, 0.8637848385, 0.7859202523, 0.8848673755, 0.7545812239, 0.8792014156, 0.8852021851, 0.8565210107, 0.8176628189, 0.8667050322, 0.7209796534, 0.7753782041, 0.8687567925, 0.8093756828, 0.8193991773, 0.8830184762, 0.7821610808, 0.8709785533, 0.7509790906, 0.808381914, 0.8052873747, 0.7233161702, 0.8585604657, 0.7766414017, 0.7, 0.6669490674, 0.8791989975, 0.6760560998, 0.8367157795, 0.7243560674, 0.7, 0.8934083181, 0.8365957136, 0.8671310581, 0.7626117485, 0.7840626163, 0.8754028301, 0.8786815096, 0.7867253241, 0.8137269685, 0.833615939, 0.8092492137, 0.7697704135, 0.8638630763, 0.7890624558, 0.8602600828, 0.7849034417, 0.8748733536, 0.8832408569, 0.8000781468, 0.7, 0.7920132875, 0.8693773836, 0.8190722541, 0.7544502618, 0.8764191357, 0.7882882883, 0.7828512867, 0.851265813, 0.8046308146, 0.8712500584, 0.7677353342, 0.8878122659, 0.7711032907, 0.714226273, 0.7707683584, 0.8590755809, 0.866634903, 0.8767703197, 0.7775670409, 0.697744431, 0.8891860577, 0.7572794437, 0.8779499746, 0.8830330543, 0.8879192881, 0.7740800022, 0.77709108, 0.7964354556, 0.8636038971, 0.7993253586, 0.8651747122, 0.8763066202, 0.809656819, 0.8674141963, 0.830634715, 0.8691919997, 0.7980062197, 0.8702198258, 0.8242342619, 0.8718720006, 0.7733151539, 0.8778986691, 0.8082229986, 0.894218134, 0.8459313062, 0.8833224511, 0.8830459004, 0.8531205023, 0.8682735009, 0.7464900476, 0.8795871043, 0.9008277323, 0.8815536308, 0.7807976565, 0.8574793324, 0.8686337317, 0.8850432353, 0.8815166325, 0.7706393341, 0.7915187377, 0.8415991291, 0.768302934, 0.8706586262, 0.8558411032, 0.62036639, 0.7917139614, 0.7636584112, 0.8558361153, 0.8610399006, 0.8677381064, 0.8649422546, 0.8330049261, 0.8839848049, 0.8678415046, 0.8496858594, 0.8719395378, 0.7112676056, 0.8814552107, 0.875109837, 0.7605405405, 0.8655442592, 0.7968730478, 0.7770779576, 0.7733309933, 0.7724618298, 0.7645129371, 0.8613624268, 0.8725005518, 0.7525262129, 0.8358941528, 0.6745454545, 0.8568474587, 0.8537876189, 0.896176893, 0.7133868675, 0.7518267821, 0.8740365894, 0.7651189785, 0.8867413723, 0.8366985318, 0.8442057589, 0.8241705247, 0.8483273114, 0.7769139583, 0.8813705484, 0.8150939622, 0.8820877322, 0.8840268478, 0.8928814446, 0.8636050288, 0.8531048514, 0.8899174979, 0.7832925192, 0.737037037, 0.8559290666, 0.8825775961, 0.870214122, 0.8829594965, 0.7, 0.7799202748, 0.8586406008, 0.7587547335, 0.879392778, 0.8119543044, 0.846756383, 0.6958945222, 0.8788044486, 0.8173678994, 0.8397555042, 0.8782720819, 0.776173336, 0.8677386639, 0.882546324, 0.8810994621, 0.7972677596, 0.8768185492, 0.8065176909, 0.737037037, 0.7, 0.8343764817, 0.8128205128, 0.8760675019, 0.7518357137, 0.8525613686, 0.8867326287, 0.8526177812, 0.7669066401, 0.9018364438, 0.8302071251, 0.7736528447, 0.8869094868, 0.7187205927, 0.8119569307, 0.8544087648, 0.8113534317, 0.8503327723, 0.7766117834, 0.8103448276, 0.8863517094, 0.8824379783, 0.8603591477, 0.7800322567, 0.7, 0.8604785196, 0.8844244011, 0.8608085363, 0.8808559389, 0.8872074883, 0.857988299, 0.7658213307, 0.887609079, 0.8514491175, 0.8816223431, 0.8725521434, 0.8128205128, 0.8470832965, 0.7475028249, 0.8904558706, 0.8668704156, 0.8592475814, 0.8546858013, 0.7, 0.8799706305, 0.8239193084, 0.7715283774, 0.8917152821, 0.8342471395, 0.7900993879, 0.8520983223, 0.8889839739, 0.8381569153, 0.8523468198, 0.7866085619, 0.8961094217, 0.8514529915, 0.8684344266, 0.793513313, 0.8730107376, 0.8506078055, 0.7979608242, 0.8147164839, 0.889168306, 0.82359769, 0.7552102278, 0.811907569, 0.8776540919, 0.7216370542, 0.8562393162, 0.8727579231, 0.7112676056, 0.8532323015, 0.7706939588, 0.8666532852, 0.8428301714, 0.8726266853, 0.8534654229, 0.8042637221, 0.7685183665, 0.893613495, 0.8770584312, 0.7786907503, 0.847593185, 0.8789940686, 0.8806511045, 0.8907899675, 0.7776230556, 0.719123506, 0.8256300873, 0.788, 0.8467022911, 0.7967054853, 0.892046084, 0.8651144748, 0.8604187656, 0.8786561762, 0.8587316698, 0.8611844385, 0.8042811794, 0.8457964124, 0.8343449107, 0.868004063, 0.7, 0.6876350776, 0.8658096682, 0.7, 0.7716518613, 0.7967767608, 0.8465417978, 0.7764704889, 0.8808643372, 0.8823277389, 0.8871713848, 0.8992481973, 0.7614330125, 0.8868442937, 0.7731600718, 0.8373075896, 0.8884865247, 0.7638045157, 0.7465116279, 0.8667447307, 0.8697230438, 0.8755607164, 0.8929348355, 0.813747646, 0.7860196016, 0.8531405036, 0.7774404828, 0.7946723773, 0.8857553466, 0.8014753047, 0.8303546056, 0.8869781344, 0.8640862582, 0.8994656554, 0.847229778, 0.8464350951, 0.8054012891, 0.8947304365, 0.8675189019, 0.8547252747, 0.8259017933, 0.8102934481, 0.7610114192, 0.7, 0.8904085816, 0.8252225519, 0.8699559716, 0.8005446712, 0.833615939, 0.866405053, 0.8609341826, 0.9018911744, 0.852801839, 0.8831319718, 0.8345949394, 0.7810741688, 0.8276997353, 0.7602183835, 0.884691326, 0.7615384615, 0.8230613913, 0.880122734, 0.772048018, 0.795483871, 0.8760603622, 0.8647980901, 0.8873244029, 0.8537304406, 0.8628067688, 0.8646832659, 0.8742350664, 0.8803142181, 0.7671262387, 0.7788917956, 0.742352859, 0.7834403447, 0.8571729376, 0.800931913, 0.833546054, 0.8760802547, 0.8549383556, 0.7094674556, 0.7896630497, 0.7917139614, 0.7944012397, 0.7782328184, 0.8687144084, 0.8791207038, 0.8270921986, 0.8459275821, 0.8584042175, 0.8794501535, 0.8351059478, 0.8837556315, 0.8745147571, 0.7860556026, 0.8576948554, 0.7935367118, 0.8849951604, 0.7924974896, 0.854582463, 0.7706660368, 0.7590796257, 0.8592143624, 0.8070427154, 0.8520166423, 0.7771510661, 0.7849769763, 0.8813045711, 0.846741476, 0.8659533013, 0.8814293946, 0.7695439241, 0.8840319491, 0.8793001014, 0.8306122449, 0.833546054, 0.8769796542, 0.8779319018, 0.8373948718, 0.7595043843, 0.8890658001, 0.6899161216, 0.8632028431, 0.8931965687, 0.83312931, 0.858813664, 0.8270846877, 0.8048294963, 0.857, 0.8942577261, 0.8253658449, 0.7605405405, 0.8755452671, 0.8257093075, 0.8811140256, 0.8646947706, 0.8837010122, 0.8170970126, 0.8805210537, 0.7687538748, 0.8222741117, 0.7, 0.8375049821, 0.7845815861, 0.8962239283, 0.830914626, 0.8368836764, 0.8688657379, 0.7504263997, 0.7780963224, 0.8632018755, 0.8534699926, 0.8588752536, 0.8464147703, 0.7956331315, 0.773019035, 0.7352941176, 0.8935890699, 0.7066666667, 0.7610114192, 0.8657222362, 0.8361407695, 0.8543472192, 0.7666411922, 0.8739218286, 0.7535865757, 0.8940953982, 0.8587712519, 0.7617081626, 0.8637599875, 0.7714303616, 0.8479039263, 0.8804279815, 0.826737795, 0.8455466226, 0.8680887942, 0.7686778223, 0.8189174324, 0.8899089727, 0.8480248952, 0.7810911414, 0.8717287867, 0.8917879258, 0.7655518395, 0.8831016043, 0.8165124788, 0.7210431203, 0.8615790797, 0.8629546587, 0.8406425833, 0.8444667387, 0.8488182459, 0.8989313748, 0.8760618004, 0.7801257383, 0.8093664283, 0.8973398462, 0.7943125917, 0.768681151, 0.815743817, 0.8215084996, 0.8836903262, 0.7614330125, 0.7713927952, 0.737037037, 0.8246427576, 0.8332209106, 0.8697230438, 0.8608432549, 0.8709422628, 0.833897256, 0.8978388673, 0.8880161331, 0.8909563852, 0.8716283716, 0.7679186534, 0.8566769578, 0.8523314788, 0.8198033995, 0.8620880827, 0.7066666667, 0.8867993884, 0.8062197026, 0.8579802341, 0.8161906357, 0.8522007097, 0.8001000333, 0.8618180425, 0.7, 0.8423881529, 0.8706601768, 0.762055865, 0.8656499432, 0.8159450525, 0.8593512539, 0.8910304135, 0.8376144089, 0.7397920728, 0.8650152677, 0.8085605998, 0.864651569, 0.8272393202, 0.7223752601, 0.7787519268, 0.8158253954, 0.8844584719, 0.7844540528, 0.7671804787, 0.7654213342, 0.8827428608, 0.8217757981, 0.8401546157, 0.8880070547, 0.8869890765, 0.7829798123, 0.755820743, 0.8923304125, 0.8890837529, 0.8052301983, 0.8744587541, 0.7024446726, 0.8913760163, 0.7793515429, 0.8584261321, 0.8117666156, 0.8769845738, 0.858813664, 0.8861749026, 0.8942126232, 0.8674034383, 0.7695355667, 0.8061889251, 0.7507137106, 0.7500968685, 0.8499668934, 0.8746494955, 0.8367095031, 0.8379305297, 0.8777379223, 0.8106928323, 0.8732934215, 0.7664526485, 0.8606139788, 0.8456511631, 0.8022222222, 0.719123506, 0.859146248, 0.7578209277, 0.8854545455, 0.8628059535, 0.8364238411, 0.8764151889, 0.7249904916, 0.899420441, 0.8496858594, 0.8261795407, 0.6007673154, 0.8597237482, 0.809178744, 0.6866534194, 0.7713564294, 0.8901602542, 0.7, 0.8826879271, 0.8583395476, 0.888016228, 0.8028774098, 0.8729201245, 0.8027932961, 0.7246109819, 0.8532425391, 0.8841524284, 0.7389588928, 0.8021165318, 0.8101989246, 0.7872018077, 0.8890224778, 0.8380465428, 0.8522023182, 0.8733648819, 0.8372647874, 0.9001285224, 0.7503612335, 0.8543271948, 0.8749771999, 0.6447769697, 0.8804823331, 0.775862069, 0.8688079239, 0.8416664122, 0.8169590643, 0.8598496104, 0.8578163544, 0.784588373, 0.8546191384, 0.8753026325, 0.8981378617, 0.8573136568, 0.7889918388, 0.8841248788, 0.7754801545, 0.7504263997, 0.7705891485, 0.7874859076, 0.8006029285, 0.8786086133, 0.8237163078, 0.7476724621, 0.8528484155, 0.7908952967, 0.860501646, 0.7799626263, 0.7615384615, 0.7845243056, 0.8423983828, 0.8027932961, 0.8679175865, 0.7737046704, 0.7670920138, 0.7269492361, 0.8345717732, 0.7849834227, 0.860837057, 0.9013295744, 0.7718623137, 0.8470827435, 0.8696310194, 0.8850470316, 0.7670786277, 0.7454194691, 0.7629764703, 0.7720882067, 0.8681627588, 0.7784504714, 0.865177665, 0.8424714607, 0.6922215944, 0.8693089973, 0.8921926721, 0.8234148363, 0.7904271871, 0.8674914542, 0.8761037094, 0.7750098078, 0.8569961652, 0.8904535509, 0.8391351653, 0.7651694095, 0.8782093989, 0.7749806352, 0.8076923077, 0.8434655686, 0.86917111, 0.7818198395, 0.8416863045, 0.769467139, 0.881911928, 0.8636686496, 0.7617748751, 0.7815120197, 0.8625498008, 0.8616838409, 0.8527023188, 0.8899089319, 0.8191049272, 0.8932966865, 0.7698795181, 0.8870988648, 0.7, 0.8616643378, 0.8277118297, 0.7, 0.8062197026, 0.7682474529, 0.7, 0.8497744676, 0.7887312845, 0.8581487168, 0.7556891175, 0.8098896688, 0.8161033324, 0.78925, 0.8426793861, 0.8086451613, 0.8006323397, 0.7792338196, 0.8258954515, 0.8702682941, 0.8475132469, 0.864023984, 0.832250777, 0.8642866405, 0.7760122855, 0.8717215415, 0.7948871341, 0.7674745537, 0.8878362436, 0.9194175417, 0.8627318485, 0.8822207038, 0.8804199185, 0.7737362249, 0.7695508273, 0.8110416055, 0.8668141196, 0.8266820975, 0.8013728233, 0.872385466, 0.8729873586, 0.7610114192, 0.8334565729, 0.8858193684, 0.808676353, 0.7796278781, 0.8210132135, 0.9013804088, 0.7112676056, 0.8433946131, 0.811747851, 0.8258739624, 0.7616216216, 0.8581487168, 0.8376489552, 0.8163252542, 0.8756435771, 0.8607406697, 0.8087007716, 0.8878100064, 0.8571428571, 0.8962811744, 0.775365755, 0.8689695558, 0.901082216, 0.8923802684, 0.7285714286, 0.8967735133, 0.8605280174, 0.76414965, 0.8467278033, 0.7570356473, 0.8841438496, 0.8115691527, 0.8489601862, 0.8822461889, 0.7792738879, 0.8290007446, 0.8451110748, 0.8056515783, 0.7833803459, 0.737037037, 0.857551645, 0.7834809395, 0.7893914732, 0.8782902534, 0.7102672868, 0.8877499862, 0.8853611952, 0.811747851, 0.8333653846, 0.9012389644, 0.8320661157, 0.872532266, 0.8214894827, 0.8452805041, 0.8670956222, 0.8703272075, 0.8448813569, 0.7466581698, 0.8105306437, 0.8632814233, 0.8953075126, 0.682122905, 0.7533854966, 0.6960215423, 0.8596835576, 0.8866923077, 0.8519293969, 0.7828743519, 0.7880695116, 0.8909680785, 0.7, 0.8264693622, 0.7800813777, 0.8138564274, 0.8927245124, 0.8079080139, 0.7944874715, 0.8625498008, 0.7570356473, 0.7843401308, 0.8258204991, 0.7, 0.7752633863, 0.8740600462, 0.8479135154, 0.870028214, 0.7, 0.7961054592, 0.7901392758, 0.7802318094, 0.8632241814, 0.876040157, 0.7463706082, 0.8536216366, 0.8607502328, 0.8933912473, 0.7882882883, 0.811747851, 0.7, 0.7808140708, 0.8876735744, 0.80407951, 0.6896031302, 0.8671857461, 0.755752142, 0.7, 0.7656838021, 0.7930223969, 0.8572209895, 0.8976465547, 0.8388347087, 0.7651694095, 0.7452610707, 0.8610645543, 0.8835530345, 0.852895885, 0.8386782265, 0.8815587807, 0.8085604989, 0.8233985044, 0.8685023879, 0.7777276826, 0.8761044924, 0.7832496702, 0.8613131313, 0.7540103372, 0.7907936508, 0.7636216353, 0.7217370209, 0.8123262822, 0.8321808747, 0.7914918964, 0.8741870431, 0.8911302842, 0.7638471612, 0.8190615836, 0.7649183809, 0.8846034265, 0.8581487168, 0.8733101784, 0.8909303889, 0.8661909152, 0.8719654931, 0.8366737168, 0.7807793426, 0.8686720123, 0.9019004585, 0.8874558102, 0.737037037, 0.7971377614, 0.8823564564, 0.8684223488, 0.7, 0.6642845757, 0.7973370064, 0.8640511793, 0.7697676463, 0.7605405405, 0.876589914, 0.887488862, 0.872582768, 0.8630492604, 0.8070866142, 0.891967189, 0.8654400896, 0.777480663, 0.8109640777, 0.7229071162, 0.8735562988, 0.7766497462, 0.7765753849, 0.8002724796, 0.8297297297, 0.7554665898, 0.823745341, 0.8642169285, 0.7765788484, 0.8624546047, 0.8372881356, 0.8434682461, 0.8749760713, 0.8979940149, 0.8107488538, 0.8163040637, 0.8843240988, 0.7738902564, 0.8637238274, 0.8920875149, 0.7610114192, 0.9002955727, 0.7726237164, 0.8812585981, 0.8958689988, 0.7618602914, 0.7577292412, 0.851491205, 0.7161309406, 0.8655598524, 0.7, 0.8602166451, 0.7339110165, 0.8840664281, 0.6347387718, 0.8397415943, 0.8740769197, 0.857459926, 0.7876571596, 0.8161562956, 0.8839196305, 0.766330699, 0.8100067913, 0.8447955136, 0.8158971498, 0.7642694996, 0.7774832755, 0.8771298152, 0.7821899702, 0.8097329554, 0.8598042242, 0.811747851, 0.8221283135, 0.845751634, 0.8602566926, 0.6811420109, 0.7467357701, 0.8831182135, 0.7453333333, 0.7890828717, 0.7525218686, 0.7736889087, 0.798313253, 0.8817312491, 0.8791310937, 0.8741429669, 0.7806646194, 0.829182435, 0.8633800384, 0.8938614394, 0.7263457505, 0.8285203115, 0.8347511697, 0.8884152408, 0.8910539401, 0.8084350819, 0.8573376913, 0.874734329, 0.8228441028, 0.888369929, 0.7692264895, 0.8928807763, 0.7759342268, 0.8530844782, 0.7888059701, 0.7447928331, 0.8644331085, 0.8961789513, 0.7640680704, 0.7697287523, 0.737037037, 0.8485087077, 0.84431035, 0.771978854, 0.8004093255, 0.8465373377, 0.8585604657, 0.7694951358, 0.8838592861, 0.8421324899, 0.8427584718, 0.7543051349, 0.8062752038, 0.876625687, 0.8352876512, 0.7461993868, 0.7715808171, 0.7568807339, 0.8476958958, 0.7879820124, 0.7, 0.7737362249, 0.7, 0.8614167886, 0.8309882747, 0.9021478365, 0.7752040374, 0.8645223878, 0.860822463, 0.8556201237, 0.8256949504, 0.8765964857, 0.8391156463, 0.7777276826, 0.796954431, 0.8608592306, 0.7604554249, 0.8714062276, 0.7917580764, 0.7453750233, 0.8943155907, 0.8902300538, 0.8630999546, 0.7803730406, 0.8802825503, 0.7749806352, 0.8682863316, 0.8744149171, 0.7814977606, 0.7692401887, 0.7625648324, 0.9059101458, 0.8698429386, 0.8570402668, 0.8576899375, 0.8728884437, 0.8309882747, 0.878521721, 0.8386256936, 0.8082522988, 0.7783196877, 0.7960149001, 0.7458630348, 0.8435355534, 0.8704454454, 0.8263120796, 0.8888505886, 0.8519721928, 0.8856593994, 0.8928331305, 0.7897728247, 0.8429080262, 0.7, 0.8756271262, 0.8640558509, 0.750640322, 0.8560439429, 0.8812650738, 0.8347511697, 0.7759554451, 0.8561001437, 0.7862560585, 0.8619267168, 0.8985030582, 0.745971554, 0.7238476614, 0.7112676056, 0.7602729107, 0.7944653383, 0.817791411, 0.7534611289, 0.806863893, 0.853972632, 0.8877864269, 0.894314094, 0.8725391057, 0.8998320811, 0.8005005776, 0.8467700415, 0.8712667434, 0.7485875706, 0.7476190476, 0.8838909261, 0.8360452627, 0.8764513324, 0.8854443776, 0.8309987184, 0.7, 0.8742552192, 0.7719764595, 0.7549088055, 0.8414277932, 0.8685413892, 0.8565858623, 0.7667998502, 0.8066753713, 0.7752040374, 0.7476724621, 0.8366205219, 0.7595572652, 0.8088329654, 0.8730178706, 0.8659803523, 0.7639320534, 0.8545295516, 0.7652391661, 0.8791012575, 0.8960142958, 0.8759293477, 0.8070866142, 0.7717794751, 0.6928368391, 0.8359351674, 0.8099204737, 0.7690909091, 0.8987101757, 0.8703479576, 0.8927661029, 0.7858190575, 0.7536238851, 0.7, 0.755820743, 0.8671576375, 0.8279680605, 0.857568534, 0.8020235337, 0.7579945799, 0.7727352451, 0.8053188705, 0.8402398243, 0.7882949765, 0.8491638387, 0.8632431486, 0.827176134, 0.8776162931, 0.8133701016, 0.7112676056, 0.7643689865, 0.8524270786, 0.8691382178, 0.7, 0.8917275258, 0.8952144, 0.892668785, 0.8437298937, 0.8800832285, 0.8863963852, 0.7766497462, 0.8149452995, 0.8465248183, 0.8489103165, 0.8154804986, 0.8772044467, 0.7670441503, 0.7884703229, 0.8350844278, 0.8926470588, 0.7736127131, 0.8801134811, 0.850133398, 0.8712503281, 0.886700058, 0.7655875315, 0.8906255104, 0.8447236181, 0.7720534508, 0.8945413341, 0.7503612335, 0.8174178762, 0.8053892216, 0.8031501495, 0.8844276734, 0.8804107221, 0.7914583032, 0.8548021904, 0.8496858594, 0.7837057196, 0.8417384828, 0.7898876404, 0.8479270562, 0.7570356473, 0.865379798, 0.8641550326, 0.8470307759, 0.8125075045, 0.7556072064, 0.8952478115, 0.8889530069, 0.780455425, 0.7899889989, 0.8184372984, 0.8464965093, 0.767387331, 0.8957852517, 0.7560932459, 0.8900794294, 0.7692725539, 0.7590796257, 0.8616723345, 0.853994268, 0.8030630631, 0.8695138415, 0.8029529712, 0.8293413174, 0.8177321157, 0.8630769231, 0.8083089455, 0.8573011253, 0.7703654433, 0.8896273385, 0.8572364836, 0.7588248183, 0.9000450979, 0.8473204676, 0.7996512854, 0.8978544021, 0.794141701, 0.7842660637, 0.883475629, 0.8757163585, 0.8654493229, 0.8717395553, 0.863788855, 0.7722905682, 0.8529266714, 0.899913883, 0.8498779258, 0.8440531889, 0.7917139614, 0.862406015, 0.8538463389, 0.7731238184, 0.8828615149, 0.7536547231, 0.9191527669, 0.8076923077, 0.8573979861, 0.8852962616, 0.8923819463, 0.8008356546, 0.867972031, 0.7767478777, 0.8672287546, 0.8472823618, 0.8075620767, 0.8868615018, 0.8235329599, 0.7, 0.7735677871, 0.6319838057, 0.8731229577, 0.6912221561, 0.8677710365, 0.8320862638, 0.8736917449, 0.874201496, 0.7706149251, 0.78, 0.8650287293, 0.7986259036, 0.8302378256, 0.8598221113, 0.8965591483, 0.7581333333, 0.7740082305, 0.9191155492, 0.8878345578, 0.8874414726, 0.8121387283, 0.7345730891, 0.7571900578, 0.8600588827, 0.8609484299, 0.88267369, 0.8209439528, 0.7700866714, 0.8573011253, 0.8079731343, 0.8221274868, 0.8444249501, 0.8347880847, 0.8153187441, 0.8918285881, 0.8135958209, 0.8793376644, 0.8647553287, 0.7453750233, 0.8414554954, 0.8187470524, 0.8318884554, 0.8585054598, 0.7773730439, 0.8959260656, 0.7466789202, 0.8817489213, 0.8581755352, 0.8748513504, 0.8189769918, 0.8663882798, 0.7962488564, 0.8420291594, 0.7469544554, 0.8513112057, 0.8565297719, 0.8446029643, 0.8753719747, 0.8652526814, 0.7814168762, 0.889302177, 0.826824609, 0.7951975971, 0.8084659746, 0.8523622904, 0.713487882, 0.7687426143, 0.7670435747, 0.9194026301, 0.8960465078, 0.7983110344, 0.7729332317, 0.886599262, 0.8696238692, 0.7890274453, 0.8736781436, 0.799628493, 0.8460043926, 0.7615384615, 0.6953088708, 0.8596484243, 0.8928421495, 0.8012820513, 0.8693166844, 0.7750293083, 0.8454034939, 0.7287179487, 0.8734000096, 0.8676781741, 0.8664832919, 0.8892743468, 0.8482462419, 0.7993197279, 0.8840123775, 0.7979300371, 0.899137537, 0.8678104775, 0.793204757, 0.7758559122, 0.8746320545, 0.785106383, 0.6866534194, 0.7617748751, 0.8456619714, 0.8683023125, 0.8931342011, 0.8195824635, 0.8612936453, 0.7697287523, 0.8446037825, 0.8393246154, 0.8115354018, 0.7799121844, 0.8328031323, 0.8031749162, 0.719123506, 0.8671480923, 0.8842703101, 0.7616216216, 0.8467878824, 0.8773919993, 0.7867053416, 0.8414195971, 0.8855703322, 0.8830822328, 0.8597128931, 0.787040178, 0.6702216646, 0.8486336155, 0.8615608863, 0.8736447039, 0.8511692312, 0.9026886182, 0.8792290336, 0.852985191, 0.8667483842, 0.8812746684, 0.8265130702, 0.6899194223, 0.8543486078, 0.8903986424, 0.8703272075, 0.8704310972, 0.8338044865, 0.8444183907, 0.7475045769, 0.8716135569, 0.7783813995, 0.7, 0.7719941842, 0.8549497073, 0.8933614135, 0.8755401419, 0.8215054217, 0.8190615836, 0.7750277703, 0.8010557712, 0.8521388292, 0.8571958295, 0.8021435228, 0.8505926084, 0.8626223092, 0.8207673861, 0.8749887574, 0.7341092264, 0.8450635415, 0.6889447236, 0.7878400671, 0.819582867, 0.8192108114, 0.8463616452, 0.8229009453, 0.8123479871, 0.8695037219, 0.9014258713, 0.8027932961, 0.8597494127, 0.7629427277, 0.8964490245, 0.860837057, 0.8315435647, 0.7711829409, 0.8364523652, 0.6319838057, 0.8822008566, 0.7617195862, 0.7907044419, 0.8707087798, 0.7664526485, 0.8113438797, 0.862422424, 0.8285203115, 0.8454113853, 0.8647449763, 0.8364664289, 0.8098080772, 0.8965140897, 0.8674883675, 0.8671445799, 0.7123613707, 0.8860963587, 0.816242766, 0.8771487391, 0.7776113566, 0.8490304709, 0.8646885679, 0.892042378, 0.7823196834, 0.8475993917, 0.7, 0.7402515723, 0.7363986089, 0.8205394738, 0.8836984391, 0.8822846229, 0.876692038, 0.7750304046, 0.8002148737, 0.8734264322, 0.7671999385, 0.7828073806, 0.8963570128, 0.8671928881, 0.8635155832, 0.7686514886, 0.8677071987, 0.7488222753, 0.8428138036, 0.8058634696, 0.8112862122, 0.7748709559, 0.7746583383, 0.8792922001, 0.8384526576, 0.8835385065, 0.8613244699, 0.8028726194, 0.7720019807, 0.7701962931, 0.888825185, 0.870855588, 0.8070866142, 0.8766512591, 0.80550829, 0.8632814233, 0.7352941176, 0.8116976542, 0.8730203602, 0.8898534151, 0.7985593414, 0.8546375682, 0.7708149058, 0.8780157194, 0.8717952811, 0.7703654433, 0.8881807989, 0.8918367347, 0.790211589, 0.8365818314, 0.7705036372, 0.8170708691, 0.7388572024, 0.8297016548, 0.8164549703, 0.848549323, 0.6921505833, 0.8739800016, 0.8646585549, 0.7586190351, 0.8745018213, 0.860963554, 0.7949482272, 0.7615384615, 0.8843091123, 0.8647982132, 0.7714199468, 0.8668111861, 0.8714780637, 0.7619211634, 0.7742562832, 0.7878849417, 0.7908715861, 0.9043189645, 0.8211510791, 0.737037037, 0.7845703727, 0.7752633863, 0.8754991726, 0.8916835896, 0.7648927924, 0.8311356633, 0.8556406152, 0.8198033995, 0.8472110164, 0.8009342436, 0.846715919, 0.8273592387, 0.7792738879, 0.7730091321, 0.8721703424, 0.7912348295, 0.8877526385, 0.8315646774, 0.829325749, 0.8734741114, 0.8752842473, 0.8815778401, 0.8081339875, 0.8089271523, 0.8580502666, 0.8563521286, 0.7898876404, 0.8244368809, 0.7, 0.8312467567, 0.7711374815, 0.7733151539, 0.7788487402, 0.81441128, 0.8727087576, 0.8813826102, 0.856976591, 0.8923169974, 0.8696389773, 0.7793425357, 0.7633996937, 0.7714169514, 0.8697857948, 0.8867075697, 0.7701252573, 0.8726618705, 0.8027932961, 0.7511101042, 0.8637536239, 0.8467785146, 0.9024916275, 0.7899967202, 0.8061889251, 0.8520837879, 0.8156246129, 0.8361865167, 0.8216684046, 0.874691676, 0.8874967439, 0.6902159513, 0.788, 0.7, 0.8125420477, 0.6962092388, 0.7893544164, 0.8668204594, 0.7611462584, 0.8814650234, 0.782208589, 0.8782964535, 0.8494478834, 0.8058719987, 0.9003508365, 0.7669066401, 0.8968879545, 0.8007286979, 0.7742000011, 0.8459833171, 0.901241226, 0.825538244, 0.8363238388, 0.7761861577, 0.7887272727, 0.8230203853, 0.8651019863, 0.7507137106, 0.8253269779, 0.8472352946, 0.8833048136, 0.7581333333, 0.828340378, 0.8550450333, 0.850428178, 0.6165551479, 0.788, 0.8514165033, 0.8252225519, 0.7714470842, 0.7642991044, 0.8799141277, 0.7861233651, 0.8136956968, 0.871366754, 0.7811210454, 0.7544594885, 0.8727579231, 0.8706971263, 0.8365092723, 0.7566418553, 0.8710671264, 0.8439598742, 0.8601853895, 0.6446885615, 0.8260062648, 0.8442066768, 0.7543196347, 0.7702313647, 0.7755954474, 0.8368956743, 0.7915187377, 0.8706131725, 0.8839060701, 0.8916096214, 0.8894686365, 0.7982569923, 0.796627144, 0.7799070848, 0.8425962684, 0.769042985, 0.8610145736, 0.8410373219, 0.8440106927, 0.8750606532, 0.7658099298, 0.7815908929, 0.8418972211, 0.797283099, 0.7, 0.7772668022, 0.8909554449, 0.7541026998, 0.840407376, 0.8159423741, 0.8542380325, 0.7504263997, 0.8695963328, 0.8870444975, 0.7468196961, 0.874715395, 0.8781810808, 0.8813850805, 0.8647437218, 0.8160847334, 0.7150723835, 0.7773613821, 0.8734208316, 0.6433467162, 0.8477004307, 0.8852411647, 0.7178217822, 0.812589812, 0.8481794677, 0.8692625387, 0.8550632261, 0.8619098611, 0.7559670782, 0.8477851343, 0.7776568892, 0.8199523762, 0.8669194746, 0.6744292237, 0.8094998671, 0.8799269061, 0.8688643826, 0.8694051891, 0.8921716098, 0.8924802435, 0.8546905607, 0.7787098653, 0.8768872557, 0.7112676056, 0.8475993917, 0.8846817197, 0.7651936069, 0.8749159058, 0.7792738879, 0.8147164839, 0.7773546017, 0.8542744518, 0.7880715339, 0.8492264178, 0.826107993, 0.7687538748, 0.8023443037, 0.8923275904, 0.8687698052, 0.8233814433, 0.7815656162, 0.6846165313, 0.8961465565, 0.7971614276, 0.8061025224, 0.832042306, 0.880573213, 0.7760623787, 0.8495672105, 0.8149393948, 0.8821231699, 0.8114885586, 0.8160155209, 0.8973928683, 0.7716981132, 0.7815964523, 0.8511104207, 0.759652416, 0.7, 0.8102710621, 0.872588897, 0.8479405313, 0.8615610848, 0.8752944739, 0.8744938048, 0.875651187, 0.8523468198, 0.7615384615, 0.8873699822, 0.8364295223, 0.7898876404, 0.7954763139, 0.7352941176, 0.8027932961, 0.8067368594, 0.8757212703, 0.7721706884, 0.8316814852, 0.8045075724, 0.8697448032, 0.7691565843, 0.8048182743, 0.8530242121, 0.7594594595, 0.8856931794, 0.8927912186, 0.8369007003, 0.8917622523, 0.762281526, 0.8668464934, 0.8800566709, 0.7923446894, 0.7685128404, 0.7777860883, 0.8116710875, 0.7724341221, 0.8451820128, 0.8923419879, 0.7643031182, 0.8737683807, 0.8177317964, 0.8799698688, 0.8596103294, 0.8731244505, 0.766330699, 0.8163012393, 0.8913566682, 0.8736593169, 0.869394974, 0.8720304126, 0.8119043553, 0.7112676056, 0.7819101124, 0.8572216097, 0.7163380282, 0.8194930684, 0.8364341085, 0.8174904943, 0.8708441767, 0.8084574167, 0.7995705372, 0.8928182843, 0.884003115, 0.7, 0.7777702831, 0.8742617228, 0.8195824635, 0.8206714628, 0.72420278, 0.8172411814, 0.8673392538, 0.811747851, 0.7112676056, 0.8913230493, 0.7905294474, 0.8748207628, 0.8628643263, 0.8536025468, 0.7534611289, 0.7687971386, 0.7, 0.7376470588, 0.789762766, 0.8864846838, 0.8931963918, 0.7832758125, 0.8851056388, 0.8534798318, 0.8363954373, 0.8704234651, 0.7905008636, 0.852679934, 0.6007673154, 0.8266820975, 0.8054852321, 0.8735092616, 0.833546054, 0.762510184, 0.8759288531, 0.8035051546, 0.7687971386, 0.8015326993, 0.7778378378, 0.7799121844, 0.8252225519, 0.7649463417, 0.7800945999, 0.8101951785, 0.761690935, 0.8490039602, 0.8255573993, 0.7465116279, 0.8083541624, 0.8579846388, 0.8259681025, 0.6744292237, 0.8887936888, 0.8657809503, 0.7860173732, 0.8310993533, 0.8472648673, 0.858210449, 0.7742785778, 0.8372015847, 0.7542283512, 0.8556944032, 0.8139544513, 0.8662759117, 0.8738871095, 0.8709062082, 0.8726618705, 0.8866335267, 0.8209439528, 0.8457641371, 0.8471431803, 0.861331453, 0.8044776119, 0.8644451455, 0.7513086758, 0.798313253, 0.9041208288, 0.8149905123, 0.7433057851, 0.893964997, 0.8599026611, 0.7, 0.8723207133, 0.7, 0.8783340568, 0.8088538658, 0.8920564411, 0.892423994, 0.8768145366, 0.8812965467, 0.7, 0.8030807183, 0.771555729, 0.8775789467, 0.8261795407, 0.8580908316, 0.7633030052, 0.7934521769, 0.7120265521, 0.8157309054, 0.7744946548, 0.8573011253, 0.895175886, 0.8861461211, 0.873300783, 0.8209439528, 0.8171944692, 0.7467136667, 0.6884545001, 0.8208616747, 0.8065883888, 0.8571246365, 0.7759026975, 0.8142210887, 0.8413676206, 0.7524778299, 0.6420742465, 0.8384565075, 0.8674021839, 0.8004360432, 0.7207647611, 0.8399061573, 0.8632962489, 0.8811506814, 0.8589818182, 0.7025644328, 0.7702638255, 0.7465116279, 0.8602145632, 0.8688184052, 0.8119684846, 0.8471628531, 0.7855012428, 0.8511545872, 0.8201146679, 0.854395902, 0.7809497132, 0.8929667332, 0.7759102088, 0.8770798233, 0.8736040609, 0.8583155895, 0.8791096868, 0.8704348258, 0.8849174174, 0.8419327373, 0.8726167242, 0.8472605332, 0.7996336996, 0.8909423931, 0.8870806079, 0.8664507267, 0.871862069, 0.8786528235, 0.8123902302, 0.8252225519, 0.7861233651, 0.8827641018, 0.7812816283, 0.8723409894, 0.7643073481, 0.7925778132, 0.7758939322, 0.8293413174, 0.9059728386, 0.8669687113, 0.799473904, 0.8619588361, 0.8604240283, 0.7762335402, 0.8596773483, 0.8001222992, 0.8260105184, 0.8137749854, 0.8968306263, 0.9000860685, 0.88602278, 0.8699596197, 0.7593272171, 0.8974141982, 0.8763812441, 0.7610114192, 0.8033969477, 0.8540422677, 0.6656842441, 0.8370371889, 0.8203442505, 0.8720983368, 0.8666532852, 0.8925557693, 0.8017587306, 0.8030384633, 0.7942225382, 0.7664630655, 0.8135897222, 0.7577806391, 0.7654159891, 0.7560688051, 0.8872505641, 0.8929042931, 0.8731852409, 0.869799447, 0.7811111901, 0.8789548045, 0.6405814613, 0.8623586455, 0.873214986, 0.6923425239, 0.8715055553, 0.8679534898, 0.7503555746, 0.8687055749, 0.8717092704, 0.9014873838, 0.8688184052, 0.8691184168, 0.8800722767, 0.8714797492, 0.8645788071, 0.8768881862, 0.6580712788, 0.7760830325, 0.8404322476, 0.7742645268, 0.8792025346, 0.8791858111, 0.8756886119, 0.755203273, 0.7710871577, 0.7776234546, 0.860498089, 0.8565516164, 0.7276293264, 0.8345949394, 0.8572216097, 0.8592761382, 0.8717465078, 0.7102861883, 0.7907207332, 0.7, 0.8822631846, 0.7499252616, 0.8542892355, 0.8189716734, 0.8842922763, 0.8472795082, 0.8895241872, 0.8486510184, 0.708650157, 0.7112676056, 0.8311219512, 0.7706337419, 0.7578209277, 0.8422778588, 0.8718090325, 0.8616231127, 0.8417764269, 0.7502213042, 0.6866534194, 0.7, 0.8781169513, 0.8786658563, 0.8451218799, 0.8677246045, 0.8813748367, 0.7704055267, 0.7, 0.8864747517, 0.7800945999, 0.8271641791, 0.8607589996, 0.7720801603, 0.8645799958, 0.7, 0.8726134061, 0.813668467, 0.8576899375, 0.8328700817, 0.8116710875, 0.873130545, 0.8269828445, 0.7970537659, 0.8536472264, 0.8668971671, 0.9188435813, 0.8782445563, 0.7769169646, 0.8829611249, 0.7616216216, 0.8740937125, 0.8658776903, 0.784588373, 0.8679906023, 0.7, 0.8523308617, 0.8510045058, 0.8629188402, 0.8679624209, 0.8651677507, 0.8283616318, 0.9016341256, 0.7767647172, 0.8305055269, 0.680575423, 0.8445007748, 0.7568807339, 0.8201187513, 0.8836873683, 0.8314495392, 0.8975116333, 0.8749423757, 0.7941176471, 0.8500611995, 0.8761345828, 0.8081239767, 0.755373247, 0.7112676056, 0.8204678363, 0.7150723835, 0.7646180087, 0.8745711403, 0.8600793876, 0.7352941176, 0.8231193135, 0.8528189637, 0.8827077401, 0.8647128775, 0.8354522042, 0.8017128698, 0.8907463606, 0.8542784703, 0.8753122713, 0.849630274, 0.8767918347, 0.830634715, 0.8178056278, 0.8301717054, 0.8935760073, 0.7712200083, 0.7348304939, 0.7973155322, 0.7655612631, 0.8521480049, 0.8588863966, 0.7947151246, 0.8049254437, 0.8384317982, 0.8761617086, 0.8858407423, 0.6973193717, 0.8653484336, 0.8514491175, 0.8646300334, 0.825321461, 0.8652645776, 0.8758947067, 0.87230287, 0.7144911268, 0.8402566171, 0.8739617927, 0.8507306889, 0.7861952862, 0.8014013142, 0.7418106996, 0.689411848, 0.690947026, 0.8867804908, 0.788, 0.8816147703, 0.7688238612, 0.7607896368, 0.8640173319, 0.8997682848, 0.7749958739, 0.8467278033, 0.8637568966, 0.7839433128, 0.8281278068, 0.752675849, 0.7770939095, 0.8727579231, 0.7974223925, 0.8357281553, 0.829916479, 0.7827630799, 0.8650797582, 0.8875412716, 0.788, 0.7965212653, 0.8980184399, 0.821739651, 0.8872879724, 0.8661550165, 0.737037037, 0.8907600116, 0.813024491, 0.6918871507, 0.8747875452, 0.7144599821, 0.7647979994, 0.8961860999, 0.7966884586, 0.8538767749, 0.8545023697, 0.7541026998, 0.7822745417, 0.8593922044, 0.7814281445, 0.7603015075, 0.8879159455, 0.8116434991, 0.8705156775, 0.8196294456, 0.6879194631, 0.7717490174, 0.7821783557, 0.7814730469, 0.8161329305, 0.8836503021, 0.8550450333, 0.8702688774, 0.8926399664, 0.8531276956, 0.8249843195, 0.737037037, 0.8973369967, 0.8667483842, 0.8602045472, 0.896864169, 0.7630520285, 0.8155998103, 0.7768289777, 0.8480248952, 0.8076923077, 0.7616216216, 0.8407443444, 0.854835365, 0.8589207631, 0.8310811418, 0.8010404624, 0.7738529574, 0.7655930281, 0.86803097, 0.8850986093, 0.8814944683, 0.8181033491, 0.877807883, 0.7578209277, 0.7725978686, 0.7851430329, 0.8758327824, 0.7780884979, 0.8568754679, 0.8000308709, 0.8990164854, 0.8739508255, 0.8143702827, 0.8146599614, 0.7615384615, 0.8497316169, 0.7390438247, 0.8166332665, 0.7775899632, 0.8696273641, 0.8198033995, 0.8702348876, 0.7701962931, 0.88877347, 0.8796998192, 0.7862865948, 0.8682248315, 0.8668464934, 0.8720160704, 0.8540802213, 0.7458091111, 0.8480648401, 0.8973343725, 0.7875319209, 0.8597192833, 0.8949045423, 0.8715821688, 0.8390858607, 0.8612219585, 0.7917139614, 0.7533237341, 0.8894116802, 0.7499252616, 0.8884054391, 0.8451536643, 0.8780021765, 0.6933098558, 0.8012146409, 0.8520712864, 0.8985397643, 0.8037174446, 0.7692153754, 0.8854676896, 0.8888630275, 0.7925778132, 0.6853682638, 0.8644084118, 0.7181336119, 0.8672611504, 0.8672173286, 0.7, 0.8256807708, 0.8153496746, 0.8100708574, 0.7799121844, 0.7080376179, 0.8934816256, 0.7747157935, 0.8847176187, 0.8903248909, 0.8898464554, 0.8377772402, 0.7907936508, 0.7703522069, 0.8605493134, 0.8660805698, 0.783990385, 0.8137571157, 0.8788990738, 0.8898573796, 0.8450506793, 0.8842846505, 0.7834809395, 0.8745417385, 0.7615000959, 0.8089908257, 0.8926478998, 0.7928877691, 0.8796255986, 0.8884135068, 0.8353216873, 0.8970902867, 0.8827014448, 0.7840943207, 0.8532069039, 0.7578209277, 0.8052046517, 0.8618207239, 0.804390386, 0.8404306983, 0.8249843195, 0.8115354018, 0.7975042209, 0.857568534, 0.8546858013, 0.8351059478, 0.767084394, 0.7533960428, 0.7700866714, 0.752992728, 0.8641748078, 0.7632339483, 0.8051680272, 0.8062209625, 0.8202426773, 0.895650672, 0.822674906, 0.8665224952, 0.8013247925, 0.8537512686, 0.8865969898, 0.8580028264, 0.8744425124, 0.7810457516, 0.8748422661, 0.8522023182, 0.8525211326, 0.8671445799, 0.7804001819, 0.8314873648, 0.773638772, 0.7621819449, 0.7712621934, 0.7615384615, 0.8604263107, 0.8892022221, 0.8676297143, 0.8263009515, 0.8328237792, 0.8516459068, 0.8610475448, 0.8128205128, 0.8352191894, 0.820855615, 0.8035945503, 0.8765416845, 0.8708853952, 0.795483871, 0.8048049353, 0.7686640011, 0.7352941176, 0.6978742745, 0.7902214181, 0.893613801, 0.8558330768, 0.8436333962, 0.8664775862, 0.874936322, 0.8065233268, 0.7513715711, 0.7524778299, 0.8495505979, 0.8024632316, 0.8614543214, 0.8386701636, 0.8807875546, 0.8009678668, 0.9067992646, 0.8814019399, 0.7615384615, 0.782208589, 0.7716699411, 0.7744180141, 0.8460094259, 0.8862136598, 0.7534611289, 0.787785048, 0.8661412841, 0.8261795407, 0.8203771452, 0.6901204157, 0.8105133085, 0.8472122424, 0.8670782048, 0.757776825, 0.874513671, 0.7615384615, 0.7508966695, 0.8821941439, 0.8389344883, 0.6927645389, 0.7947151246, 0.7452034845, 0.8510614446, 0.7, 0.7973779773, 0.8602760025, 0.7861286392, 0.876119081, 0.8477127226, 0.7806932411, 0.8266820975, 0.8693736361, 0.7628185701, 0.8439157391, 0.7689343816, 0.8834546572, 0.8502445551, 0.8175056527, 0.866612873, 0.8689935116, 0.7954029885, 0.8558093565, 0.8138564274, 0.7503612335, 0.7235578862, 0.7965566973, 0.8710716408, 0.8131964085, 0.8625593672, 0.8740900585, 0.7766170266, 0.851377327, 0.7736921747, 0.8785120854, 0.7552388306, 0.7771094333, 0.7926517572, 0.7112676056, 0.8692857076, 0.680382991, 0.7587017825, 0.7524778299, 0.8915058118, 0.8266321783, 0.7750293083, 0.760169832, 0.8059480315, 0.778919022, 0.8935704806, 0.8547105386, 0.8505914978, 0.7917139614, 0.8504824277, 0.8874526424, 0.8811008306, 0.860837057, 0.8792117473, 0.8530487913, 0.7960818109, 0.7725958345, 0.8669112252, 0.7, 0.8734678934, 0.8209439528, 0.8885858025, 0.8585523481, 0.8605723507, 0.8696335746, 0.7650141024, 0.8638969101, 0.7617953603, 0.8643617855, 0.7558663276, 0.7713903633, 0.8110109831, 0.8712917744, 0.7615384615, 0.8908755429, 0.8473162593, 0.8759165482, 0.877480603, 0.8616065099, 0.8373948718, 0.7730461196, 0.8662658967, 0.8156708005, 0.821859545, 0.7720945891, 0.8756764754, 0.8541204557, 0.863692854, 0.7524124929, 0.8986082024, 0.8208542641, 0.8521017794, 0.7662312551, 0.8951672557, 0.8503814841, 0.889471066, 0.8509233798, 0.8953503666, 0.7449614215, 0.7666411922, 0.7569054021, 0.8293209877, 0.7112676056, 0.7871473354, 0.8778606012, 0.8867369174, 0.767935968, 0.8003347927, 0.8132728772, 0.7784527905, 0.8226431172, 0.8546375682, 0.8523622904, 0.8757446334, 0.8315826828, 0.7993277975, 0.7880402799, 0.7615384615, 0.7476724621, 0.696272081, 0.8177177892, 0.8184107555, 0.8129036, 0.8920711619, 0.8603119584, 0.8121184089, 0.7922651016, 0.7, 0.8840903771, 0.7961867687, 0.737037037, 0.8599109131, 0.7763886936, 0.8147562479, 0.8301415165, 0.9055712165, 0.7610114192, 0.8107499693, 0.8546375682, 0.7948344822, 0.8396323318, 0.8669553926, 0.7581333333, 0.8553109858, 0.7524778299, 0.7600652884, 0.8394816154, 0.8001222992, 0.8812132442, 0.7541026998, 0.6744292237, 0.7578209277, 0.8265839321, 0.7616216216, 0.8874934862, 0.8520097139, 0.7396060297, 0.7593272171, 0.8365940695, 0.8194477505, 0.773570652, 0.7861952862, 0.8349679324, 0.7285714286, 0.795483871, 0.862832109, 0.8438984765, 0.8738506543, 0.7682391283, 0.8626837477, 0.7887398016, 0.8765672044, 0.8980871974, 0.868104971, 0.8136743062, 0.8850321345, 0.7682144419, 0.7843667002, 0.8968337731, 0.865648089, 0.8588863966, 0.8854724718, 0.8859348309, 0.8513277566, 0.6963258673, 0.7619211634, 0.8960127733, 0.9005213636, 0.8834837298, 0.7, 0.7600652884, 0.8882239729, 0.7715601783, 0.8538221338, 0.7458654906, 0.813577794, 0.8825163129, 0.812639495, 0.8770578215, 0.8916686586, 0.8548916551, 0.8743109327, 0.8737947895, 0.8756641853, 0.866454511, 0.8545023697, 0.7568807339, 0.8279680605, 0.8515678866, 0.9002317996, 0.8564127847, 0.8562291418, 0.7670412801, 0.8784435372, 0.87621691, 0.7636241135, 0.7742452726, 0.88160588, 0.8663004552, 0.8323484551, 0.8592319055, 0.8852994819, 0.7811446114, 0.862041846, 0.7285714286, 0.8568973116, 0.8869027063, 0.8721983473, 0.8832483387, 0.7602959277, 0.8576703841, 0.6800391209, 0.882235587, 0.8453287197, 0.8633046472, 0.7901392758, 0.7505097582, 0.7919443831, 0.8232805914, 0.8232982386, 0.6806631205, 0.8781507214, 0.7578209277, 0.8009056612, 0.8023154366, 0.8122251123, 0.7437130802, 0.8269044948, 0.8485029761, 0.8473974705, 0.8893069498, 0.8008356546, 0.8181390977, 0.8864322097, 0.7692973918, 0.8643450314, 0.7786399449, 0.817791411, 0.8541144682, 0.8576316024, 0.8108577211, 0.7, 0.8859196485, 0.9001918307, 0.7598808879, 0.871959119, 0.866765019, 0.8845070879, 0.886187082, 0.8384565075, 0.869640269, 0.7178217822, 0.8065883888, 0.7967344953, 0.7, 0.7858211235, 0.8647374515, 0.8696242234, 0.7701581497, 0.8042637221, 0.8978936282, 0.8427578284, 0.7598808879, 0.8937010449, 0.8781619713, 0.8488389798, 0.8782621312, 0.8998942638, 0.8252225519, 0.746695652, 0.8404682072, 0.7813485409, 0.7615384615, 0.8471814197, 0.7225080386, 0.8746398235, 0.8096967902, 0.8641419459, 0.8539956936, 0.7839108607, 0.8317464058, 0.7914221725, 0.7499218555, 0.8254892145, 0.7525218686, 0.8511062491, 0.8663741257, 0.7619211634, 0.7136258948, 0.8206725296, 0.7898876404, 0.8237526313, 0.7778378378, 0.788, 0.8825606716, 0.8892840039, 0.8548111964, 0.7839779006, 0.7646243817, 0.879391807, 0.7562336224, 0.7417316017, 0.8550803884, 0.8096166818, 0.6876350776, 0.8249630099, 0.8245548654, 0.8714936488, 0.7878459909, 0.845229096, 0.8927271535, 0.8210691203, 0.8876739563, 0.6929618992, 0.8245158632, 0.8788899603, 0.7991466883, 0.8839983439, 0.8793602831, 0.824400818, 0.8707978984, 0.7396586904, 0.8601278154, 0.8285203115, 0.8181818182, 0.8889022903, 0.8626045692, 0.8403249631, 0.8703443233, 0.7143007361, 0.862660068, 0.8345949394, 0.8324883819, 0.8383800499, 0.806894834, 0.7809113493, 0.8835674796, 0.8696021429, 0.8333937498, 0.8205158825, 0.7846927374, 0.775862069, 0.8441378805, 0.8794608472, 0.7847473131, 0.8064733017, 0.8640396488, 0.8731057154, 0.8109479569, 0.8355585626, 0.8887622841, 0.8564479466, 0.8923841657, 0.7547851049, 0.8655598524, 0.6519313305, 0.8396257392, 0.8619549275, 0.7787510491, 0.6931276649, 0.6879194631, 0.8299387281, 0.7760840061, 0.8548989445, 0.8675980911, 0.8010610225, 0.8277734678, 0.8579729266, 0.8204678363, 0.775862069, 0.8199923902, 0.8436426117, 0.7077317323, 0.8833909364, 0.8520837879, 0.7772142005, 0.7722905682, 0.8304522915, 0.8909402596, 0.8195824635, 0.8070314071, 0.8008356546, 0.8514840874, 0.8874557936, 0.788, 0.7063448276, 0.8757523315, 0.873116612, 0.7615384615, 0.8524676569, 0.7716699411, 0.7777860883, 0.7444012945, 0.7875220966, 0.7915130793, 0.8644214319, 0.8760949325, 0.737037037, 0.8757109541, 0.8714724745, 0.8280011871, 0.8405557265, 0.7745585565, 0.8527273159, 0.8432739755, 0.8121387283, 0.8066344265, 0.8980229444, 0.8287618125, 0.8787339892, 0.7742893948, 0.741866579, 0.8735912077, 0.8373948718, 0.811037177, 0.8603837992, 0.8620171505, 0.708650157, 0.7683491703, 0.8442057589, 0.8102797609, 0.8663768116, 0.8056478974, 0.6872821085, 0.8444482951, 0.6879194631, 0.7842628313, 0.8034421027, 0.7891462939, 0.860749697, 0.8179982608, 0.7471363594, 0.8012490816, 0.7, 0.8871674491, 0.8868416799, 0.8074108674, 0.784362892, 0.7216508291, 0.8464485052, 0.8748868022, 0.8706458925, 0.8877822302, 0.8417764269, 0.7664630655, 0.8477354222, 0.8141770422, 0.8011823168, 0.8229009453, 0.8396558273, 0.7688714367, 0.830841234, 0.7553160607, 0.8515095372, 0.8295143213, 0.8555314088, 0.847411231, 0.8175768701, 0.8894587591, 0.8577056604, 0.8935188978, 0.8540920458, 0.6831243469, 0.775391237, 0.7874475265, 0.8523470641, 0.8141115799, 0.7704520331, 0.8719058161, 0.8828861494, 0.8863414634, 0.841290602, 0.8455355262, 0.8085617423, 0.8539459459, 0.866157913, 0.8370295184, 0.8497806036, 0.8057075841, 0.8787697949, 0.8654415681, 0.8110448459, 0.8678841135, 0.6275613912, 0.860457185, 0.7686546521, 0.8670982483, 0.8615874036, 0.8714431934, 0.8429457841, 0.8369403777, 0.8031313542, 0.8875206531, 0.8720206699, 0.8305866938, 0.7691047522, 0.8628371876, 0.8585224605, 0.844834167, 0.8060296981, 0.8083286332, 0.7759523534, 0.8608864902, 0.7534611289, 0.880563336, 0.8368224299, 0.7909090909, 0.7670087307, 0.7750293083, 0.8008322044, 0.7522561238, 0.7521174836, 0.8134202608, 0.8867108125, 0.8090181396, 0.8597960626, 0.7, 0.8104410825, 0.7991202346, 0.7719529535, 0.6879194631, 0.8206463196, 0.8589494316, 0.7842474869, 0.838477312, 0.8288052127, 0.8315219479, 0.8534501627, 0.8358941528, 0.8799531706, 0.8670113493, 0.8780702141, 0.8084735811, 0.7840888041, 0.8609341826, 0.8856093054, 0.8784784611, 0.8584555304, 0.7919990522, 0.8583235267, 0.8713414342, 0.8654419067, 0.8060094254, 0.869429128, 0.8562192201, 0.8738136828, 0.842443216, 0.8856548857, 0.7605405405, 0.7, 0.8502857696, 0.7957763598, 0.8763977008, 0.8794798149, 0.8135867807, 0.8216836202, 0.8853414373, 0.7809631985, 0.7800942285, 0.7933999691, 0.8131949592, 0.8034495481, 0.8655172414, 0.8454501801, 0.8391156463, 0.788, 0.8523715415, 0.7934084223, 0.8256698711, 0.7595914352, 0.6462489004, 0.766330699, 0.7954912879, 0.7805080562, 0.8293413174, 0.8935298841, 0.8014203292, 0.8739897623, 0.7839058507, 0.737037037, 0.7418069476, 0.8215830605, 0.7541026998, 0.7778378378, 0.837361342, 0.8890119656, 0.8779255128, 0.7632632633, 0.8701984826, 0.8077101155, 0.8925547707, 0.736370086, 0.8944977003, 0.8652187549, 0.6966979362, 0.8093346517, 0.7722852603, 0.7633996937, 0.8654179894, 0.7507137106, 0.7638471612, 0.8050123206, 0.8296569143, 0.7630232526, 0.7183584401, 0.8114692654, 0.8692415484, 0.8902375299, 0.8881645327, 0.8813751754, 0.8034421027, 0.775992605, 0.8484301328, 0.7, 0.855984556, 0.6519313305, 0.859621103, 0.8465359729, 0.8351059478, 0.8464763254, 0.7482598608, 0.8839535607, 0.8534391534, 0.8847830128, 0.877214776, 0.8372881356, 0.8902059348, 0.7858758522, 0.8683289057, 0.8669070028, 0.8681471908, 0.7702267087, 0.8843267738, 0.8524497742, 0.876940005, 0.705991342, 0.7711374815, 0.8498881432, 0.6568809135, 0.785263647, 0.8691538445, 0.8550065098, 0.8699462433, 0.8045196288, 0.8718862622, 0.8140968543, 0.8789744911, 0.8517696012, 0.8171380034, 0.8739395285, 0.8624683864, 0.8256807708, 0.8687518707, 0.8642059918, 0.7544502618, 0.8847819523, 0.7690909091, 0.8204678363, 0.7792970424, 0.8424714607, 0.7820450003, 0.7920941256, 0.8571490441, 0.7600652884, 0.8562034419, 0.8747297038, 0.8620613454, 0.8849701995, 0.8058400221, 0.7609070923, 0.8919632449, 0.776809694, 0.803523253, 0.8013098477, 0.8842947958, 0.8743031891, 0.719835624, 0.8608592306, 0.8639570868, 0.7826814366, 0.8052631579, 0.7629874287, 0.8649165995, 0.7845703727, 0.833334282, 0.8719708681, 0.86549975, 0.8614567765, 0.9013684221, 0.8348910364, 0.790250884, 0.8494877939, 0.8857612552, 0.8899858587, 0.8716238757, 0.8432739755, 0.8839241378, 0.7752556698, 0.8814751393, 0.8771927022, 0.7149340777, 0.7882073506, 0.8667391647, 0.6861175244, 0.8767791785, 0.8748545044, 0.7805411241, 0.8582581691, 0.8388347087, 0.7, 0.7, 0.8680545906, 0.8493969973, 0.8784035508, 0.6912221561, 0.8552909648, 0.8978715558, 0.8599403713, 0.6864424985, 0.8055311217, 0.6867159019, 0.7352941176, 0.7882882883, 0.8576891659, 0.7725783874, 0.7402515723, 0.8831575823, 0.8756003719, 0.788, 0.8435639413, 0.8781900174, 0.7485875706, 0.8732590182, 0.804771867, 0.8964125561, 0.8937087939, 0.8653298323, 0.8058157226, 0.7948871341, 0.8673621646, 0.8018525853, 0.8823714382, 0.8753152228, 0.8792983586, 0.8784425314, 0.8852303545, 0.8431691006, 0.8628845559, 0.8475993917, 0.8021993535, 0.785881295, 0.857663028, 0.7974843229, 0.8522096326, 0.861486049, 0.7731600718, 0.8316091352, 0.774611399, 0.8632982514, 0.8779008728, 0.8512749988, 0.8177037312, 0.8488021256, 0.8619625912, 0.7706196673, 0.7719653748, 0.7711722197, 0.8711810798, 0.7858358475, 0.7924854465, 0.8070668334, 0.8699250493, 0.7598808879, 0.7927880362, 0.8620027903, 0.8103856163, 0.8238589326, 0.7605405405, 0.8647723476, 0.8919711248, 0.8814752388, 0.8397740695, 0.8867049826, 0.8682750724, 0.8152234694, 0.8134249316, 0.870040628, 0.8716385768, 0.857191786, 0.7857938399, 0.8688997555, 0.8933274802, 0.8903784719, 0.8836925474, 0.7112676056, 0.7556614121, 0.8244701349, 0.8429970808, 0.6744292237, 0.8767954321, 0.8169590643, 0.8523645426, 0.7552784705, 0.8841287466, 0.8792389457, 0.8798027182, 0.8585456467, 0.88828257, 0.891534115, 0.8826477574, 0.7960552114, 0.8607084663, 0.737037037, 0.8029067717, 0.8672224781, 0.811747851, 0.7664630655, 0.7649602461, 0.8543008434, 0.8611180124, 0.8154532997, 0.8640912036, 0.8598611301, 0.8578767708, 0.7574397429, 0.7632676059, 0.8959522164, 0.8488750107, 0.8775007789, 0.8628660436, 0.8609583575, 0.858744804, 0.713487882, 0.8651937375, 0.772208011, 0.8026065178, 0.7484096199, 0.8765890537, 0.7898876404, 0.8591549296, 0.8093163128, 0.8587304571, 0.8545841286, 0.8114645461, 0.8446787013, 0.75574527, 0.7372844037, 0.8051360872, 0.8600858181, 0.7728066108, 0.863812737, 0.7600652884, 0.8751199739, 0.7861015265, 0.6686376928, 0.7535865757, 0.8275920045, 0.8592770309, 0.8765936767, 0.8451578616, 0.8691998997, 0.7163380282, 0.7687971386, 0.7875104296, 0.7712049651, 0.8668923346, 0.8274740603, 0.864230908, 0.812451835, 0.8435556769, 0.8842474748, 0.8597959658, 0.8120548423, 0.8010404624, 0.8912674308, 0.7872868217, 0.8232035114, 0.8772420212, 0.8586312862, 0.875424685, 0.7602460005, 0.8810378746, 0.8391750607, 0.8367723224, 0.737848726, 0.880848923, 0.7849215428, 0.8589539712, 0.8155538623, 0.7551257402, 0.8989224024, 0.87790353, 0.886588977, 0.8885486994, 0.8309445286, 0.8679397825, 0.690260777, 0.850484836, 0.8673957398, 0.8391120379, 0.6865746723, 0.868591692, 0.8517714111, 0.80694729, 0.7834191655, 0.797484506, 0.7804999193, 0.8743512173, 0.7112676056, 0.7, 0.8938202522, 0.7618602914, 0.8512570173, 0.829238536, 0.8362180022, 0.8164618726, 0.8172283551, 0.8127417653, 0.856927616, 0.893801043, 0.8057757118, 0.7112676056, 0.8023510115, 0.7907936508, 0.8722358951, 0.8008356546, 0.8097598797, 0.8347575449, 0.8432865671, 0.8921933693, 0.8606603377, 0.8665760936, 0.8457641371, 0.8277118297, 0.8837357793, 0.8675152749, 0.8559042908, 0.8846041251, 0.8663741257, 0.8795980658, 0.8613065327, 0.8475537117, 0.8924546902, 0.8655598524, 0.8496593392, 0.7692892489, 0.8845072733, 0.8574044629, 0.8706231265, 0.8505690136, 0.8972530375, 0.808277037, 0.774084757, 0.8334734401, 0.8502855934, 0.7812216888, 0.9202354222, 0.7654792594, 0.7808748857, 0.6961589404, 0.8760169247, 0.8469263506, 0.8651059815, 0.9180067144, 0.8898615103, 0.7901392758, 0.8431606906, 0.7719653748, 0.8785294315, 0.8693773904, 0.7643766018, 0.8039839034, 0.77680681, 0.8433550663, 0.8498580716, 0.8973475499, 0.8551296071, 0.7855692281, 0.7618602914, 0.7721249555, 0.8716036772, 0.7643113154, 0.8866402853, 0.8195146613, 0.7705891485, 0.8837333376, 0.7738954714, 0.9023968396, 0.899090813, 0.8839186542, 0.7352941176, 0.8788679489, 0.7643382516, 0.8271825162, 0.8132640029, 0.8861132262, 0.9039513577, 0.8636858744, 0.8854384891, 0.7780115937, 0.8538304454, 0.7957114935, 0.7638511206, 0.8100671915, 0.8965066595, 0.7542774769, 0.7906597438, 0.6879194631, 0.8514160829, 0.7701577972, 0.825866865, 0.7646859722, 0.8609144772, 0.7779486022, 0.8790444003, 0.7594594595, 0.8556163447, 0.8320909637, 0.863692854, 0.8687335469, 0.8150977315, 0.8761889563, 0.8033490607, 0.7614330125, 0.8547643132, 0.860402579, 0.7, 0.7937623686, 0.8106482139, 0.8456134731, 0.7693797691, 0.7763827967, 0.775862069, 0.7, 0.8345254517, 0.8549238969, 0.778060072, 0.6856716418, 0.7880503145, 0.7882882883, 0.7328243503, 0.8085529005, 0.7137906948, 0.8676023355, 0.8328031323, 0.7709354242, 0.8878256333, 0.7715479602, 0.8745661787, 0.8957118924, 0.6879194631, 0.8427584718, 0.8186829038, 0.8688677203, 0.8839856454, 0.882963786, 0.8039839034, 0.745938464, 0.8823899777, 0.7514235376, 0.7541026998, 0.7815777167, 0.788381444, 0.895334319, 0.8771457869, 0.8524685412, 0.788, 0.8639342682, 0.8756877045, 0.8600964755, 0.6906071019, 0.8245630759, 0.8310038558, 0.7533821927, 0.8726618705, 0.8719483006, 0.737037037, 0.8327993511, 0.7879103597, 0.8037011254, 0.8808034082, 0.9012000862, 0.8252225519, 0.8042647402, 0.7656401208, 0.8788189461, 0.8855576188, 0.8246427576, 0.8108925606, 0.7203420342, 0.8558330768, 0.8558708809, 0.8921765525, 0.8794418791, 0.8824284253, 0.8737323652, 0.7615384615, 0.7352941176, 0.6879194631, 0.7699453258, 0.7579945799, 0.8722493157, 0.754675232, 0.8580961147, 0.8581296461, 0.7782015615, 0.7537954497, 0.7845703727, 0.8564600678, 0.6912560103, 0.8253124092, 0.7996512854, 0.7815964523, 0.8295619062, 0.8499113032, 0.8818723667, 0.7428885748, 0.8172477064, 0.8956538961, 0.8601811628, 0.8488750107, 0.8208597198, 0.821703854, 0.8713032032, 0.8917825275, 0.8265480721, 0.8151537993, 0.7960552114, 0.8845244457, 0.8014753047, 0.8428290766, 0.8119808548, 0.7720840729, 0.7984936734, 0.8231456085, 0.8355153413, 0.8906436564, 0.6930097495, 0.854854746, 0.8870391192, 0.8838328558, 0.6879194631, 0.8917767796, 0.8849145139, 0.8681288451, 0.869234964, 0.8255619414, 0.8147693309, 0.7971135173, 0.7849761685, 0.761586895, 0.8281731169, 0.8502167034, 0.8571510487, 0.7581333333, 0.849258541, 0.8681104331, 0.8683611173, 0.7778378378, 0.7974223925, 0.8341651637, 0.7590689538, 0.8314885635, 0.7761902121, 0.7718267856, 0.7832899959, 0.8857551339, 0.8283557995, 0.8770392749, 0.8740512821, 0.7579945799, 0.8161301807, 0.8647720175, 0.8058004964, 0.8358941528, 0.8749162492, 0.8612969191, 0.8108711839, 0.8465137197, 0.8639858341, 0.7231845875, 0.8026417732, 0.8919891431, 0.8460637558, 0.8367415479, 0.8405713491, 0.7085791397, 0.8600858181, 0.8768390925, 0.7414117297, 0.895203474, 0.7, 0.8785604824, 0.7524281561, 0.8698974312, 0.8293413174, 0.874535647, 0.8598987385, 0.8506316697, 0.8529592584, 0.7881975182, 0.8665478867, 0.7, 0.844284624, 0.7812905742, 0.8725412442, 0.8010094275, 0.7827947051, 0.8234477149, 0.817637793, 0.7611462584, 0.7520547945, 0.7740740741, 0.8833846553, 0.8322341697, 0.7830449827, 0.8903916449, 0.8942853394, 0.8868274098, 0.8444975492, 0.8696683372, 0.7799121844, 0.8686093819, 0.7157197673, 0.8382096363, 0.8234148363, 0.7, 0.8622159091, 0.8946766082, 0.8062051441, 0.7873880659, 0.8814253112, 0.8672684691, 0.7, 0.8546858013, 0.8809184763, 0.8106287425, 0.7637718032, 0.8402044345, 0.8103448276, 0.7, 0.7516239316, 0.847330709, 0.8627370058, 0.8740653591, 0.7723551805, 0.8923595756, 0.8550642314, 0.7847131305, 0.887990236, 0.6653047168, 0.8950590537, 0.8860203268, 0.8258613933, 0.8790056798, 0.8433927874, 0.8679766372, 0.8371337994, 0.8941335154, 0.8689865308, 0.8204678363, 0.8971411389, 0.8655218343, 0.6903646596, 0.790570695, 0.8561792526, 0.7926517572, 0.8896923705, 0.8361865167, 0.8060779413, 0.8506958913, 0.7812759206, 0.8681733284, 0.7749249538, 0.8674860585, 0.8103448276, 0.8794609246, 0.8012820513, 0.7478151815, 0.866769462, 0.7585052125, 0.897327812, 0.8271641791, 0.8243333333, 0.8701921713, 0.7950853246, 0.8169729221, 0.7862706078, 0.8810041351, 0.7745490135, 0.7670118859, 0.88201634, 0.8172346177, 0.7831804281, 0.8528094542, 0.7871473354, 0.8330365043, 0.8299532802, 0.8975750008, 0.8865870382, 0.8593583408, 0.7947368421, 0.7918767507, 0.868246968, 0.8787618859, 0.8999867594, 0.9016142029, 0.854539573, 0.8465138549, 0.8665623051, 0.8707107843, 0.8750059336, 0.8732202363, 0.8901158049, 0.7703257859, 0.6956349206, 0.817235519, 0.7772169938, 0.8401546157, 0.8594646738, 0.8414981561, 0.8705039415, 0.8840588328, 0.8027932961, 0.8813402062, 0.8928067932, 0.7682592213, 0.7397586601, 0.7639433097, 0.8102965238, 0.8058881799, 0.7982569923, 0.682122905, 0.8120278908, 0.7692170819, 0.8971189282, 0.8620638821, 0.8351658095, 0.796, 0.8600858181, 0.8789376059, 0.7593272171, 0.8708247416, 0.8784772245, 0.7908614781, 0.775862069, 0.882006534, 0.6173955632, 0.8951264428, 0.8324883819, 0.8672074266, 0.8666469329, 0.864998415, 0.7778378378, 0.8648806533, 0.8492407809, 0.7926517572, 0.8295153594, 0.7335393657, 0.8685549852, 0.9205084573, 0.8182616984, 0.7713627648, 0.8040353529, 0.7372844037, 0.8681674033, 0.7352941176, 0.9051493855, 0.8580441154, 0.8432739755, 0.7490424492, 0.8405707907, 0.7841676943, 0.8356082329, 0.8575731122, 0.8495117217, 0.825918534, 0.7487901298, 0.8866929634, 0.7414865272, 0.8680285464, 0.8396083386, 0.8765848778, 0.8283557995, 0.6757111647, 0.8128105384, 0.7, 0.8567488978, 0.8238021219, 0.7627211246, 0.8860222266, 0.7815964523, 0.8607567201, 0.8303313144, 0.8747782011, 0.8335909193, 0.8864391324, 0.881770315, 0.8024549918, 0.7690429835, 0.8956306369, 0.870788415, 0.8138564274, 0.7112676056, 0.818243283, 0.7612521553, 0.7, 0.8850677161, 0.7763745844, 0.8581782709, 0.8057399363, 0.8690224445, 0.8681009739, 0.8146722164, 0.8495614777, 0.7, 0.8860191496, 0.8607502328, 0.8810874429, 0.690537811, 0.777238081, 0.8716659192, 0.8713791099, 0.7945032286, 0.8874706883, 0.8431944103, 0.8086523335, 0.7796833625, 0.8663130377, 0.7769581269, 0.8903348766, 0.7755554317, 0.7285714286, 0.7748023005, 0.8200699657, 0.7840621139, 0.8204678363, 0.7648143965, 0.8024549918, 0.8848707315, 0.737037037, 0.8603741172, 0.7704431645, 0.8219501121, 0.8809226893, 0.8734267135, 0.8744113179, 0.8210714136, 0.7817421435, 0.8171033826, 0.8233154639, 0.8507645351, 0.7953311618, 0.7549787161, 0.7438001547, 0.7615000959, 0.8722988887, 0.8347820924, 0.8356709854, 0.7352941176, 0.8914236391, 0.8840147377, 0.8069112199, 0.83312931, 0.8542137611, 0.7919990522, 0.8149915778, 0.8011881591, 0.8217730958, 0.8840227968, 0.7841334738, 0.7893185114, 0.8246427576, 0.7751571581, 0.7914272853, 0.8012658228, 0.8420689655, 0.8914430548, 0.7698384914, 0.775862069, 0.8398564115, 0.8090447707, 0.8779204108, 0.8269297115, 0.8076337385, 0.8961737482, 0.8861718313, 0.659660178, 0.8173020694, 0.8837255659, 0.8681460817, 0.7719828665, 0.7541026998, 0.7748861887, 0.8530054645, 0.7741229128, 0.8733073526, 0.8436426117, 0.8275113522, 0.7548292631, 0.7768913534, 0.861075012, 0.8603364899, 0.7927404788, 0.860140916, 0.8686817664, 0.8602502546, 0.7774214337, 0.890852242, 0.8694786421, 0.827176134, 0.8674034383, 0.8631870638, 0.8858312394, 0.8600505689, 0.8876846494, 0.714010582, 0.8588327254, 0.7598808879, 0.8470307759, 0.8337310421, 0.8759812861, 0.8651019863, 0.7342871178, 0.7611462584, 0.7247622942, 0.8441943642, 0.7739241108, 0.827741873, 0.8518830017, 0.8721763397, 0.8848339729, 0.8728437405, 0.824681564, 0.7950792612, 0.7760219713, 0.8720376797, 0.8937778532, 0.8846308616, 0.8797665024, 0.8197175033, 0.8730831714, 0.8767287142, 0.8198996276, 0.8538791256, 0.8866859923, 0.7736974262, 0.8638967715, 0.8000622167, 0.770269251, 0.7862865948, 0.8050443895, 0.7607997465, 0.8213203902, 0.8530340405, 0.7738798505, 0.8922515043, 0.827176134, 0.8169271593, 0.7429040733, 0.7995978476, 0.863692854, 0.8614391521, 0.7998643148, 0.8009961955, 0.782208589, 0.7240134907, 0.7, 0.811747851, 0.8046181222, 0.8612676531, 0.8138201605, 0.872987836, 0.8942678259, 0.8550642314, 0.8033261437, 0.804791199, 0.7690514987, 0.796931856, 0.8702455615, 0.7821060283, 0.8709172442, 0.8756781323, 0.690023742, 0.8472168824, 0.8612944278, 0.6879194631, 0.8397260274, 0.7628526023, 0.8995773764, 0.8179347044, 0.7648057437, 0.8820678566, 0.7882882883, 0.7949482272, 0.7649064331, 0.8285604186, 0.8885244565, 0.8579602791, 0.7, 0.8690070036, 0.8057225196, 0.8568214167, 0.8932790759, 0.7769216158, 0.8911398186, 0.8012842704, 0.662121909, 0.8960004612, 0.8830195072, 0.7235962763, 0.8642723047, 0.8519960327, 0.8773164557, 0.7662671944, 0.7928955667, 0.7499060027, 0.83772927, 0.8343383585, 0.8463226671, 0.8110871075, 0.7533962889, 0.8277955272, 0.8620270973, 0.8365882632, 0.763923144, 0.8605928865, 0.778321795, 0.8221544113, 0.8358578719, 0.7935303406, 0.8184004527, 0.6177226814, 0.7387237354, 0.7616216216, 0.807518797, 0.8857573521, 0.8280279503, 0.8125335519, 0.8733753296, 0.7718203937, 0.7604554249, 0.7440033085, 0.7839779006, 0.882931052, 0.8472168824, 0.8300666525, 0.8359566508, 0.7932680647, 0.746711381, 0.7, 0.8913831904, 0.8553311166, 0.7996521894, 0.8546096376, 0.7685342209, 0.7845468117, 0.8562808515, 0.8648148148, 0.7156045265, 0.8324883819, 0.7830449827, 0.7786335786, 0.8336082474, 0.8531005957, 0.8439684173, 0.7752040374, 0.7692910134, 0.7930444698, 0.7, 0.8782263625, 0.8448565595, 0.8925470724, 0.7598808879, 0.8194351042, 0.8791977685, 0.8631697478, 0.7876454294, 0.8815141695, 0.8816942301, 0.7771498327, 0.8587601671, 0.8508601743, 0.8730709483, 0.7287179487, 0.8523387758, 0.8550642314, 0.7792883813, 0.7, 0.8863876559, 0.9046558873, 0.8790397074, 0.8908498604, 0.8243333333, 0.6912739801, 0.8078050526, 0.7806858959, 0.8969354274, 0.817791411, 0.7, 0.8045101801, 0.8762258619, 0.8563876852, 0.8390858607, 0.8406042979, 0.8460543552, 0.7522491213, 0.7651563208, 0.8675428005, 0.7686198997, 0.8758479291, 0.7856790647, 0.7592797201, 0.8427830724, 0.8728036669, 0.8664951956, 0.719835624, 0.7581333333, 0.8474785918, 0.7, 0.8520510931, 0.8469616889, 0.8717019588, 0.8766971452, 0.6919057295, 0.7781634572, 0.8587185847, 0.8350893921, 0.8781218295, 0.7, 0.7930444698, 0.6955763016, 0.8566767037, 0.8053869833, 0.775862069, 0.8731479443, 0.7869040974, 0.7456858731, 0.8772677157, 0.8437735849, 0.8675952249, 0.8846990221, 0.821703854, 0.8711734856, 0.7692262717, 0.9007587879, 0.8818742433, 0.8260350354, 0.8856011669, 0.8849546044, 0.8312652471, 0.737037037, 0.7682098765, 0.8411186232, 0.8810556784, 0.8581400414, 0.8793388693, 0.8839223258, 0.8353138491, 0.7765063105, 0.8129036, 0.8403107403, 0.8076923077, 0.8293413174, 0.8543594605, 0.8531101348, 0.8439598742, 0.8008356546, 0.7713388348, 0.8509032188, 0.8155937839, 0.7783219942, 0.8024549918, 0.836017982, 0.790070922, 0.6789268293, 0.7765884526, 0.7222877925, 0.7079021637, 0.8664755733, 0.8595173679, 0.7578209277, 0.8842902787, 0.8809037595, 0.8104935711, 0.7482598608, 0.7453333333, 0.8489536961, 0.7568807339, 0.7590509882, 0.8857056881, 0.7563100525, 0.7459194093, 0.7786124025, 0.7826703301, 0.8410676274, 0.8260335785, 0.833800206, 0.7740712181, 0.6553113553, 0.7219071341, 0.8032114649, 0.8119299526, 0.8638333735, 0.7579945799, 0.842745098, 0.8776828671, 0.7735508911, 0.8414536694, 0.8629725717, 0.8061025224, 0.8448427459, 0.7605405405, 0.6884545001, 0.8491080484, 0.8683514465, 0.8330126778, 0.8369403777, 0.8522023182, 0.7615384615, 0.7, 0.776759397, 0.6758547903, 0.7687702896, 0.7505097582, 0.7930444698, 0.8770633297, 0.7799070848, 0.8832512468, 0.8157778901, 0.6714028342, 0.8389439962, 0.8237830626, 0.7802091781, 0.8918840238, 0.8671138759, 0.8083411434, 0.7709290643, 0.8931378669, 0.8939359159, 0.6955349007, 0.7589201137, 0.8131110203, 0.7690984171, 0.7751272641, 0.801466656, 0.8941502807, 0.8799493018, 0.7, 0.7596381084, 0.8700334614, 0.737037037, 0.8008322044, 0.6884545001, 0.6921505833, 0.8807329263, 0.7921212121, 0.8308138684, 0.8007013692, 0.8764921367, 0.8547654321, 0.8600803541, 0.8037746884, 0.8293413174, 0.6959910913, 0.784588373, 0.8019426457, 0.776576974, 0.7930185497, 0.811747851, 0.8035466794, 0.7460022635, 0.7501857202, 0.7948871341, 0.8136359485, 0.8908134395, 0.7548191546, 0.8776159043, 0.7964401294, 0.8599125112, 0.7874859076, 0.7553160607, 0.7453750233, 0.8874983299, 0.8601853895, 0.8836299207, 0.7615384615, 0.8651436283, 0.7183308931, 0.8771967424, 0.8635301916, 0.7499252616, 0.809178744, 0.7775899632, 0.7628780582, 0.8609073433, 0.8615158892, 0.7917139614, 0.8689030076, 0.6953847867, 0.7112676056, 0.8670619031, 0.7763100909, 0.8925928823, 0.8004190199, 0.8531295171, 0.7, 0.718238691, 0.8412459174, 0.8568063168, 0.8238080986, 0.8442057589, 0.8733006463, 0.7464315352, 0.8386256936, 0.7, 0.8916471643, 0.8603165015, 0.8102674397, 0.896700707, 0.8712779312, 0.795608923, 0.7659620989, 0.893942361, 0.8774451098, 0.8836987345, 0.861114349, 0.7423157446, 0.8665224952, 0.8513877208, 0.7238476514, 0.8902250846, 0.8785796305, 0.7546201232, 0.8227155611, 0.8638055843, 0.8672583388, 0.8000937793, 0.8514529915, 0.8640880012, 0.8813481427, 0.7, 0.8616365067, 0.8769960349, 0.7502213042, 0.7710672301, 0.8714402868, 0.8709605718, 0.7772276149, 0.8442025917, 0.7671999385, 0.8375058824, 0.8790251371, 0.8651019863, 0.7352941176, 0.8442025917, 0.8253660027, 0.7688071609, 0.860775053, 0.8279959303, 0.8654288817, 0.7372844037, 0.7544134031, 0.8631657541, 0.866122561, 0.9007072159, 0.7571900578, 0.8835305058, 0.86064542, 0.8836692152, 0.8433946131, 0.8272502502, 0.81, 0.7377685009, 0.6162671001, 0.8778538162, 0.8709947564, 0.8010969183, 0.8805584281, 0.85025891, 0.8932759969, 0.7112676056, 0.8216684046, 0.7, 0.8184260384, 0.8305673158, 0.9025021985, 0.7720521645, 0.8775894777, 0.8234148363, 0.8554260117, 0.8014685648, 0.7598808879, 0.8639811327, 0.7285714286, 0.8322931442, 0.854943535, 0.7766497462, 0.7861380092, 0.8250127448, 0.87217566, 0.8223013277, 0.7646655232, 0.8123222749, 0.8648240542, 0.8858436932, 0.8553357293, 0.7726534554, 0.6870449704, 0.8052741032, 0.8161906357, 0.8003490598, 0.8249089431, 0.6793939394, 0.8602705721, 0.8085171425, 0.771067823, 0.8523622904, 0.8044933051, 0.8810592345, 0.800727055, 0.8585334297, 0.7832392384, 0.8108522773, 0.8573855724, 0.8576921355, 0.8540082483, 0.8641407687, 0.8450506793, 0.8699599992, 0.8424714607, 0.8259708523, 0.7191459409, 0.8205309739, 0.8851921999, 0.7659547419, 0.8576375586, 0.6856716418, 0.7815964523, 0.8929534248, 0.8742595312, 0.7758962796, 0.7448736999, 0.8939891484, 0.8665026951, 0.788, 0.7600652884, 0.8921363041, 0.8684365828, 0.7584969109, 0.8737075473, 0.6879194631, 0.8701422477, 0.8293413174, 0.8827846029, 0.85931844, 0.8765812541, 0.8894443905, 0.8698130812, 0.8471628531, 0.8774768783, 0.8518892863, 0.870499699, 0.8665318074, 0.8689939255, 0.869207884, 0.8735862103, 0.8860113914, 0.8908948195, 0.8450382599, 0.8880578707, 0.7716808256, 0.864836338, 0.8245158632, 0.6519313305, 0.7352941176, 0.8578083544, 0.8025910491, 0.9177134529, 0.852895885, 0.8892044618, 0.8915683564, 0.8738729508, 0.8626828038, 0.8271641791, 0.8283557995, 0.7544502618, 0.7487011283, 0.8593774564, 0.8785276509, 0.7583641446, 0.8847845065, 0.7721146676, 0.8451439727, 0.8247049078, 0.7908657665, 0.7396586904, 0.7450105579, 0.6879194631, 0.8886563177, 0.7829838954, 0.7464702456, 0.8844271672, 0.858813664, 0.7, 0.8990468626, 0.8009678668, 0.8254314123, 0.8392975238, 0.870362821, 0.8767455932, 0.7598913725, 0.7, 0.8722625924, 0.8704066265, 0.8480175035, 0.8678358387, 0.7784433422, 0.8008322044, 0.837396392, 0.7453333333, 0.8256301237, 0.8529076653, 0.8061025224, 0.9199476315, 0.8768349098, 0.8136885854, 0.7581333333, 0.7821906761, 0.7, 0.8057225196, 0.8611772469, 0.866754935, 0.9046189223, 0.7274822035, 0.8146200863, 0.7541026998, 0.8608599194, 0.8951555199, 0.8169060359, 0.7615384615, 0.7799638173, 0.7780487805, 0.7695037724, 0.768121564, 0.8829089144, 0.8014753047, 0.7954885966, 0.8024639702, 0.8703589275, 0.6958416174, 0.8531755915, 0.7629303888, 0.8706009097, 0.7616216216, 0.8930384099, 0.8716239862, 0.7145462486, 0.8670906754, 0.8687701842, 0.869802519, 0.8968924468, 0.761595824, 0.7466834593, 0.8959102846, 0.8447367609, 0.5877778343, 0.7558663276, 0.7, 0.7, 0.8006499536, 0.7587545114, 0.7, 0.8641746854, 0.8897154611, 0.8567161444, 0.7, 0.5655993297, 0.8241748833, 0.8806203538, 0.852895885, 0.87576184, 0.8547949848, 0.8321000758, 0.8392325881, 0.799378882, 0.7598808879, 0.788, 0.8850257668, 0.7267616785, 0.8700303002, 0.7648143965, 0.7, 0.8434412061, 0.8057555054, 0.6744292237, 0.8560357745, 0.7624558101, 0.83475598, 0.7487839492, 0.809643961, 0.8090705948, 0.6806525704, 0.7102861883, 0.8657427876, 0.7399500624, 0.8216326531, 0.8679094734, 0.843633728, 0.8837862348, 0.6897792564, 0.7891690915, 0.7739881803, 0.844714909, 0.8259367946, 0.8115671642, 0.7602045424, 0.8010404624, 0.8639381224, 0.8728192087, 0.823694404, 0.7903636364, 0.8661661933, 0.854890154, 0.867289575, 0.8451496636, 0.8214612363, 0.7666411922, 0.6804800473, 0.8169802006, 0.8932620985, 0.7595204659, 0.8251996167, 0.8281491119, 0.7769169646, 0.788, 0.8715566141, 0.7754266117, 0.7690894881, 0.8457641371, 0.7672752327, 0.8154804986, 0.8457500109, 0.7810174725, 0.7713475294, 0.7764482219, 0.8788479004, 0.788, 0.8131571987, 0.8341239186, 0.7748891509, 0.7826937809, 0.8596249726, 0.6165551479, 0.7565655414, 0.769189069, 0.8811728395, 0.8067368594, 0.8600258907, 0.891188251, 0.8833165829, 0.8601354171, 0.8911105314, 0.8691781886, 0.8252225519, 0.7570118115, 0.8592241054, 0.7608298439, 0.7917139614, 0.8476440405, 0.7770744636, 0.8166255022, 0.7220927152, 0.6811420109, 0.7713145413, 0.8429113324, 0.8698392563, 0.851859664, 0.788, 0.8869178845, 0.8695118867, 0.8694355093, 0.7578209277, 0.6264632198, 0.7912916946, 0.8216634648, 0.8189089465, 0.7811885124, 0.8367946333, 0.8271641791, 0.7508511807, 0.8267455986, 0.8592136897, 0.7702305576, 0.8556163447, 0.8735304207, 0.8807352173, 0.8199056988, 0.7879994474, 0.7902587598, 0.8563404464, 0.8493791088, 0.860439285, 0.9030897334, 0.8625874126, 0.8312702079, 0.832923368, 0.8837358289, 0.833615939, 0.8631491843, 0.8993847477, 0.7999016716, 0.7474601331, 0.8697147107, 0.8523635912, 0.8538449648, 0.8110871075, 0.8299387281, 0.75949271, 0.8455078125, 0.7579766854, 0.8764580124, 0.8898851719, 0.7978298375, 0.8696049989, 0.8658876306, 0.8245022349, 0.879051695, 0.883010823, 0.7945902227, 0.8590792041, 0.7565701559, 0.8345597167, 0.7788554516, 0.853686769, 0.8188369678, 0.8654079874, 0.7629660946, 0.838549636, 0.8287974779, 0.8417764269, 0.7886827668, 0.8767906236, 0.7524778299, 0.8551608243, 0.7777276826, 0.7285714286, 0.7717110278, 0.8024253731, 0.8273256159, 0.8235582564, 0.7793716647, 0.8391156463, 0.7727930394, 0.8920293655, 0.8062477408, 0.8164605138, 0.7308817749, 0.8172056414, 0.8620557081, 0.799776686, 0.804001815, 0.8888283423, 0.8620880827, 0.8375167046, 0.835253716, 0.7501271833, 0.7, 0.7, 0.8738343545, 0.8234440691, 0.8755416007, 0.8793005067, 0.8542596649, 0.7727735918, 0.7858358475, 0.8686520921, 0.7768289777, 0.8118567871, 0.8363258758, 0.7960552114, 0.8363924579, 0.8324354827, 0.7719792263, 0.902704948, 0.8419327373, 0.8644185637, 0.8434897212, 0.7785280964, 0.8621598717, 0.7967654377, 0.7925778132, 0.8935298479, 0.8706376287, 0.8059099253, 0.785106383, 0.8771216424, 0.9010615711, 0.8208013058, 0.7556824401, 0.7629352138, 0.7615384615, 0.8593254468, 0.7824851432, 0.7498978434, 0.859431049, 0.776278622, 0.7478182779, 0.8494672755, 0.7287179487, 0.8862701267, 0.8493078539, 0.7806258788, 0.8208622754, 0.8054661665, 0.6347387718, 0.7697508897, 0.7374145891, 0.8748639056, 0.8052631579, 0.8363610761, 0.8796698442, 0.7352941176, 0.8005005776, 0.8711552621, 0.7448736999, 0.7749806352, 0.7285714286, 0.8515032753, 0.6624960737, 0.849844209, 0.7638972049, 0.8668464934, 0.6519313305, 0.8914609827, 0.8550317618, 0.756675337, 0.7778378378, 0.8706566133, 0.8480294577, 0.7568807339, 0.7979778885, 0.8346734944, 0.8453508746, 0.7930444698, 0.8973082365, 0.839092375, 0.8069392983, 0.8225904044, 0.8523695905, 0.6879194631, 0.7700324224, 0.754264704, 0.7615384615, 0.8460839463, 0.788, 0.7898876404, 0.8162957901, 0.7524658583, 0.8670066395, 0.7, 0.8434070065, 0.8496593392, 0.8055114462, 0.7287179487, 0.8708368956, 0.8114614102, 0.721956968, 0.8811318353, 0.7629285569, 0.7287179487, 0.7767089644, 0.8075485602, 0.8906987091, 0.8814880479, 0.8887548523, 0.8139051772, 0.7784433422, 0.7839779006, 0.8298998403, 0.7935153584, 0.8845295625, 0.878402307, 0.8465994517, 0.8444205663, 0.8715656838, 0.6929224593, 0.8648913117, 0.7630544168, 0.8656373809, 0.7428631579, 0.7997702953, 0.8958311401, 0.7726758273, 0.7584755404, 0.8685576448, 0.8755586013, 0.7861952862, 0.8390858607, 0.8451820128, 0.7809197365, 0.7352941176, 0.7541004974, 0.7980051128, 0.8612820323, 0.8472222222, 0.8732263251, 0.8316052175, 0.8244212864, 0.8451796596, 0.8743038758, 0.8162613257, 0.7564254062, 0.7720641984, 0.8224568138, 0.8635202945, 0.6673306773, 0.8823753526, 0.8319294722, 0.8118756477, 0.7518938202, 0.7828294939, 0.7447928331, 0.7352941176, 0.7855480491, 0.8586394726, 0.7787146531, 0.8758816966, 0.8066369083, 0.8773147333, 0.8821423389, 0.8486653157, 0.8637565478, 0.8698558363, 0.863297691, 0.8954665261, 0.8672943981, 0.6162671001, 0.867763689, 0.8401546157, 0.7848648649, 0.8269376267, 0.8697479527, 0.8050302163, 0.8534127389, 0.8743642607, 0.8948361522, 0.8021165318, 0.8605944441, 0.7824208304, 0.8121886416, 0.8745050193, 0.7565433462, 0.7, 0.8760645693, 0.8358317092, 0.8906723593, 0.7732347566, 0.7853185279, 0.7200750329, 0.8012658228, 0.7954792043, 0.8076923077, 0.7669066401, 0.8891394145, 0.8735310824, 0.8688328404, 0.8653660349, 0.7654617983, 0.8304509833, 0.8663739021, 0.8374217207, 0.7508511807, 0.7652813378, 0.7615384615, 0.800779459, 0.8640920946, 0.8347379566, 0.7535865757, 0.8888006069, 0.8024911032, 0.8627172196, 0.8603626612, 0.8671273461, 0.8233814433, 0.8699460119, 0.7715087986, 0.7, 0.8718947958, 0.7621819449, 0.80748422, 0.8391223156, 0.757236912, 0.8726664318, 0.7682098765, 0.7719728111, 0.8638155014, 0.7620967742, 0.8723212045, 0.7, 0.7544221044, 0.8209439528, 0.7489119363, 0.8318624599, 0.8327607931, 0.8220322825, 0.8659882086, 0.7799121844, 0.8716210826, 0.7813533773, 0.8807253939, 0.7599726482, 0.8118853923, 0.7839779006, 0.8567898556, 0.7112676056, 0.8659499862, 0.7145814439, 0.7331921173, 0.8554667998, 0.8353036718, 0.793948963, 0.7602375246, 0.6718637836, 0.7976765597, 0.7721469076, 0.7892035398, 0.7267161777, 0.8691058629, 0.7, 0.7, 0.8337978957, 0.7754553242, 0.6979305041, 0.8118756477, 0.7944230933, 0.7701962931, 0.7767929882, 0.8069870793, 0.861500341, 0.6553113553, 0.7745166045, 0.8169590643, 0.8840027378, 0.7721202003, 0.8898924507, 0.7, 0.7464265626, 0.8876453157, 0.761595824, 0.786175268, 0.7557670307, 0.7861384663, 0.7665909712, 0.8568973116, 0.7570700637, 0.8560950936, 0.8808036277, 0.8332209106, 0.8520545997, 0.6475467217, 0.7626296276, 0.7398818316, 0.8085605998, 0.8367723224, 0.8435639413, 0.885863102, 0.7917139614, 0.7872805883, 0.8866363193, 0.8817562574, 0.8509084168, 0.7972143937, 0.7874859076, 0.875750085, 0.7, 0.7912568622, 0.804558845, 0.749851279, 0.7534611289, 0.8692910127, 0.8370371889, 0.7718863403, 0.8376133841, 0.6833028641, 0.7838916818, 0.892239119, 0.7662544446, 0.8271641791, 0.7692249812, 0.8658471276, 0.8636645679, 0.7994678278, 0.901187447, 0.8723554302, 0.8300240964, 0.8361105627, 0.806304648, 0.8212832176, 0.7974322076, 0.7, 0.851589778, 0.8657882943, 0.8726540213, 0.7670511342, 0.8294994922, 0.843988551, 0.8738779429, 0.8129367328, 0.6251837136, 0.759393658, 0.7, 0.708650157, 0.7519088088, 0.7524778299, 0.7725, 0.8497793036, 0.7987532651, 0.6553113553, 0.7982156885, 0.8860625741, 0.8443187408, 0.8750133933, 0.8767021579, 0.793603255, 0.8095671028, 0.6926791099, 0.8524290996, 0.783954608, 0.8190615836, 0.7113516676, 0.7, 0.831581104, 0.8584562709, 0.7769169646, 0.8899754108, 0.6610498061, 0.7423309345, 0.8271641791, 0.8507140485, 0.8258086, 0.8903662296, 0.8307611784, 0.8464147703, 0.7903729853, 0.8818771273, 0.8739031763, 0.7352941176, 0.8729791634, 0.8745906019, 0.7551463441, 0.7856596384, 0.8772153819, 0.8715678305, 0.8636606263, 0.8751304513, 0.796, 0.8787721124, 0.8820584727, 0.8273968151, 0.7695145236, 0.734113603, 0.7731203578, 0.7753585398, 0.8840710634, 0.6863917914, 0.7524276072, 0.8559168105, 0.8121387283, 0.8551962689, 0.846741476, 0.6841717533, 0.7991881784, 0.871410249, 0.8844660822, 0.7590589813, 0.8013870077, 0.8678871711, 0.8752103958, 0.7, 0.7874859076, 0.7511425913, 0.8593834951, 0.8596434309, 0.7984339257, 0.8920142579, 0.8234796535, 0.8892329541, 0.7884984197, 0.8661259954, 0.7751351351, 0.7112676056, 0.7721298802, 0.8205623964, 0.7653914866, 0.7690671031, 0.82512336, 0.8826781327, 0.7727753945, 0.8237515506, 0.7837015329, 0.875582942, 0.8342866525, 0.6879194631, 0.878460073, 0.8104387807, 0.7, 0.8187136222, 0.8229527105, 0.8720974249, 0.7066666667, 0.8638640597, 0.858592834, 0.8080928283, 0.8412531564, 0.7590796257, 0.8581389233, 0.7287179487, 0.8117518667, 0.7, 0.721831515, 0.872795312, 0.8127225033, 0.887979568, 0.8435355534, 0.787790077, 0.7352941176, 0.894114556, 0.8596179409, 0.7793425357, 0.8526983837, 0.862902829, 0.7778549597, 0.8851640051, 0.8111030822, 0.8598676557, 0.7700866714, 0.8806747485, 0.8548990526, 0.8290566038, 0.8848737542, 0.7705891485, 0.8199016192, 0.8471695839, 0.8252225519, 0.8304241118, 0.8381011621, 0.8675833209, 0.8012457468, 0.8436562376, 0.7959623677, 0.7762773114, 0.7718365589, 0.7663035255, 0.7, 0.7970930734, 0.8626169923, 0.7651685393, 0.7, 0.7, 0.8549177, 0.7668111588, 0.8815698787, 0.8014013142, 0.8127122105, 0.8647843015, 0.774465299, 0.7909090909, 0.8622159091, 0.7996963744, 0.7699383785, 0.8204678363, 0.8229884783, 0.6519313305, 0.8791855786, 0.783193912, 0.8834809215, 0.7694390985, 0.8674573489, 0.7956701505, 0.8098232784, 0.8244877822, 0.7761581032, 0.7810871738, 0.8712136816, 0.8779108157, 0.8309111495, 0.8649321523, 0.743843303, 0.7826498423, 0.774571741, 0.8538918055, 0.8512265296, 0.8838737563, 0.8376774685, 0.6347387718, 0.6745454545, 0.8785520635, 0.8797023004, 0.784025024, 0.8578478269, 0.8543693084, 0.8547252747, 0.7909090909, 0.7832403481, 0.8391989024, 0.8252225519, 0.8719926395, 0.8837804004, 0.8104713376, 0.777238081, 0.8134202608, 0.7543233687, 0.8706813895, 0.7763272289, 0.8137500732, 0.7615384615, 0.8598660735, 0.7809977833, 0.7758251905, 0.8019910874, 0.8526345745, 0.7811589197, 0.8778150577, 0.8212938324, 0.8031796316, 0.7995705372, 0.8784264298, 0.820752905, 0.8204678363, 0.869503012, 0.7705152302, 0.8557271955, 0.7693797691, 0.8050271147, 0.7725029576, 0.6745454545, 0.88122809, 0.7, 0.8769442431, 0.8777064804, 0.7687238291, 0.858333707, 0.7308683497, 0.8418931584, 0.8587601671, 0.8514365726, 0.7693627549, 0.8756454251, 0.8896234201, 0.8932773109, 0.8746489564, 0.8820125858, 0.804634991, 0.7500849473, 0.7788502235, 0.8371502091, 0.8741689182, 0.8195844512, 0.804193849, 0.7757785467, 0.796, 0.8713485394, 0.8343613001, 0.8131709596, 0.8818041635, 0.7540973193, 0.8482550193, 0.8188369678, 0.8271641791, 0.8703272075, 0.8602167043, 0.8240338548, 0.6951375992, 0.7871491462, 0.8622444105, 0.7844913277, 0.866269891, 0.7826498423, 0.8608558061, 0.8715636224, 0.8504501002, 0.7861845168, 0.8739532198, 0.8765841883, 0.8509282013, 0.7600652884, 0.8775954624, 0.8221274868, 0.737037037, 0.781920559, 0.8712429938, 0.8431092425, 0.7552704303, 0.7549813973, 0.7544502618, 0.876564671, 0.8683844857, 0.8823413162, 0.7, 0.749851279, 0.7451371146, 0.7396682512, 0.8794033265, 0.8201360214, 0.8183580819, 0.7945988841, 0.8319553338, 0.8161329305, 0.8487179487, 0.8364238411, 0.8311412213, 0.7243006684, 0.7, 0.8592604707, 0.8751587544, 0.8258774553, 0.8237087165, 0.7967577118, 0.8234255409, 0.7843399252, 0.7600439547, 0.8459239004, 0.8672824702, 0.7188069695, 0.7178960097, 0.7882882883, 0.860230522, 0.8830072091, 0.7352941176, 0.7838216043, 0.8121387283, 0.6671543462, 0.8709254759, 0.8884946806, 0.7687639219, 0.7780487805, 0.8198157455, 0.7855677253, 0.7548629442, 0.8072143109, 0.8476492593, 0.8478382964, 0.8301459487, 0.8793992507, 0.7523485611, 0.7930107527, 0.7946337642, 0.7524778299, 0.8364222471, 0.7815964523, 0.8761589236, 0.7600652884, 0.7898830518, 0.7322898242, 0.8956569018, 0.8694904438, 0.8592561791, 0.7633996937, 0.8705619503, 0.8633567137, 0.8789168185, 0.7894446655, 0.8126693227, 0.8160572936, 0.770685889, 0.8717383252, 0.863944271, 0.788, 0.8863470844, 0.7780115937, 0.8412688822, 0.8503794147, 0.8759810056, 0.8426556711, 0.8531929177, 0.8336388156, 0.8837981408, 0.8548619027, 0.7893863682, 0.8058925577, 0.8550613683, 0.856976591, 0.7713187634, 0.859775775, 0.8733891059, 0.7715283774, 0.8328237792, 0.8807993929, 0.8085126286, 0.7, 0.6879194631, 0.8656256424, 0.8595494589, 0.7583461629, 0.8521017794, 0.8599384836, 0.8480514065, 0.8525097478, 0.8050577549, 0.8884435538, 0.8483724999, 0.8472753148, 0.6519313305, 0.8403389831, 0.8241825904, 0.7753864767, 0.8603825864, 0.7476190476, 0.7708575863, 0.7645499316, 0.8382978664, 0.7949482272, 0.8833263244, 0.7501646059, 0.8176275167, 0.8805280015, 0.8803521083, 0.8643062186, 0.8706659904, 0.8336082474, 0.7701171864, 0.9002149696, 0.8554054704, 0.6903646596, 0.8547261512, 0.6715171095, 0.7555029818, 0.6940804154, 0.8260162243, 0.7978123265, 0.8756130397, 0.788, 0.8766064491, 0.7930444698, 0.891385682, 0.8278706071, 0.8027932961, 0.877728974, 0.7995149393, 0.7712241303, 0.8146722164, 0.7451371146, 0.7572018779, 0.737037037, 0.8608592306, 0.8732082074, 0.859995052, 0.6803277215, 0.7552863041, 0.8604187656, 0.8113981639, 0.8665415313, 0.6879194631, 0.8351404959, 0.7875843319, 0.7902162364, 0.8557385609, 0.7702475075, 0.8121387283, 0.8641470147, 0.8205032919, 0.7, 0.80066704, 0.8190615836, 0.7641707675, 0.8012121988, 0.8831781681, 0.8518863137, 0.8520073595, 0.8665405581, 0.8594909862, 0.820295421, 0.8949992254, 0.8806686111, 0.8768353657, 0.8663220089, 0.8615924492, 0.801767172, 0.8501481373, 0.837890732, 0.8480452015, 0.8511371468, 0.8544101987, 0.767766359, 0.7233006024, 0.8509350968, 0.8763851861, 0.7709196355, 0.7781002267, 0.788, 0.9008924735, 0.8557383542, 0.7532526221, 0.8364281499, 0.8778708115, 0.8718226625, 0.7603505141, 0.8293413174, 0.8702755252, 0.8710943396, 0.6857559973, 0.8181369942, 0.8147715294, 0.7827608126, 0.7407948568, 0.7956990759, 0.7687347709, 0.8562620838, 0.845017162, 0.8190326914, 0.8216426983, 0.7758379306, 0.8670407633, 0.8169590643, 0.737037037, 0.8190615836, 0.737037037, 0.8620514647, 0.7192509572, 0.7574394464, 0.7, 0.8201877472, 0.872795584, 0.8864806678, 0.7636282008, 0.8452340306, 0.8076923077, 0.8536479161, 0.7729108307, 0.8264746773, 0.8745496388, 0.8827685018, 0.8434571154, 0.8618693551, 0.669020096, 0.8397260274, 0.8021165318, 0.7914410959, 0.8494330113, 0.8121387283, 0.7581333333, 0.8977841339, 0.8594278283, 0.8258003083, 0.8015553522, 0.835262757, 0.8169590643, 0.7615384615, 0.8625636751, 0.8890041494, 0.8050445477, 0.808272408, 0.770397094, 0.8434758756, 0.7760050828, 0.8689529765, 0.8335033146, 0.8583874632, 0.7746561279, 0.8643271309, 0.8423453663, 0.8632906325, 0.8257800398, 0.7829229949, 0.836239842, 0.8574071243, 0.820855615, 0.8605245502, 0.7878384057, 0.7579945799, 0.8245158632, 0.8796487109, 0.854662858, 0.7, 0.84757971, 0.8895935405, 0.7610114192, 0.7711195488, 0.8522118704, 0.7617953603, 0.6713242522, 0.8506423014, 0.7534611289, 0.8105951561, 0.836317432, 0.8802692541, 0.8357374308, 0.8071052265, 0.8717118258, 0.7686317062, 0.7947368421, 0.7178217822, 0.8491525424, 0.7, 0.7720178372, 0.8433946131, 0.8427584718, 0.803220423, 0.7608053493, 0.8937880254, 0.788, 0.8442970666, 0.7285714286, 0.8022054893, 0.7378088702, 0.821996008, 0.8310732858, 0.8434064711, 0.756, 0.8787244379, 0.8934810438, 0.8666526132, 0.8621325238, 0.761607075, 0.8823842254, 0.796, 0.8848840594, 0.8366745197, 0.8067456278, 0.8465470029, 0.8235365083, 0.7, 0.8719776892, 0.8933069391, 0.8951032165, 0.8232245901, 0.8572216097, 0.8199016192, 0.8453690137, 0.8867728959, 0.8012045493, 0.7535249501, 0.788, 0.8846793311, 0.8868076887, 0.8183553341, 0.862720403, 0.839743398, 0.8548983364, 0.8050388297, 0.8518432272, 0.8423728814, 0.739381239, 0.7378796112, 0.6344262295, 0.845363371, 0.6380030003, 0.8009430447, 0.7544502618, 0.8080930233, 0.8748025992, 0.7352941176, 0.8190728661, 0.8487601545, 0.7605135122, 0.7671999385, 0.8423737336, 0.8010875126, 0.8195146613, 0.7617953603, 0.775862069, 0.784743241, 0.8256625274, 0.6879194631, 0.7366583236, 0.7143620178, 0.7823046251, 0.762614492, 0.8734800123, 0.8069589092, 0.8663061172, 0.7909090909, 0.7574243036, 0.7565833333, 0.8774359229, 0.8328031323, 0.7721155831, 0.8735776431, 0.8325380061, 0.7711535851, 0.7930357443, 0.7749430844, 0.871475848, 0.7194174757, 0.7400304702, 0.8169590643, 0.7847131305, 0.7414523281, 0.862651587, 0.8177112742, 0.8636080386, 0.8762315896, 0.8891497131, 0.8361865167, 0.7723668756, 0.7112676056, 0.8118756477, 0.7930444698, 0.7, 0.8366277837, 0.8785938579, 0.8003557726, 0.8518685324, 0.7481076535, 0.7556824401, 0.8745774962, 0.8588863966, 0.8625032619, 0.8700380042, 0.7858689984, 0.7762547153, 0.8282279401, 0.8017390211, 0.8837580387, 0.8638948904, 0.7538450052, 0.7608298439, 0.788472884, 0.8697809453, 0.8716440422, 0.7, 0.8922140805, 0.8030491739, 0.7717935212, 0.8616882427, 0.8789042499, 0.8116710875, 0.8502320633, 0.7808089888, 0.8169194071, 0.8936023478, 0.8537515375, 0.8062197026, 0.7, 0.8857942061, 0.8372881356, 0.8798881261, 0.781062577, 0.6942280212, 0.8266695048, 0.7849668098, 0.8744364738, 0.7777422885, 0.6879194631, 0.7646859722, 0.8784263621, 0.8359607092, 0.8841203458, 0.7800356824, 0.8066177129, 0.8744072096, 0.794372444, 0.8909781793, 0.7594594595, 0.7760469657, 0.8807582631, 0.8753659768, 0.8080222399, 0.8204678363, 0.8095101864, 0.7778378378, 0.7544502618, 0.8126644575, 0.6901991094, 0.7726812557, 0.8087131748, 0.8593922044, 0.6519313305, 0.7, 0.8055191768, 0.8610283591, 0.859887075, 0.8539238801, 0.8807752131, 0.8594278283, 0.7, 0.8874010103, 0.8686593742, 0.888315357, 0.7690909091, 0.8617304896, 0.8881780078, 0.8147164839, 0.8316312819, 0.7809702412, 0.8467278033, 0.8384462317, 0.9179993218, 0.7352941176, 0.8804091638, 0.8662299854, 0.7724915129, 0.8581277772, 0.6855332773, 0.8532430121, 0.7822568447, 0.7708575863, 0.7849668098, 0.7287179487, 0.8542643214, 0.8139544513, 0.8408582371, 0.8252225519, 0.8478569108, 0.8935794079, 0.7, 0.7847131305, 0.737037037, 0.7913944918, 0.7112676056, 0.8208111683, 0.8331230665, 0.7543059994, 0.8003375378, 0.8047912681, 0.8711764428, 0.6488459396, 0.848949369, 0.8827071539, 0.8366852995, 0.8245548654, 0.8328841608, 0.8494023311, 0.6879194631, 0.8161906357, 0.7, 0.8591792275, 0.859958271, 0.8706124796, 0.8694158659, 0.7886689871, 0.7809561445, 0.7074098389, 0.8222741117, 0.76077082, 0.7112676056, 0.8648705982, 0.8627122385, 0.7, 0.8677809268, 0.8509751747, 0.6879194631, 0.7718549698, 0.8162335965, 0.8436634201, 0.7400397875, 0.8486474882, 0.724157269, 0.7877963126, 0.8620009653, 0.7, 0.8676626197, 0.8027932961, 0.847875895, 0.8791556862, 0.7287179487, 0.7743762841, 0.7, 0.7500968685, 0.827568728, 0.8760068335, 0.888644691, 0.6811420109, 0.7, 0.8840613027, 0.7610114192, 0.6879194631, 0.8736343006, 0.7592728852, 0.8536289748, 0.8136374269, 0.8748385688, 0.740353461, 0.8487689127, 0.7079609397, 0.7984714401, 0.8466417573, 0.8600577738, 0.7, 0.8688772032, 0.8636462942, 0.8359613328, 0.7476190476, 0.8010213556, 0.6319838057, 0.7835344828, 0.8233587276, 0.8088275989, 0.6745454545, 0.7745990134, 0.8646203405, 0.8650759358, 0.7389941028, 0.8794180441, 0.8741168047, 0.7, 0.788, 0.7, 0.8356258094, 0.9001920498, 0.788, 0.8727378064, 0.8625121612, 0.7, 0.8544328161, 0.7859063282, 0.77701349, 0.7190447216, 0.8334255417, 0.705991342, 0.7239907058, 0.7540291635, 0.8843525146, 0.8650598434, 0.8587601671, 0.8736433617, 0.7706892309, 0.8008356546, 0.8330758948, 0.8542784703, 0.8542765689, 0.7, 0.798313253, 0.7725958345, 0.8826693951, 0.8491766456, 0.8003991626, 0.8275217933, 0.7731600718, 0.7459563115, 0.7913170443, 0.8239400995, 0.7979555556, 0.788, 0.8056132305, 0.6879194631, 0.8436985161, 0.8065883888, 0.8623181612, 0.8445231252, 0.8667799903, 0.8131473631, 0.785888608, 0.8201892183, 0.8151747326, 0.7643345279, 0.7670511342, 0.8397088897, 0.809178475, 0.7640321328, 0.8460160483, 0.7760791367, 0.8087417916, 0.7619399311, 0.8360750426, 0.8004046737, 0.8438733569, 0.783296317, 0.8677122685, 0.7687438604, 0.8198654726, 0.7904026965, 0.8352487525, 0.7544502618, 0.8285547751, 0.8892885691, 0.8596658223, 0.8100377659, 0.8293413174, 0.8611231154, 0.8581176826, 0.7738627909, 0.7648143965, 0.7785560345, 0.6879194631, 0.696040146, 0.8884860911, 0.8127441036, 0.8773981357, 0.7744525928, 0.8676692447, 0.8637600194, 0.8223766183, 0.8572737277, 0.6823463687, 0.8337453646, 0.7724685949, 0.8889515509, 0.7965917248, 0.8367723224, 0.8203525313, 0.7562740508, 0.7150723835, 0.7509964127, 0.8145878451, 0.832735426, 0.8247049078, 0.7605405405, 0.7, 0.8809212194, 0.8671445799, 0.791032873, 0.7247136829, 0.7615384615, 0.8712826899, 0.860162917, 0.8450461018, 0.7, 0.8406524239, 0.7, 0.7716981132, 0.7651694095, 0.8001417314, 0.7, 0.7456858731, 0.8013711364, 0.8300979653, 0.8561967063, 0.7523398128, 0.8308413433, 0.6956315332, 0.8239193084, 0.7608298439, 0.8127718374, 0.7976093093, 0.7796737977, 0.8260890563, 0.8659425437, 0.8452412156, 0.7768133014, 0.7715283774, 0.8606701459, 0.8444258016, 0.7706521436, 0.7845088694, 0.7221531144, 0.7605405405, 0.8517314157, 0.8136697478, 0.7815613543, 0.8115354018, 0.8506078055, 0.7615384615, 0.6013760058, 0.8190615836, 0.8419327373, 0.8206531364, 0.7670511342, 0.7816619128, 0.6359118727, 0.8500428041, 0.7, 0.7598808879, 0.7861952862, 0.8121387283, 0.8462755599, 0.8436274375, 0.8188253987, 0.6803782615, 0.8275763359, 0.8479697469, 0.8460043926, 0.7805132444, 0.8426793861, 0.8070243493, 0.6319838057, 0.8703340659, 0.8626936571, 0.7533333333, 0.793114567, 0.8315301437, 0.7258064516, 0.8271641791, 0.8562885977, 0.8875333927, 0.8164639767, 0.8493190285, 0.7616239286, 0.7839915247, 0.7930444698, 0.7853166433, 0.8240990586, 0.8663323319, 0.8335007474, 0.6549731801, 0.8521017794, 0.6982176012, 0.775862069, 0.7697287523, 0.8135815571, 0.7602147806, 0.8565547964, 0.7522582944, 0.8131147541, 0.7765531062, 0.7815964523, 0.833897256, 0.5880302621, 0.8496823819, 0.8912233912, 0.8726389109, 0.6879194631, 0.6898353507, 0.8915508144, 0.8692547649, 0.8598401165, 0.8370983213, 0.789112734, 0.8578550419, 0.8555943783, 0.8599440674, 0.7782799944, 0.8674985527, 0.8119433994, 0.8193396226, 0.7538503784, 0.7839144708, 0.7461561858, 0.8104935711, 0.8932066989, 0.78579947, 0.7896043226, 0.8190526899, 0.8285581395, 0.7393544382, 0.7, 0.6366238894, 0.8257970145, 0.8463399002, 0.8649169169, 0.8163461263, 0.7448736999, 0.8580412749, 0.8794416653, 0.737037037, 0.707803736, 0.8546827795, 0.8760833192, 0.8090223489, 0.6549731801, 0.8619832398, 0.7800356824, 0.879298992, 0.8120278908, 0.8605194337, 0.8256150808, 0.7, 0.8259254973, 0.7, 0.7793425357, 0.8378222115, 0.7722316865, 0.7646295762, 0.7112676056, 0.8449249074, 0.7708575863, 0.8704719404, 0.851904176, 0.8836759163, 0.8510037476, 0.7276713178, 0.7534611289, 0.8042737993, 0.8090535562, 0.7931374607, 0.7753805642, 0.6879194631, 0.7534611289, 0.8811656303, 0.7553160607, 0.7970632552, 0.6239432527, 0.8271641791, 0.7844138337, 0.7979735683, 0.737037037, 0.8355114649, 0.776843712, 0.7815964523, 0.8204678363, 0.816092258, 0.8805264763, 0.8717369971, 0.7760582158, 0.7693101399, 0.7690514987, 0.8300596604, 0.7805230369, 0.7352941176, 0.7936138628, 0.7651251353, 0.7398818316, 0.8646275898, 0.8114851035, 0.8722747122, 0.769287721, 0.7811408357, 0.8172625476, 0.8131548586, 0.7458381884, 0.8004134963, 0.8121387283, 0.8662481263, 0.737037037, 0.78828487, 0.7905379594, 0.8771794171, 0.8516224189, 0.8889898677, 0.7777076925, 0.8543999752, 0.6879194631, 0.7875717558, 0.7, 0.8243909644, 0.8808579221, 0.8288930412, 0.7705056743, 0.7994678278, 0.799628493, 0.8722647031, 0.7571630024, 0.7, 0.7390865246, 0.8146129281, 0.7935407215, 0.8455930881, 0.8693726475, 0.8901106433, 0.7979929319, 0.810407918, 0.8034421027, 0.8414305929, 0.5870792565, 0.8832465615, 0.7352941176, 0.8770147782, 0.8542380325, 0.85115581, 0.6745454545, 0.796, 0.8161094225, 0.7407948568, 0.6606588059, 0.6228017462, 0.7595088822, 0.8446399623, 0.8608662169, 0.8262147538, 0.8333333333, 0.7524778299, 0.8538791256, 0.8548731326, 0.7780487805, 0.7831525782, 0.8592035409, 0.7568324856, 0.8238206104, 0.737037037, 0.8916936248, 0.7766117834, 0.8572786993, 0.8696644427, 0.7618731118, 0.8831522268, 0.7996336996, 0.8654270101, 0.8960220151, 0.8542784703, 0.6700780409, 0.7398818316, 0.849749031, 0.8126693227, 0.8666507968, 0.7778549597, 0.8330926594, 0.7723633322, 0.7844838687, 0.858083959, 0.8687688239, 0.8710431457, 0.7550942993, 0.7779688525, 0.8704244163, 0.798469847, 0.8031313542, 0.808542099, 0.8370087403, 0.8575966202, 0.7579945799, 0.6319838057, 0.8901174088, 0.889435065, 0.8312264861, 0.8657682496, 0.8582192201, 0.8733890545, 0.8056137899, 0.6866449832, 0.8699405424, 0.6856716418, 0.7816562252, 0.8381328212, 0.6905183872, 0.8109261836, 0.7814122086, 0.7823648194, 0.7711091626, 0.876123397, 0.7649307796, 0.7873760408, 0.8778562533, 0.761595824, 0.8703474713, 0.862454909, 0.8101842651, 0.8620948242, 0.7112676056, 0.7, 0.8737334889, 0.7605405405, 0.8494330113, 0.8285604186, 0.6879194631, 0.6815057737, 0.8366277837, 0.8095952577, 0.845766692, 0.7414203166, 0.7558663276, 0.8540891705, 0.7909090909, 0.7615384615, 0.7813843462, 0.8014013142, 0.7352941176, 0.81946003, 0.690260777, 0.8108041441, 0.7533644661, 0.7696019831, 0.7352941176, 0.8820794536, 0.8392725666, 0.8695952957, 0.7, 0.8183216306, 0.7945770254, 0.7, 0.8424687089, 0.8332209106, 0.8207439708, 0.8730085902, 0.8525420278, 0.7352941176, 0.8680334691, 0.8744319218, 0.8131147541, 0.7485875706, 0.8651194795, 0.788, 0.8247717027, 0.8007488892, 0.8460043926, 0.8726125254, 0.7279956535, 0.851386456, 0.6931225171, 0.7815964523, 0.7352941176, 0.8803453947, 0.8137885379, 0.785340526, 0.7898876404, 0.8109381663, 0.8750156496, 0.7719888406, 0.7851430329, 0.8496823819, 0.8163751405, 0.7909090909, 0.7754996602, 0.740115918, 0.8343057682, 0.7254738532, 0.690260777, 0.8278026906, 0.7836327324, 0.8007230094, 0.8389691687, 0.7, 0.7800480702, 0.8704290552, 0.8044915955, 0.8666820123, 0.6745454545, 0.7823071694, 0.8656189983, 0.8267663982, 0.885265724, 0.8558882236, 0.7569405028, 0.8391099887, 0.854154745, 0.8460053068, 0.8568080619, 0.768360864, 0.7937990773, 0.6879194631, 0.7503660624, 0.6793939394, 0.8110871075, 0.7823256518, 0.8919989709, 0.8829614604, 0.6556420233, 0.8900162417, 0.7616216216, 0.7641386782, 0.7969480388, 0.6944936162, 0.8494330113, 0.6344262295, 0.8681886494, 0.8433962264, 0.6698767376, 0.6879194631, 0.7759711715, 0.7615384615, 0.7793689116, 0.7651974347, 0.7876663312, 0.7073511558, 0.8990494108, 0.8715495811, 0.6947704988, 0.6831243469, 0.8092224231, 0.6867222814, 0.8529102845, 0.7953255503, 0.7374145891, 0.7799822468, 0.7721606762, 0.8969230567, 0.7, 0.7713145413, 0.8660527456, 0.7604554249, 0.830634715, 0.7554154543, 0.7, 0.7352941176, 0.8629161147, 0.788, 0.8342356975, 0.7859113735, 0.7579945799, 0.7687919374, 0.737037037, 0.7825141354, 0.7831201709, 0.858603587, 0.8551695257, 0.78010278, 0.8691298428, 0.8861490372, 0.7834809395, 0.7578209277, 0.834476248, 0.7472590253, 0.7, 0.8106613638, 0.7352941176, 0.8169590643, 0.8579656863, 0.8548983364, 0.788, 0.7570767468, 0.8160194991, 0.8059807074, 0.7, 0.780883197, 0.8916640142, 0.7582092639, 0.849974735, 0.7556843194, 0.9051863966, 0.8294460982, 0.7725958345, 0.8109465117, 0.8605872274, 0.8472222222, 0.8227244212, 0.7544502618, 0.7788735695, 0.8688812363, 0.8677800674, 0.8274104407, 0.8313544584, 0.7, 0.5759966936, 0.7407948568, 0.7, 0.8804013903, 0.7815964523, 0.800282006, 0.8028823222, 0.775862069, 0.7864145234, 0.8603109425, 0.8006776379, 0.796, 0.7754957526, 0.7568163973, 0.8690771209, 0.7520292121, 0.8184468253, 0.7, 0.788, 0.860087146, 0.7, 0.8854313681, 0.8908909596, 0.8620715417, 0.7496762858, 0.7616216216, 0.808498545, 0.8621519065, 0.6016641452, 0.768387276, 0.8052631579, 0.8514733209, 0.7916572684, 0.7615384615, 0.7, 0.7851430329, 0.7636586969, 0.7640144039, 0.8886683743, 0.8786227134, 0.7621386091, 0.7505097582, 0.7862407327, 0.8207840825, 0.7183584401, 0.8315691736, 0.7750098078, 0.6963562, 0.7085185366, 0.7840608035, 0.7107509881, 0.8683287397, 0.7956882475, 0.7947658402, 0.7570166936, 0.7930444698, 0.8579454153, 0.7514890801, 0.8388034388, 0.7, 0.8686665413, 0.8312825367, 0.7, 0.7862865948, 0.7682094295, 0.8651152607, 0.8718612604, 0.8005927653, 0.796, 0.7, 0.6961402258, 0.7995705372, 0.8236047575, 0.7869306154, 0.7366420535, 0.7821128707, 0.8506078055, 0.6792746114, 0.6362808843, 0.8863912894, 0.8585921566, 0.749851279, 0.8148927688, 0.6180391359, 0.8538935454, 0.8529102845, 0.8637269616, 0.8610921502, 0.8126693227, 0.8496784155, 0.7905483214, 0.8311183096, 0.7562784915, 0.7780115937, 0.6803999254, 0.8708290269, 0.8000088865, 0.8448546012, 0.7716659565, 0.867976818, 0.8494487781, 0.6948282516, 0.8681453108, 0.6162671001, 0.7643471526, 0.8236008928, 0.7352941176, 0.8419312459, 0.8305910165, 0.6276300941, 0.8616451516, 0.7563803714, 0.8400565504, 0.7587887104, 0.8733085359, 0.8008943734, 0.8066753713, 0.8571930811, 0.8121387283, 0.7585084202, 0.8154990501, 0.8719642602, 0.7524778299, 0.8718205341, 0.8732361331, 0.8913815951, 0.6952710691, 0.8796380234, 0.7269861286, 0.8725811258, 0.6319838057, 0.6951105567, 0.6756831987, 0.8338429144, 0.7995321112, 0.8737626263, 0.8645771981, 0.8363924579, 0.7352941176, 0.8517179617, 0.8229009453, 0.7234444926, 0.7, 0.8209439528, 0.8531013258, 0.833473261, 0.7352941176, 0.9002971964, 0.6879194631, 0.6879194631, 0.8459567451, 0.8669600456, 0.7436156395, 0.8796851193, 0.7, 0.8449743259, 0.8321447801, 0.8459761817, 0.7570885274, 0.8801731954, 0.788, 0.7984714401, 0.7, 0.7525783424, 0.8412957353, 0.8417791644, 0.8162966066, 0.8649397336, 0.7882882883, 0.8655120963, 0.7524219618, 0.8505179302, 0.8614605418, 0.7585992712, 0.7655444999, 0.7510458124, 0.8734490771, 0.8247044882, 0.7548347613, 0.7615384615, 0.8576943384, 0.7984436883, 0.8368954899, 0.6903646596, 0.8406752412, 0.8494672755, 0.7112676056, 0.8580655363, 0.8630985209, 0.7581566421, 0.7588598078, 0.6879194631, 0.8310439143, 0.8676514473, 0.7846569436, 0.7799143775, 0.860087146, 0.8515095372, 0.8679638232, 0.7864169257, 0.7804952137, 0.8437146444, 0.8558093565, 0.7959623677, 0.868811889, 0.8548983364, 0.7586500635, 0.7799143775, 0.7810087822, 0.8301271676, 0.6921505833, 0.8228060893, 0.8612812058, 0.7887858919, 0.7450105579, 0.6792746114, 0.7775899632, 0.6919950681, 0.8594195739, 0.7946337642, 0.8412319007, 0.788, 0.8558093565, 0.751712197, 0.796, 0.8751430004, 0.7706332957, 0.6519313305, 0.7984714401, 0.8532600127, 0.9018255746, 0.7249362085, 0.7, 0.847754253, 0.8525013606, 0.8448865466, 0.8585523481, 0.6007673154, 0.7716839594, 0.6931889731, 0.8520846638, 0.8592761382, 0.8207600681, 0.8893913155, 0.8717268299, 0.8800021879, 0.7857347096, 0.6576670504, 0.853982035, 0.8572139904, 0.7112676056, 0.6879194631, 0.7649961383, 0.8939745075, 0.7, 0.6549731801, 0.8011188549, 0.7400080947, 0.8384462317, 0.8175186674, 0.7352941176, 0.796, 0.8490264661, 0.849630274, 0.8756886119, 0.7768195275, 0.8212313623, 0.7773085952, 0.7, 0.8314801861, 0.7451371146, 0.772470486, 0.7552214046, 0.8085121853, 0.8416121547, 0.7605405405, 0.7778549597, 0.874193268, 0.8265240396, 0.7652020421, 0.737037037, 0.8371886776, 0.8778035352, 0.7733151539, 0.7889018652, 0.7544502618, 0.8379940877, 0.8485653809, 0.7749928092, 0.8247874655, 0.7620574296, 0.8331721983, 0.6019517796, 0.8312615955, 0.7547770701, 0.904070484, 0.8529102845, 0.6355412186, 0.8787028971, 0.7, 0.7819150645, 0.788, 0.8587143424, 0.8523684291, 0.8625874126, 0.6913455037, 0.7636740061, 0.7615384615, 0.8146722164, 0.8107273847, 0.7832895888, 0.8077286681, 0.8178661761, 0.7352941176, 0.7376470588, 0.7881718086, 0.782708621, 0.7722040513, 0.8450090648, 0.7980013899, 0.7467831192, 0.8600697038, 0.8511371468, 0.6879194631, 0.6963335305, 0.856685534, 0.6879194631, 0.6879194631, 0.6950224381, 0.7778378378, 0.7799143775, 0.6215478167, 0.6879194631, 0.8846206096, 0.8153306803, 0.6715836547, 0.7469823867, 0.8247838449, 0.8309698381, 0.8670554044, 0.8285351788, 0.8019493848, 0.8756886119, 0.7882174732, 0.6951423677, 0.7, 0.6953272761, 0.8622338636, 0.7352941176, 0.7496804985, 0.8983937441, 0.7112676056, 0.6319838057, 0.7579945799, 0.8708478787, 0.6827893175, 0.8161625051, 0.8500496947, 0.800589391, 0.6879194631, 0.7838172072, 0.8644986834, 0.8567940111, 0.6319838057, 0.8552911687, 0.6165551479, 0.7620574296, 0.8506702893, 0.7128462226, 0.7896901808, 0.8463512895, 0.7814673654, 0.8625874126, 0.8395871751, 0.8677283592, 0.7450105579, 0.7799121844, 0.7909090909, 0.8715067532, 0.8498566651, 0.8003875517, 0.7971664064, 0.705991342, 0.6712523323, 0.7827037423, 0.7997270246, 0.7928863862, 0.860045539, 0.8323187635, 0.7978029067, 0.7, 0.6319838057, 0.7950228902, 0.8729868841, 0.8229399643, 0.732282097, 0.8267118644, 0.7, 0.851253386, 0.7555603102, 0.8684040705, 0.6592536294, 0.6474182105, 0.8263306309, 0.7890395383, 0.7569938695, 0.8008356546, 0.7800356824, 0.8247292818, 0.7655304464, 0.7652123719, 0.8394408721, 0.7, 0.883268717, 0.7578056665, 0.7438507999, 0.7, 0.8710242845, 0.8587143424, 0.8121886416, 0.6251504118, 0.8421093697, 0.8120278908, 0.6556420233, 0.7352941176, 0.8845791563, 0.8511371468, 0.7458577406, 0.8114500959, 0.8506801961, 0.849630274, 0.8305911388, 0.8407595521, 0.7585713843, 0.7721084706, 0.7557768661, 0.752395466, 0.8716658525, 0.6808047539, 0.8644493786, 0.7534382767, 0.6876758999, 0.7352941176, 0.8741824, 0.8101152509, 0.7, 0.7589174266, 0.7247568523, 0.788, 0.7914026893, 0.6319838057, 0.8854284444, 0.6572963605, 0.575776001, 0.7832633782, 0.8199840107, 0.6951698113, 0.9190932247, 0.788, 0.693236715, 0.7938969138, 0.8376783813, 0.8270048615, 0.7607468328, 0.7, 0.7, 0.8728569831, 0.6810411594, 0.8384775725, 0.7930364458, 0.7725463415, 0.7352941176, 0.8494672755, 0.7858897418, 0.788, 0.788, 0.8048276618, 0.7719672008, 0.6879194631, 0.7093735385, 0.7793578072, 0.5870792565, 0.757986374, 0.8587143424, 0.8363924579, 0.8306043122, 0.6288004508, 0.874656339, 0.7725463415, 0.7, 0.7, 0.8198028507, 0.5870792565, 0.788, 0.7518618301, 0.8834105945, 0.7, 0.6192344959, 0.7525455813, 0.7557840787, 0.6556420233, 0.7725463415, 0.7721770071, 0.7931820569, 0.772325629, 0.7, 0.7414216393, 0.6180487805, 0.556618548]

sort5 = list(np.argsort(sp5))
sp5= [sp5[i] for i in sort5] 
fid5= [fid5[i] for i in sort5]

data2 = list(zip(sp2, fid2))
data3 = list(zip(sp3, fid3))
data4 = list(zip(sp4, fid4))
data5 = list(zip(sp5, fid5))


def monotone(data):
    init_f = data[0][1]
    monotone_sequence = []
    for i, tup in enumerate(reversed(data)):
        if i > 0:
            if tup[1] >= prev:
                monotone_sequence.append(tup)
                prev = tup[1]
        else:
            prev = tup[1]
            monotone_sequence.append(tup)
    return monotone_sequence

data2_mono = monotone(data2)
data3_mono = monotone(data3)
data4_mono = monotone(data4)
data5_mono = monotone(data5)

from sage.plot.scatter_plot import ScatterPlot

rgbcolors2=[(0+x/(4-1), 1-x/(4-1),  1-x/(4-1)   ) for x in range(4)]

g = Graphics()
g += scatter_plot(data2, marker=".", facecolor=rgbcolors2[0], alpha=0.25, edgecolor='None', markersize=25)
g += scatter_plot(data3, marker=".", facecolor=rgbcolors2[1], alpha=0.2, edgecolor='None', markersize=25)
g += scatter_plot(data4, marker=".", facecolor=rgbcolors2[2], alpha=0.1, edgecolor='None', markersize=25)
g += scatter_plot(data5, marker=".", facecolor=rgbcolors2[3], alpha=0.015, edgecolor='None', markersize=20)

g += list_plot(data2_mono, plotjoined=True, color=rgbcolors2[0], linestyle='steps-post', thickness=2, legend_label='n=2')
g += list_plot(data3_mono, plotjoined=True, color=rgbcolors2[1], linestyle='steps-post', thickness=2, legend_label='n=3')
g += list_plot(data4_mono, plotjoined=True, color=rgbcolors2[2], linestyle='steps-post', thickness=2, legend_label='n=4')
g += list_plot(data5_mono, plotjoined=True, color=rgbcolors2[3], linestyle='steps-post', thickness=2, legend_label='n=5')

g.set_axes_range(xmin=0.185, xmax=0.86, ymin=0.69, ymax=0.93)
g.set_legend_options(loc='upper right');
# save(g,'scatterplot.pdf',axes=True, axes_labels=[r'$p_\textrm{suc}$', r'$F_\textrm{out}$'],axes_labels_size=1.6, legend_font_size=16)
save(g,'scatterplot2.pdf',axes=True, axes_labels=[r'$F_\textrm{in}$','Rate'],axes_labels_size=1.6, legend_font_size=16)